In [4]:
import sys
import os
from dotenv import load_dotenv

# 1. 환경 변수 로드 (.env 파일 읽기)
load_dotenv()

# (참고) 만약 노트북 파일을 'notebooks' 같은 하위 폴더에 만들었다면 
# 아래 주석을 풀어서 프로젝트 루트 경로를 추가해줘야 합니다.
# sys.path.append(os.path.abspath(".."))

# 2. 우리가 만든 함수 가져오기
# 수정을 했을 때 자동으로 다시 불러오게(Reload) 하는 매직 커맨드입니다.
%load_ext autoreload
%autoreload 2

try:
    from src.tools import get_detailed_route
    print("✅ 함수 불러오기 성공!")
except ImportError as e:
    print(f"❌ 오류 발생: {e}")

❌ 오류 발생: cannot import name 'get_detailed_route' from 'src.tools' (/Users/seoungmun/Documents/work/3-2/project/travle_agent4/src/tools.py)


In [34]:
# [TEST 1] 구글 맵 경로 탐색 테스트
from src.tools import get_detailed_route

# 1. 가까운 거리 (버스/지하철)
print("--- 🚌 단거리 이동 테스트 ---")
short_route = get_detailed_route("서울역", "N서울타워", mode="transit")
if short_route:
    print(f"소요시간: {short_route['duration']}")
    print(f"이동경로: {short_route['steps']}")
else:
    print("❌ 단거리 경로 탐색 실패")

# 2. 먼 거리 (복합 환승)
print("\n--- 🚇 장거리 이동 테스트 ---")
long_route = get_detailed_route("홍대입구역", "롯데월드", mode="transit")
if long_route:
    print(f"소요시간: {long_route['duration']}")
    print(f"이동경로: {long_route['steps']}")
else:
    print("❌ 장거리 경로 탐색 실패")

--- 🚌 단거리 이동 테스트 ---
소요시간: 21분
이동경로: ['[BUS] 402', '🚶 도보 8분']

--- 🚇 장거리 이동 테스트 ---
소요시간: 49분
이동경로: ['[SUBWAY] 2호선', '[SUBWAY] 2호선', '🚶 도보 5분']


In [1]:
# [TEST 2] RAG 검색(Retriever) 테스트
from src.tools import search_attractions_and_reviews

# 1. 구체적인 검색어 테스트
query = "부산 해운대 근처 오션뷰 카페 추천해줘"
print(f"🔎 검색 질문: {query}")

# 도구 실행 (내부적으로 FAISS 검색 -> LLM 요약 과정을 거침)
# 주의: 처음 실행 시 DB 로딩 때문에 시간이 조금 걸릴 수 있습니다.
result = search_attractions_and_reviews.invoke(query)

print("\n[검색 결과]")
print(result)

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


🔎 검색 질문: 부산 해운대 근처 오션뷰 카페 추천해줘


ValidationError: 1 validation error for search_attractions_and_reviews
destination
  Field required [type=missing, input_value={'query': '부산 해운... 카페 추천해줘'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [1]:
# Jupyter Notebook (디버깅용)

import sys
import os

# 경로 설정
if 'src' not in os.listdir('.') and 'src' not in sys.path:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

from src.config import load_faiss_index
from src.search import RegionPreFilteringRetriever

print("🚀 검색 함수 디버깅 시작\n")

# DB 로드
try:
    db = load_faiss_index()
    print("✅ DB 로드 성공")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    exit()

def test_search(query, location):
    print(f"\n🔎 검색 테스트: '{query}' (지역: {location})")
    
    retriever = RegionPreFilteringRetriever(
        vectorstore=db, 
        k=3, 
        fixed_location=location
    )
    
    docs = retriever.invoke(query)
    
    if not docs:
        print("   -> 결과 없음")
        return

    for i, doc in enumerate(docs):
        # 메타데이터에서 지역 정보 추출해 출력
        meta = doc.metadata
        region_info = meta.get('지역') or meta.get('주소') or meta.get('addr') or "정보없음"
        print(f"   {i+1}. [{doc.page_content[:20]}...] (지역정보: {region_info})")

# --- 테스트 케이스 ---

# 1. 부산 송정 맛집 (강원도가 나오면 실패)
test_search("송정 맛집", "부산")

# 2. 서울 카페 (서울특별시 매칭 확인)
test_search("조용한 카페", "서울특별시")

# 3. 엉뚱한 검색 (제주도에서 경복궁 찾기 -> 결과 없어야 정상)
test_search("경복궁", "제주도")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-04 22:47:27.067 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:47:27.097 
  command:

    streamlit run

🚀 검색 함수 디버깅 시작



2025-12-04 22:47:27.607 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:47:27.608 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:47:27.613 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:47:36.136 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:47:36.137 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:47:36.138 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ DB 로드 성공

🔎 검색 테스트: '송정 맛집' (지역: 부산)
   1. [지역: 부산광역시 서구 | 장소명: ...] (지역정보: 부산광역시 서구)

🔎 검색 테스트: '조용한 카페' (지역: 서울특별시)
   1. [지역: 서울특별시 성북구 | 장소명:...] (지역정보: 서울특별시 성북구)
   2. [지역: 서울특별시 노원구 | 장소명:...] (지역정보: 서울특별시 노원구)
   3. [지역: 서울특별시 금천구 | 장소명:...] (지역정보: 서울특별시 금천구)

🔎 검색 테스트: '경복궁' (지역: 제주도)
   -> 결과 없음


In [1]:
import random
import pandas as pd
from tqdm import tqdm
import sys
import os
import time

# [설정] 프로젝트 루트 경로 추가
current_dir = os.getcwd()
if 'src' not in os.listdir('.') and 'src' not in sys.path:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

try:
    from src.config import load_faiss_index
    from src.search import RegionPreFilteringRetriever
    from src.region_cut_fuzz import normalize_region_name
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")

# --- 1. 테스트 데이터 생성기 (기존 코드 활용) ---
def generate_test_dataset(num_samples=50):
    """50개의 랜덤 지역별 질문 생성 (비교를 위해 샘플 수 조정 가능)"""
    regions = [
        "서울", "부산", "제주", "강릉", "여수", "경주", "전주", "대구", "대전", 
        "춘천", "속초", "포항", "거제", "광주", "울산", "수원", "가평", "양평"
    ]
    templates = [
        "{region} 맛집 추천해줘", "{region} 예쁜 카페", "{region} 가볼만한 곳", 
        "{region} 아이랑 갈만한 곳", "{region} 핫플레이스"
    ]
    
    dataset = []
    for _ in range(num_samples):
        region = random.choice(regions)
        query = random.choice(templates).format(region=region)
        target_region = normalize_region_name(region) # 예: 부산 -> 부산광역시
        
        dataset.append({
            "query": query,
            "target_region_full": target_region, # 정규화된 이름 (필터링용)
            "target_region_short": region # 짧은 이름 (검증용)
        })
    return dataset

# --- 2. 정답 판별 헬퍼 함수 ---
def is_correct_region(doc, target_full, target_short):
    """문서의 메타데이터가 목표 지역과 일치하는지 판별"""
    doc_region = str(doc.metadata.get('지역') or doc.metadata.get('addr') or "")
    
    # 1. 광주광역시 vs 경기도 광주 구분이 제일 중요
    if target_short == "광주" and target_full == "광주광역시":
        if "경기도" in doc_region: return False
        
    # 2. 고성 (강원 vs 경남) 구분
    if target_short == "고성":
        if "강원" in target_full and "경남" in doc_region: return False
        if "경남" in target_full and "강원" in doc_region: return False

    # 3. 일반적인 포함 여부 확인
    # (문서 지역명에 '부산' 혹은 '부산광역시'가 포함되어 있는지)
    clean_target = target_full.replace("특별시", "").replace("광역시", "").replace("특별자치시", "").replace("특별자치도", "").replace("도", "")
    
    if clean_target in doc_region or target_short in doc_region:
        return True
    return False

# --- 3. 비교 실행 함수 ---
def run_comparison():
    print("🔄 FAISS 인덱스 로딩 중...")
    db = load_faiss_index()
    
    test_data = generate_test_dataset(50) # 50개 테스트
    results = []
    
    print(f"🚀 [Baseline vs Current] 성능 비교 시작 (총 {len(test_data)}건)...")
    print("-" * 60)

    total_baseline_accuracy = 0
    total_current_accuracy = 0

    for item in tqdm(test_data):
        query = item['query']
        target_full = item['target_region_full']
        target_short = item['target_region_short']
        
        # ---------------------------------------------------------
        # A. [Baseline] 기존 랭체인 기본 검색 (유사도 기반)
        # ---------------------------------------------------------
        # 단순히 쿼리와 유사한 문서를 가져옴. 지역 필터링 없음.
        baseline_docs = db.similarity_search(query, k=5)
        
        baseline_score = 0
        baseline_regions = []
        for doc in baseline_docs:
            r_name = doc.metadata.get('지역', 'Unknown')[:10] # 너무 길면 자름
            baseline_regions.append(r_name)
            if is_correct_region(doc, target_full, target_short):
                baseline_score += 1
        
        baseline_acc = (baseline_score / 5) * 100 # 5개 중 몇 개나 맞았는지 (%)

        # ---------------------------------------------------------
        # B. [Current] 현재 구현한 선필터링 검색 (Fetch & Filter)
        # ---------------------------------------------------------
        retriever = RegionPreFilteringRetriever(
            vectorstore=db,
            k=5,
            fixed_location=target_full # 목적지 강제 지정
        )
        try:
            current_docs = retriever.invoke(query)
        except:
            current_docs = []

        current_score = 0
        current_regions = []
        for doc in current_docs:
            r_name = doc.metadata.get('지역', 'Unknown')[:10]
            current_regions.append(r_name)
            if is_correct_region(doc, target_full, target_short):
                current_score += 1
        
        # 결과가 0개면 정확도 0 처리 (혹은 필터링 성공으로 볼 수도 있으나 여기선 검색 실패로 간주)
        if len(current_docs) > 0:
            current_acc = (current_score / len(current_docs)) * 100
        else:
            current_acc = 0 # 너무 빡빡해서 결과가 없으면 0점

        # 통계 누적
        total_baseline_accuracy += baseline_acc
        total_current_accuracy += current_acc

        # ---------------------------------------------------------
        # 결과 기록 (차이가 나는 경우만 로그에 강조)
        # ---------------------------------------------------------
        note = ""
        if current_acc > baseline_acc:
            note = "✅ 개선됨"
        elif current_acc < baseline_acc:
            note = "🔻 성능하락"
        
        results.append({
            "Query": query,
            "Target": target_short,
            "Baseline_Regions": str(baseline_regions), # 문자열로 변환해 보기 좋게
            "Baseline_Acc": baseline_acc,
            "Current_Regions": str(current_regions),
            "Current_Acc": current_acc,
            "Note": note
        })

    # --- 4. 결과 리포트 출력 ---
    df = pd.DataFrame(results)
    
    avg_base = total_baseline_accuracy / len(test_data)
    avg_curr = total_current_accuracy / len(test_data)

    print("\n" + "="*60)
    print(f"📊 최종 비교 리포트")
    print(f"🔹 기존 방식(Baseline) 평균 지역 정확도: {avg_base:.2f}%")
    print(f"🔸 현재 방식(Current)  평균 지역 정확도: {avg_curr:.2f}%")
    print(f"📈 성능 향상 폭: +{avg_curr - avg_base:.2f}%p")
    print("="*60)

    # 특히 개선된 케이스 상위 5개 출력
    improved = df[df['Note'] == "✅ 개선됨"].head(10)
    if not improved.empty:
        print("\n[✨ 주요 개선 사례: 엉뚱한 지역이 제거된 경우]")
        # 보기 좋게 컬럼 선택
        print(improved[['Query', 'Baseline_Regions', 'Current_Regions']].to_string(index=False))
    
    return df

# 실행
if __name__ == "__main__":
    df_result = run_comparison()

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-04 22:50:49.447 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:50:49.489 
  command:

    streamlit run

🔄 FAISS 인덱스 로딩 중...


2025-12-04 22:50:49.999 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:50:50.000 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:50:50.001 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:50:58.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:50:58.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:50:58.374 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


🚀 [Baseline vs Current] 성능 비교 시작 (총 50건)...
------------------------------------------------------------


100%|██████████| 50/50 [00:19<00:00,  2.55it/s]


📊 최종 비교 리포트
🔹 기존 방식(Baseline) 평균 지역 정확도: 91.20%
🔸 현재 방식(Current)  평균 지역 정확도: 100.00%
📈 성능 향상 폭: +8.80%p

[✨ 주요 개선 사례: 엉뚱한 지역이 제거된 경우]
     Query                                                       Baseline_Regions                                                        Current_Regions
대구 맛집 추천해줘          ['경상북도 안동시', '대구광역시 남구', '경상북도 안동시', '대구광역시 수성구', '대구광역시 남구']          ['대구광역시 남구', '대구광역시 수성구', '대구광역시 남구', '대구광역시 중구', '대구광역시 남구']
전주 맛집 추천해줘     ['전북특별자치도 전주', '경상북도 울릉군', '경상북도 울릉군', '전북특별자치도 전주', '강원특별자치도 양구'] ['전북특별자치도 전주', '전북특별자치도 전주', '전북특별자치도 전주', '전북특별자치도 전주', '전북특별자치도 전주']
  가평 핫플레이스              ['대구광역시 달성군', '경기도 가평군', '경기도 화성시', '경기도 가평군', '경기도 가평군']                ['경기도 가평군', '경기도 가평군', '경기도 가평군', '경기도 가평군', '경기도 가평군']
  속초 핫플레이스 ['강원특별자치도 속초', '강원특별자치도 속초', '강원특별자치도 속초', '강원특별자치도 속초', '강원특별자치도 양구'] ['강원특별자치도 속초', '강원특별자치도 속초', '강원특별자치도 속초', '강원특별자치도 속초', '강원특별자치도 속초']
  울산 예쁜 카페       ['강원특별자치도 속초', '울산광역시 중구', '울산광역시 울주군', '울산광역시 북구', '울산광역시 울주군']         ['울산광역시 중구

In [3]:
import pandas as pd
import os

# ⚠️ 파일 경로를 본인의 실제 파일명으로 수정하세요!
# 예: "new_reviews.csv" 또는 "data/place_data.csv"
csv_file_path = "/Users/seoungmun/Documents/work/3-2/project/travle_agent4/data_csv/metadata_preview.csv" 

if os.path.exists(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path)
        
        # '지역' 컬럼 확인
        if '지역' in df.columns:
            unique_regions = sorted(df['지역'].dropna().unique())
            print(f"✅ 발견된 지역 목록 (총 {len(unique_regions)}개):")
            print("-" * 30)
            
            # 너무 많으면 앞부분만 출력
            for region in unique_regions:
                print(f"- {region}")

                
            print("-" * 30)
            print("💡 확인 포인트: '부산광역시 해운대구' 처럼 '시/도 + 구/군'이 모두 포함되어 있나요?")
        else:
            print(f"❌ '{csv_file_path}' 파일에 '지역' 컬럼이 없습니다. 컬럼 목록: {df.columns}")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {csv_file_path}")

✅ 발견된 지역 목록 (총 254개):
------------------------------
- 강원특별자치도 강릉시
- 강원특별자치도 고성군
- 강원특별자치도 동해시
- 강원특별자치도 삼척시
- 강원특별자치도 속초시
- 강원특별자치도 양구군
- 강원특별자치도 양양군
- 강원특별자치도 영월군
- 강원특별자치도 원주시
- 강원특별자치도 인제군
- 강원특별자치도 정선군
- 강원특별자치도 철원군
- 강원특별자치도 춘천시
- 강원특별자치도 태백시
- 강원특별자치도 평창군
- 강원특별자치도 홍천군
- 강원특별자치도 화천군
- 강원특별자치도 횡성군
- 경기도 가평군
- 경기도 고양시
- 경기도 과천시
- 경기도 광명시
- 경기도 광주시
- 경기도 구리시
- 경기도 군포시
- 경기도 김포시
- 경기도 남양주시
- 경기도 동두천시
- 경기도 부천시
- 경기도 성남시
- 경기도 수원시
- 경기도 시흥시
- 경기도 안산시
- 경기도 안성시
- 경기도 안양시
- 경기도 양주시
- 경기도 양평군
- 경기도 여주시
- 경기도 연천군
- 경기도 오산시
- 경기도 용인시
- 경기도 의왕시
- 경기도 의정부시
- 경기도 이천시
- 경기도 파주시
- 경기도 평택시
- 경기도 포천시
- 경기도 하남시
- 경기도 화성시
- 경상남도 거제시
- 경상남도 거창군
- 경상남도 고성군
- 경상남도 김해시
- 경상남도 남해군
- 경상남도 밀양시
- 경상남도 사천시
- 경상남도 산청군
- 경상남도 양산시
- 경상남도 의령군
- 경상남도 진주시
- 경상남도 창녕군
- 경상남도 창원시
- 경상남도 통영시
- 경상남도 하동군
- 경상남도 함안군
- 경상남도 함양군
- 경상남도 합천군
- 경상북도 경산시
- 경상북도 경주시
- 경상북도 고령군
- 경상북도 구미시
- 경상북도 김천시
- 경상북도 문경시
- 경상북도 봉화군
- 경상북도 상주시
- 경상북도 성주군
- 경상북도 안동시
- 경상북도 영덕군
- 경상북도 영양군
- 경상북도 영주시
- 경상북도 영천시
- 경상북도 예천군
- 경상북도 울릉군
- 경상북도 울

In [4]:
import pandas as pd
import os
import pickle
from src.config import review_faiss

# 1. 확인할 CSV 파일명 (사용자님의 실제 파일명으로 꼭 변경!)
CSV_FILE = "/Users/seoungmun/Documents/work/3-2/project/travle_agent4/data_csv/metadata_preview.csv"

# 2. FAISS 메타데이터 파일 경로 (일반적으로 review_faiss 폴더 안에 있음)
META_PKL_PATH = os.path.join(review_faiss, "metadata_list.pkl")  # 혹은 "index.pkl" 등 저장 방식에 따라 다름

print("🔍 데이터 정밀 진단을 시작합니다...\n")

# --- 진단 1: 원본 CSV 파일 확인 ---
print(f"[진단 1] 원본 CSV 파일 확인: {CSV_FILE}")
if os.path.exists(CSV_FILE):
    try:
        df = pd.read_csv(CSV_FILE)
        print(f"✅ 파일 로드 성공! 총 {len(df)}개 행")
        
        # '지역' 컬럼 확인
        if '지역' in df.columns:
            null_count = df['지역'].isnull().sum()
            print(f"✅ '지역' 컬럼 존재함 (결측치: {null_count}개)")
            
            # 부산 지역 샘플 확인
            busan_sample = df[df['지역'].str.contains("부산", na=False)].head(3)
            if not busan_sample.empty:
                print("\n   [부산 데이터 샘플]")
                for _, row in busan_sample.iterrows():
                    print(f"   - {row['지역']} | {row.get('장소', '이름없음')}")
            else:
                print("⚠️ 경고: '부산'이 포함된 데이터가 CSV에 없습니다.")
        else:
            print(f"❌ 치명적 오류: CSV에 '지역' 컬럼이 없습니다! (현재 컬럼: {list(df.columns)})")
            
    except Exception as e:
        print(f"❌ CSV 파일 읽기 오류: {e}")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {CSV_FILE}")


# --- 진단 2: FAISS 메타데이터 (pickle) 확인 ---
# 이전에 지역 검색이 잘 되었다면, 이 파일이 살아있을 수 있습니다.
print(f"\n[진단 2] FAISS 메타데이터(pickle) 확인: {META_PKL_PATH}")

if os.path.exists(META_PKL_PATH):
    try:
        with open(META_PKL_PATH, "rb") as f:
            meta_list = pickle.load(f)
            
        print(f"✅ 메타데이터 파일 로드 성공! 총 {len(meta_list)}개 항목")
        
        # 첫 번째 데이터 까보기
        if len(meta_list) > 0:
            first_item = meta_list[0]
            print("\n   [첫 번째 메타데이터 구조 확인]")
            print(f"   Type: {type(first_item)}")
            print(f"   Content: {first_item}")
            
            # 우리가 필요한 'region' 키가 있는지 확인
            if isinstance(first_item, dict):
                if 'region' in first_item:
                    print(f"   ✅ 'region' 키가 존재합니다. (값 예시: {first_item['region']})")
                elif '지역' in first_item:
                    print(f"   ⚠️ 'region' 대신 '지역' 키가 있습니다. 리트리버 코드를 수정해야 할 수도 있습니다.")
                else:
                    print(f"   ❌ 'region' 관련 키가 없습니다! (키 목록: {list(first_item.keys())})")
            elif hasattr(first_item, 'metadata'): # Document 객체인 경우
                print(f"   ✅ Document 객체입니다. Metadata: {first_item.metadata}")
            else:
                print("   ⚠️ 알 수 없는 데이터 타입입니다.")
                
    except Exception as e:
        print(f"❌ 메타데이터 파일 읽기 오류: {e}")
else:
    print(f"⚠️ 메타데이터 파일을 찾을 수 없습니다.")
    print("   (참고: FAISS DB 폴더 안에 'index.faiss'만 있고 메타데이터 파일이 분리되어 저장되지 않았을 수도 있습니다.)")

print("\n------------------------------")
print("💡 진단 가이드:")
print("1. [진단 1]에서 '지역' 컬럼이 있고 샘플이 잘 나온다면 -> 원본 데이터는 정상입니다.")
print("2. [진단 2]에서 파일이 없거나 'region' 키가 없다면 -> DB에 메타데이터가 누락된 상태입니다.")
print("   이 경우, 아쉽지만 검색 시 필터링을 할 수 없으므로 DB 재구축(rebuild)이 불가피할 수 있습니다.")

🔍 데이터 정밀 진단을 시작합니다...

[진단 1] 원본 CSV 파일 확인: /Users/seoungmun/Documents/work/3-2/project/travle_agent4/data_csv/metadata_preview.csv
✅ 파일 로드 성공! 총 168591개 행
✅ '지역' 컬럼 존재함 (결측치: 0개)

   [부산 데이터 샘플]
   - 부산광역시 중구 | 이름없음
   - 부산광역시 중구 | 이름없음
   - 부산광역시 중구 | 이름없음

[진단 2] FAISS 메타데이터(pickle) 확인: /Users/seoungmun/Documents/work/3-2/project/travle_agent4/review_faiss/metadata_list.pkl
✅ 메타데이터 파일 로드 성공! 총 168591개 항목

   [첫 번째 메타데이터 구조 확인]
   Type: <class 'dict'>
   Content: {'지역': '부산광역시 중구', '카테고리': '맛집', '장소명': '이정태본참치', '별점': 4.4, '상세 주소': '부산 중구 중앙대로81번길 9 1층', '리뷰': '여기는 점심 굶고 저녁에 와야합니다.\n배찢어져요\n스키다시에 회에 정신못차려요', 'text_for_embedding': '지역: 부산광역시 중구 | 장소명: 이정태본참치 | 카테고리: 맛집 | 리뷰: 여기는 점심 굶고 저녁에 와야합니다.\n배찢어져요\n스키다시에 회에 정신못차려요'}
   ⚠️ 'region' 대신 '지역' 키가 있습니다. 리트리버 코드를 수정해야 할 수도 있습니다.

------------------------------
💡 진단 가이드:
1. [진단 1]에서 '지역' 컬럼이 있고 샘플이 잘 나온다면 -> 원본 데이터는 정상입니다.
2. [진단 2]에서 파일이 없거나 'region' 키가 없다면 -> DB에 메타데이터가 누락된 상태입니다.
   이 경우, 아쉽지만 검색 시 필터링을 할 수 없으므로 DB 재구축(rebuild)이 불가피할 수

In [3]:
# [Cell] 수정된 디버깅 코드 (한글 키 지원)
from src.config import load_faiss_index
from src.search import RegionPreFilteringRetriever 

def debug_retriever_v2(target_location, query):
    print(f"\n🧪 [테스트] 목적지: '{target_location}' | 검색어: '{query}'")
    
    db = load_faiss_index()
    
    # 수정된 리트리버 사용
    retriever = RegionPreFilteringRetriever(
        vectorstore=db, 
        k=5, 
        fixed_location=target_location
    )
    
    docs = retriever.invoke(query)
    
    print(f"📄 검색된 문서 수: {len(docs)}개")
    print("-" * 60)
    
    error_count = 0
    for i, doc in enumerate(docs):
        # [핵심 수정] 한글 키 '지역', '장소명'으로 가져오기
        region = doc.metadata.get('지역') or doc.metadata.get('region') or '지역정보 없음'
        place_name = doc.metadata.get('장소명') or doc.metadata.get('place_name') or '이름 없음'
        
        # 검증 로직
        check_keyword = target_location.replace("특별시", "").replace("광역시", "").replace("특별자치시", "").replace("특별자치도", "")
        
        is_valid = check_keyword in region
        status_icon = "✅" if is_valid else "❌ [오류: 타지역]"
        
        if not is_valid: error_count += 1
            
        print(f"{status_icon} {i+1}. [{region}] {place_name}")
        
    print("-" * 60)
    if error_count == 0 and len(docs) > 0:
        print("🎉 테스트 성공: 모든 결과가 정상입니다!")
    elif len(docs) == 0:
        print("⚠️ 검색 결과가 0개입니다. (키워드 매칭 실패 또는 데이터 없음)")
    else:
        print(f"⚠️ 테스트 실패: {error_count}개의 타지역 데이터가 발견되었습니다.")

# --- 실행 ---
debug_retriever_v2("부산광역시", "해운대 오션뷰 카페")
debug_retriever_v2("서울특별시", "강남 맛집")


🧪 [테스트] 목적지: '부산광역시' | 검색어: '해운대 오션뷰 카페'
📄 검색된 문서 수: 5개
------------------------------------------------------------
✅ 1. [부산광역시 서구] 오그남
✅ 2. [부산광역시 서구] 오그남
✅ 3. [부산광역시 해운대구] 스타벅스 하버타운점
✅ 4. [부산광역시 서구] EL16.52
✅ 5. [부산광역시 수영구] 프론트커피 광안리
------------------------------------------------------------
🎉 테스트 성공: 모든 결과가 정상입니다!

🧪 [테스트] 목적지: '서울특별시' | 검색어: '강남 맛집'
📄 검색된 문서 수: 5개
------------------------------------------------------------
✅ 1. [서울특별시 강남구] 능라도 강남점
✅ 2. [서울특별시 강남구] 창고43 강남점
✅ 3. [서울특별시 강남구] 큰물참치 강남점
✅ 4. [서울특별시 강남구] 큰물참치 강남점
✅ 5. [서울특별시 강남구] 농민백암순대 강남직영점
------------------------------------------------------------
🎉 테스트 성공: 모든 결과가 정상입니다!


In [ ]:
import time
import random
import pandas as pd
from tqdm import tqdm

# 프로젝트 모듈 가져오기
from src.config import load_faiss_index
from src.region_cut_fuzz import normalize_region_name
from src.search import RegionPreFilteringRetriever

# DB 로드 (시간이 좀 걸릴 수 있습니다)
print("📂 FAISS 인덱스 로딩 중...")
db = load_faiss_index()
print("✅ DB 로드 완료!")

# 질문 템플릿 (다양한 변주)
TEMPLATES = [
    "{region} 맛집 추천해줘",
    "{region} 가볼 만한 곳 있어?",
    "{region} 예쁜 카페 알려줘",
    "{region} 데이트 코스 추천",
    "{region} 1박 2일 여행지",
    "{region} 아이랑 갈만한 식당",
    "{region} 분위기 좋은 술집",
    "{region} 유명한 빵집",
    "{region} 현지인 맛집 어디야",
    "{region} 주차하기 좋은 카페"
]

def generate_evaluation_dataset(num_samples=1000):
    """
    전국 행정구역과 템플릿을 조합하여 테스트 셋을 생성합니다.
    """
    print(f"⚙️ {num_samples}개의 테스트 쿼리 생성 중...")
    dataset = []
    
    # 1. 테스트할 지역 리스트 준비
    all_targets = []
    for city, districts in REGION_STRUCTURE.items():
        # 광역시/도 단위 (예: 부산광역시)
        all_targets.append((city, city)) 
        # 구/군 단위 (예: 부산광역시 해운대구)
        for dist in districts:
            all_targets.append((city, f"{city} {dist}")) 

    # 2. 랜덤 샘플링
    for _ in range(num_samples):
        # 지역 선택
        target_city, target_detail = random.choice(all_targets)
        
        # 템플릿 선택
        template = random.choice(TEMPLATES)
        
        # 쿼리 생성 (노이즈 추가: 30% 확률로 상위 지역명 생략 -> 기존 모델 헷갈리게 유도)
        # 예: "부산광역시 해운대구" -> "해운대구 맛집"
        if random.random() < 0.3 and target_city != target_detail:
            query_region = target_detail.split()[-1] # "해운대구"
        else:
            query_region = target_detail # "부산광역시 해운대구"
            
        query = template.format(region=query_region)
        
        dataset.append({
            "query": query,
            "target_city": target_city,       # 정답 판별용 (상위)
            "target_detail": target_detail,   # 정답 판별용 (하위)
            "fixed_location": target_city     # 제안 모델 입력용 (세션 정보 가정)
        })
        
    return dataset

# 데이터셋 생성 테스트 (5개만 미리보기)
sample_data = generate_evaluation_dataset(5)
print("\n[생성된 데이터 예시]")
for d in sample_data:
    print(f"- Q: {d['query']} | Target: {d['target_detail']}")

ImportError: cannot import name 'REGION_STRUCTURE' from 'src.region_cut_fuzz' (/Users/seoungmun/Documents/work/3-2/project/travle_agent4/src/region_cut_fuzz.py)

In [14]:
# [Model A] Baseline: 기존 랭체인 기본 검색 (필터링 X)
class BaselineRetriever:
    def __init__(self, vectorstore, k=5):
        # 랭체인의 기본 as_retriever 사용 (단순 유사도 검색)
        self.retriever = vectorstore.as_retriever(search_kwargs={"k": k})
        
    def invoke(self, query):
        return self.retriever.invoke(query)

# [Model B] Proposed: 제안하는 하이브리드 필터링 검색
# (이미 src.search.RegionPreFilteringRetriever 클래스가 정의되어 있으므로 그대로 사용)

In [ ]:
import time
import pandas as pd
from tqdm import tqdm
import sys
import os

# 경로 설정
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

from src.config import load_faiss_index
from src.search import RegionPreFilteringRetriever
from src.region_cut_fuzz import normalize_region_name

# --- 1. 헷갈리는 지역 데이터셋 생성 (집중 공격) ---
def generate_confusing_dataset():
    """
    이름이 같거나 비슷해서 검색기가 헷갈리기 쉬운 지역들만 모아 테스트셋 생성
    """
    # 1. 중구 (전국 6개 광역시)
    jung_gu_list = [
        "서울특별시 중구", "부산광역시 중구", "대구광역시 중구", 
        "인천광역시 중구", "대전광역시 중구", "울산광역시 중구"
    ]
    
    # 2. 강서구 (서울 vs 부산)
    gangseo_gu_list = ["서울특별시 강서구", "부산광역시 강서구"]
    
    # 3. 서구, 동구, 남구, 북구 (광역시에 공통)
    common_gu_list = [
        "부산광역시 서구", "대구광역시 서구", "인천광역시 서구", "광주광역시 서구", "대전광역시 서구",
        "부산광역시 동구", "대구광역시 동구", "인천광역시 동구", "광주광역시 동구", "대전광역시 동구"
    ]
    
    # 4. 고성 (강원 vs 경남)
    goseong_list = ["강원특별자치도 고성군", "경상남도 고성군"]
    
    # 5. 광주 (광주광역시 vs 경기도 광주시)
    gwangju_list = ["광주광역시", "경기도 광주시"]

    # 전체 리스트 합치기
    target_regions = jung_gu_list + gangseo_gu_list + common_gu_list + goseong_list + gwangju_list
    
    dataset = []
    templates = ["{r} 맛집 추천", "{r} 가볼만한 곳", "{r} 유명한 카페", "{r} 여행 코스"]
    
    for full_region_name in target_regions:
        # 쿼리 생성 (예: "부산 중구 맛집", "강원도 고성 여행")
        # 정규화된 이름에서 시/도, 구/군을 분리하여 자연스러운 쿼리 생성
        parts = full_region_name.split()
        if len(parts) >= 2:
            short_name = f"{parts[0][:2]} {parts[1]}" # 예: "부산 중구"
        else:
            short_name = parts[0]

        for temp in templates:
            query = temp.format(r=short_name)
            dataset.append({
                "query": query,          # 예: "부산 중구 맛집 추천"
                "target": full_region_name # 정답: "부산광역시 중구"
            })
            
    return dataset

# --- 2. 평가 루프 (이전과 동일 로직) ---
def run_stress_test():
    print("📂 FAISS 인덱스 로딩 중...")
    try:
        db = load_faiss_index()
    except Exception as e:
        print(f"❌ DB 로드 실패: {e}")
        return

    # 데이터 준비
    test_data = generate_confusing_dataset()
    print(f"⚠️ [Hard Mode] 동음이의 지역 집중 테스트 셋 생성 완료 (총 {len(test_data)}개)")
    print("   (예시: 서울 중구 vs 부산 중구, 강원 고성 vs 경남 고성 등)")

    results = {
        "Baseline": {"correct": 0, "total": 0},
        "Proposed": {"correct": 0, "total": 0}
    }
    
    failed_cases_baseline = [] # 베이스라인이 틀린 경우 수집

    print("🚀 평가 시작...")
    
    for item in tqdm(test_data):
        query = item['query']
        target_region = item['target']
        
        # --- (A) Baseline (단순 검색) ---
        base_retriever = db.as_retriever(search_kwargs={"k": 3})
        try:
            base_docs = base_retriever.invoke(query)
        except:
            base_docs = []
        
        # Baseline 채점
        # 검색 결과 3개 중 하나라도 '타 지역'이 섞이면 실패로 간주 (Strict Mode)
        base_is_correct = True
        retrieved_wrong = []
        
        for doc in base_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            
            # [검증 로직] 정답 지역의 시/도 이름이 포함되어 있는지 확인
            # 예: 정답 '부산광역시 중구' -> 문서에 '부산'이 있어야 함. '서울'이 있으면 오답.
            target_sido = target_region.split()[0][:2] # '부산'
            
            if target_sido not in doc_region:
                base_is_correct = False
                retrieved_wrong.append(doc_region)
        
        if base_is_correct and base_docs:
            results["Baseline"]["correct"] += 1
        else:
            # 실패 사례 저장 (분석용)
            if len(failed_cases_baseline) < 5: # 5개만 샘플링
                failed_cases_baseline.append({
                    "Query": query,
                    "Target": target_region,
                    "Wrong_Result": retrieved_wrong[:1] # 하나만 표시
                })

        results["Baseline"]["total"] += 1

        # --- (B) Proposed (필터링 검색) ---
        # 정답 지역명(target_region)을 기반으로 필터링 수행
        prop_retriever = RegionPreFilteringRetriever(
            vectorstore=db, 
            k=3, 
            fixed_location=target_region # 이미 정규화된 정답셋을 넣음
        )
        try:
            prop_docs = prop_retriever.invoke(query)
        except:
            prop_docs = []

        # Proposed 채점
        prop_is_correct = True
        for doc in prop_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            target_sido = target_region.split()[0][:2]
            
            if target_sido not in doc_region:
                prop_is_correct = False
        
        if prop_is_correct and prop_docs:
            results["Proposed"]["correct"] += 1
        results["Proposed"]["total"] += 1

    # --- 3. 결과 리포트 ---
    print("\n" + "="*60)
    print("📊 [동음이의 지역] 성능 비교 리포트")
    print("="*60)

    final_stats = []
    for model_name, metrics in results.items():
        acc = (metrics['correct'] / metrics['total']) * 100 if metrics['total'] > 0 else 0
        final_stats.append({
            "모델": model_name,
            "정확도 (Accuracy)": f"{acc:.2f}% ({metrics['correct']}/{metrics['total']})"
        })

    df = pd.DataFrame(final_stats)
    print(df)
    
    print("-" * 60)
    print("📉 Baseline(기존) 주요 실패 사례:")
    for case in failed_cases_baseline:
        print(f" - Q: '{case['Query']}' (목표: {case['Target']})")
        print(f"   -> ❌ 오답 검색됨: {case['Wrong_Result']}")
    print("-" * 60)

# 실행
if __name__ == "__main__":
    run_stress_test()

In [18]:
# 평가 실행 (샘플 수 1000개)
run_comparative_evaluation(1000)

⚙️ 1000개의 테스트 쿼리 생성 중...

🚀 [기존 모델 vs 제안 모델] 비교 평가 시작 (총 1000회)


100%|██████████| 1000/1000 [07:57<00:00,  2.10it/s]


📊 평가 결과 리포트


,모델,지역 정확도 (Precision),평균 속도 (Latency)
0,Baseline (기존),84.96%,235.07 ms
1,Proposed (제안),86.27%,241.34 ms


------------------------------------------------------------
💡 분석:
1. Baseline은 전국 데이터 유사도 검색이므로 동명 지역(예: 중구, 고성)에서 오류가 발생합니다.
2. Proposed는 메타데이터 필터링을 통해 해당 지역 내에서만 검색하므로 정확도가 획기적으로 높습니다.


In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sys
import os
import platform

# --- 1. 환경 설정 ---
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# 한글 폰트 설정
if platform.system() == 'Darwin': # Mac
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows
    plt.rc('font', family='Malgun Gothic')
else: # Linux
    plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

try:
    from src.config import load_faiss_index
    from src.search import RegionPreFilteringRetriever
    from src.region_cut_fuzz import normalize_region_name
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")

# --- 2. 헷갈리는 지역 데이터셋 생성 ---
def generate_confusing_dataset():
    # 동음이의 지역 리스트
    target_regions = [
        "서울특별시 중구", "부산광역시 중구", "대구광역시 중구", 
        "인천광역시 중구", "대전광역시 중구", "울산광역시 중구",
        "서울특별시 강서구", "부산광역시 강서구",
        "강원특별자치도 고성군", "경상남도 고성군",
        "광주광역시", "경기도 광주시"
    ]
    
    dataset = []
    templates = ["{r} 맛집 추천", "{r} 가볼만한 곳", "{r} 유명한 카페", "{r} 여행 코스"]
    
    for full_region_name in target_regions:
        parts = full_region_name.split()
        if len(parts) >= 2:
            short_name = f"{parts[0][:2]} {parts[1]}" # 예: "부산 중구"
        else:
            short_name = parts[0]

        for temp in templates:
            dataset.append({
                "query": temp.format(r=short_name), 
                "target": full_region_name
            })
    return dataset

# --- 3. 평가 및 시각화 실행 ---
def run_stress_test_and_visualize():
    print("📂 FAISS 인덱스 로딩 중...")
    try:
        db = load_faiss_index()
    except Exception as e:
        print(f"❌ DB 로드 실패: {e}")
        return

    test_data = generate_confusing_dataset()
    print(f"⚠️ [Hard Mode] 동음이의 지역 집중 테스트 시작 (총 {len(test_data)}회)")
    print("   (Target: 서울 중구 vs 부산 중구, 강원 고성 vs 경남 고성 등)")

    results = {
        "Baseline": {"correct": 0, "total": 0, "time": []},
        "Proposed": {"correct": 0, "total": 0, "time": []}
    }
    
    for item in tqdm(test_data):
        query = item['query']
        target_region = item['target']
        
        # --- (A) Baseline ---
        start_time = time.time()
        base_retriever = db.as_retriever(search_kwargs={"k": 3})
        try: base_docs = base_retriever.invoke(query)
        except: base_docs = []
        end_time = time.time()
        
        base_is_correct = True
        for doc in base_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            target_sido = target_region.split()[0][:2] # '부산'
            if target_sido not in doc_region:
                base_is_correct = False
        
        if base_is_correct and base_docs:
            results["Baseline"]["correct"] += 1
        results["Baseline"]["total"] += 1
        results["Baseline"]["time"].append(end_time - start_time)

        # --- (B) Proposed ---
        start_time = time.time()
        # [핵심] 정답 지역을 미리 알고 필터링한다고 가정 (의도 파악 성공 시)
        prop_retriever = RegionPreFilteringRetriever(vectorstore=db, k=3, fixed_location=target_region)
        try: prop_docs = prop_retriever.invoke(query)
        except: prop_docs = []
        end_time = time.time()

        prop_is_correct = True
        for doc in prop_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            target_sido = target_region.split()[0][:2]
            if target_sido not in doc_region:
                prop_is_correct = False
        
        if prop_is_correct and prop_docs:
            results["Proposed"]["correct"] += 1
        results["Proposed"]["total"] += 1
        results["Proposed"]["time"].append(end_time - start_time)

    # --- 4. 결과 집계 ---
    final_stats = []
    for model_name, metrics in results.items():
        precision = (metrics['correct'] / metrics['total']) * 100 if metrics['total'] > 0 else 0
        avg_latency = (sum(metrics['time']) / len(metrics['time'])) * 1000
        
        # [핵심] 시각화 코드와 키 이름을 통일했습니다.
        final_stats.append({
            "모델": model_name,
            "지역 정확도 (Precision)": precision, 
            "평균 속도 (Latency)": avg_latency
        })

    df_result = pd.DataFrame(final_stats)
    
    # 텍스트 출력용 포맷팅
    df_display = df_result.copy()
    df_display['지역 정확도 (Precision)'] = df_display['지역 정확도 (Precision)'].apply(lambda x: f"{x:.2f}%")
    df_display['평균 속도 (Latency)'] = df_display['평균 속도 (Latency)'].apply(lambda x: f"{x:.2f} ms")
    
    print("\n📊 [동음이의 지역] 평가 결과 리포트")
    print(df_display)

    # --- 5. 시각화 ---
    visualize_stress_results(df_result)

def visualize_stress_results(df_plot):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    sns.set_style("whitegrid")

    # 정확도 그래프
    sns.barplot(x='모델', y='지역 정확도 (Precision)', data=df_plot, ax=axes[0], palette='viridis')
    axes[0].set_title('동음이의 지역 정확도 비교', fontsize=15, fontweight='bold')
    axes[0].set_ylabel('정확도 (%)', fontsize=12)
    axes[0].set_ylim(0, 110)
    for p in axes[0].patches:
        axes[0].annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', fontsize=12, fontweight='bold')

    # 속도 그래프
    sns.barplot(x='모델', y='평균 속도 (Latency)', data=df_plot, ax=axes[1], palette='rocket')
    axes[1].set_title('평균 검색 속도 (Latency)', fontsize=15, fontweight='bold')
    axes[1].set_ylabel('평균 소요 시간 (ms)', fontsize=12)
    for p in axes[1].patches:
        axes[1].annotate(f'{p.get_height():.2f} ms', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()
    
    # CSV 저장
    df_plot.to_csv("stress_test_result.csv", index=False, encoding='utf-8-sig')
    print("💾 결과가 'stress_test_result.csv'로 저장되었습니다.")

# 실행
if __name__ == "__main__":
    run_stress_test_and_visualize()

📂 FAISS 인덱스 로딩 중...
⚠️ [Hard Mode] 동음이의 지역 집중 테스트 시작 (총 48회)
   (Target: 서울 중구 vs 부산 중구, 강원 고성 vs 경남 고성 등)


100%|██████████| 48/48 [00:19<00:00,  2.47it/s]


📊 [동음이의 지역] 평가 결과 리포트
         모델 지역 정확도 (Precision) 평균 속도 (Latency)
0  Baseline             77.08%       207.83 ms
1  Proposed             93.75%       196.83 ms
💾 결과가 'stress_test_result.csv'로 저장되었습니다.



/var/folders/_f/zr1lq38j0sb4t8xvj43g2khm0000gn/T/ipykernel_81611/1025229079.py:152: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='모델', y='지역 정확도 (Precision)', data=df_plot, ax=axes[0], palette='viridis')
/var/folders/_f/zr1lq38j0sb4t8xvj43g2khm0000gn/T/ipykernel_81611/1025229079.py:160: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='모델', y='평균 속도 (Latency)', data=df_plot, ax=axes[1], palette='rocket')
/var/folders/_f/zr1lq38j0sb4t8xvj43g2khm0000gn/T/ipykernel_81611/1025229079.py:167: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [42]:
import os
import sys
from dotenv import load_dotenv

# 환경 변수 로드 (.env 파일이 루트 폴더에 있다고 가정)
load_dotenv()

# 프로젝트 루트 경로 추가 (src/ 모듈 임포트용)
# 만약 노트북이 src 폴더 밖에 있다면 이 코드가 필요합니다.
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

# 함수 및 모듈 임포트
try:
    from src.tools import search_attractions_and_reviews, get_weather_forecast
    # normalize_region_name은 search_attractions_and_reviews 내부에서 사용됨
    print("✅ 필수 함수 임포트 성공!")
except ImportError as e:
    print(f"❌ 임포트 오류: {e}")
    print("   (src/tools.py 파일이 프로젝트 경로에 저장되었는지 확인해주세요.)")

# RAG 결과가 너무 길어 콘솔 출력을 정리하기 위한 헬퍼 함수
def print_search_result(result):
    if result.startswith("오류"):
        print(f"🚨 오류 발생: {result}")
    else:
        # LLM에게 전달할 최종 프롬프트 (요약된 결과)를 확인합니다.
        print("\n--- LLM 처리 최종 결과 (카테고리/설명 포함) ---")
        print(result)

✅ 필수 함수 임포트 성공!


In [ ]:
import os
import sys
from typing import Dict, Any
from langchain_core.messages import HumanMessage

# 1. 경로 설정 (필수)
# 노트북이 src 폴더 밖에 있다면 이 코드가 src 모듈을 찾게 해줍니다.
# (이전 셀에서 실행했다면 생략 가능)
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. Graph Flow 임포트
try:
    from src.graph_flow import build_graph, AgentState
    print("✅ Graph Flow 임포트 성공!")
except ImportError as e:
    print(f"❌ Graph Flow 임포트 오류: {e}")
    print("   (src/graph_flow.py 파일을 확인해주세요.)")
    sys.exit()

# 3. 그래프 인스턴스 생성
app = build_graph()

# 4. 초기 상태 설정 (Streamlit 세션 상태 시뮬레이션)
# '여행 정보 입력' 페이지에서 사용자가 입력하고 저장했다고 가정합니다.

# ⚠️ 여기에 원하는 테스트 값을 입력하세요!
TEST_DESTINATION = "부산"
TEST_DATES = "2025년 12월 5일 ~ 2025년 12월 7일" # 3일치 테스트
TEST_PREFERENCE = "오션뷰 카페와 해산물 맛집 위주"
TEST_TOTAL_DAYS = 3

# Graph Flow의 초기 입력 상태
initial_state_input: AgentState = {
    "messages": [HumanMessage(content="안녕하세요! 입력한 정보로 1일차 계획을 시작해주세요.")],
    "current_weather": "", # 날씨를 비워둬서 WeatherAgent가 호출되도록 유도
    "itinerary": [],
    "destination": TEST_DESTINATION,
    "dates": TEST_DATES,
    "preference": TEST_PREFERENCE,
    "total_days": TEST_TOTAL_DAYS,
    "activity_level": 3,
    "current_planning_day": 1,
    "show_pdf_button": False
}

# 5. 그래프 실행 (WeatherAgent 호출까지 진행)
print("\n=======================================================")
print(f"🚀 Graph Flow 실행 시작 (목적지: {TEST_DESTINATION}, 기간: {TEST_DATES})")
print("=======================================================")

# Graph Flow는 Entry Point에서 WeatherAgent를 선택할 것입니다.
final_state = app.invoke(initial_state_input)

# 6. 디버깅 로그 확인
print("\n[디버깅 로그 검증 포인트]")
print("1. WeatherAgent 호출 시:")
print("   - 'DEBUG_W_1': Agent 전달 목적지가 '부산'으로 정확히 찍혔는가?")
print("   - 'DEBUG_W_2': Agent 전달 기간이 '2025년 12월 5일 ~ 2025년 12월 7일'로 찍혔는가?")
print("   - 'DEBUG_W_3': OWM 데이터에 3일 이상의 날짜 정보가 포함되어 있는가?")
print("2. Graph Flow의 최종 상태:")
print(f"   - 최종 Weather 상태: {final_state.get('current_weather', '업데이트 실패')[:50]}...")

✅ Graph Flow 임포트 성공!

🚀 Graph Flow 실행 시작 (목적지: 부산, 기간: 2025년 12월 5일 ~ 2025년 12월 7일)
--- (Supervisor) 다음 작업 결정 ---
--- (Supervisor) 다음 작업 결정 ---
Supervisor -> SupervisorAgent (ToolMessage 결과 처리 요청)

[디버깅 로그 검증 포인트]
1. WeatherAgent 호출 시:
   - 'DEBUG_W_1': Agent 전달 목적지가 '부산'으로 정확히 찍혔는가?
   - 'DEBUG_W_2': Agent 전달 기간이 '2025년 12월 5일 ~ 2025년 12월 7일'로 찍혔는가?
   - 'DEBUG_W_3': OWM 데이터에 3일 이상의 날짜 정보가 포함되어 있는가?
2. Graph Flow의 최종 상태:
   - 최종 Weather 상태: [부산] (2025년 12월 5일 ~ 2025년 12월 7일) 날씨 정보 (기능 점검 중)...


In [28]:
import os
import sys
from typing import Dict, Any
from langchain_core.messages import HumanMessage

# 경로 설정 (이전과 동일)
# ... (경로 설정 코드는 생략하지만, 이전 셀에서 이미 실행되었다고 가정)

try:
    from src.graph_flow import build_graph, AgentState
    print("✅ Graph Flow 임포트 성공!")
except ImportError as e:
    print(f"❌ Graph Flow 임포트 오류: {e}")
    sys.exit()

# 그래프 인스턴스 생성
app = build_graph()

# 1. 가짜 일정 데이터 (시간 정보가 없는 원본 데이터 시뮬레이션)
TEST_ITINERARY = [
    {'day': 1, 'type': '식당', 'name': '해운대 소문난 암소갈비', 'description': '부산 3대 갈비집으로 유명합니다.'},
    {'day': 1, 'type': '관광지', 'name': '해운대 해변', 'description': '해수욕을 즐기기 좋습니다.'},
    {'day': 1, 'type': '카페', 'name': '웨이브온 카페', 'description': '오션뷰가 끝내주는 곳입니다.'},
]

# 2. Graph Flow의 초기 입력 상태
initial_state_input: AgentState = {
    "messages": [HumanMessage(content="경로 최적화 해줘")], # 👈 [핵심] 최적화 요청
    "current_weather": "맑음",
    "itinerary": TEST_ITINERARY, # 👈 가짜 일정 주입
    "destination": "부산",
    "dates": "2025년 12월 5일",
    "preference": "해변 위주",
    "total_days": 1,
    "activity_level": 3,
    "current_planning_day": 1,
    "show_pdf_button": False
}

print("\n=======================================================")
print(f"🚀 Graph Flow 실행 시작: '경로 최적화' 요청 테스트")
print("=======================================================")

# Graph Flow 실행
final_state = app.invoke(initial_state_input)

# 3. 디버깅 로그 확인
print("\n[디버깅 로그 검증 포인트]")
print("1. '--- (Supervisor) 다음 작업 결정 ---' 로그에서 'plan_itinerary_timeline' 도구가 호출되었는가?")
print("2. 'DEBUG TIME PLANNER' 로그에 LLM이 반환한 'estimated_start_time'이 포함된 JSON이 있는가?")
print("3. 최종 상태의 itinerary에 'estimated_start_time' 키가 추가되었는가? (Task 4 성공 여부)")

# 최종 상태에서 시간 정보 포함 여부 확인
final_itinerary = final_state.get('itinerary', [])
has_time_data = any('estimated_start_time' in item for item in final_itinerary)

print(f"\n[최종 상태 요약]")
print(f"Time Data 포함 여부: {'✅ 성공 (시간 계획 완료)' if has_time_data else '❌ 실패 (시간 계획 누락)'}")
print(f"최종 일정 항목 수: {len(final_itinerary)}")

✅ Graph Flow 임포트 성공!

🚀 Graph Flow 실행 시작: '경로 최적화' 요청 테스트
--- (Supervisor) 다음 작업 결정 ---
Router -> call_tools (도구: ['plan_itinerary_timeline'])

--- [DEBUG TIME PLANNER] 시간 계획 시작 ---
--- (Supervisor) 다음 작업 결정 ---
Supervisor -> SupervisorAgent (ToolMessage 결과 처리 요청)
Router -> END

[디버깅 로그 검증 포인트]
1. '--- (Supervisor) 다음 작업 결정 ---' 로그에서 'plan_itinerary_timeline' 도구가 호출되었는가?
2. 'DEBUG TIME PLANNER' 로그에 LLM이 반환한 'estimated_start_time'이 포함된 JSON이 있는가?
3. 최종 상태의 itinerary에 'estimated_start_time' 키가 추가되었는가? (Task 4 성공 여부)

[최종 상태 요약]
Time Data 포함 여부: ❌ 실패 (시간 계획 누락)
최종 일정 항목 수: 3


In [29]:
import unittest
from unittest.mock import patch, MagicMock
import datetime
import os
import sys

# -------------------------------------------------------------------------
# [설정] 프로젝트 루트 경로 추가
# -------------------------------------------------------------------------
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

try:
    from src.tools import get_weather_forecast, optimize_and_get_routes
except ImportError:
    print("⚠️ 'src' 모듈을 찾을 수 없습니다. 현재 작업 경로(os.getcwd())가 프로젝트 루트인지 확인해주세요.")
    print(f"현재 경로: {os.getcwd()}")


# -------------------------------------------------------------------------
# [테스트 클래스]
# -------------------------------------------------------------------------
class TestNewTools(unittest.TestCase):

    @patch('src.tools.requests.get')
    def test_weather_forecast_api_limit(self, mock_get):
        """
        [테스트 1] 날씨 API 기간 초과 시 '한계 보고' 메시지 출력 테스트
        """
        print("\n🧪 [Test 1] 날씨 API 기간 한계 로직 테스트 중...")
        
        future_date = datetime.date.today() + datetime.timedelta(days=100)
        dates_str = f"{future_date.year}년 {future_date.month}월 {future_date.day}일부터 3일간"
        
        with patch.dict(os.environ, {"OWM_API_KEY": "TEST_KEY"}):
            result = get_weather_forecast.invoke({"destination": "서울", "dates": dates_str})
        
        print(f"   👉 결과 요약: {result[:60]}...")
        self.assertIn("[날씨 API 한계 보고]", result)
        self.assertIn("최대 5일 후까지만", result)
        print("✅ [Pass] 날짜 범위 초과 시 안내 메시지 정상 출력 확인!")

    @patch('src.tools.GMAPS_CLIENT')
    def test_optimize_and_get_routes_detail(self, mock_gmaps):
        """
        [테스트 2] 경로 최적화 결과에 '상세 경로(버스/지하철)' 정보 포함 여부 테스트
        """
        print("\n🧪 [Test 2] 상세 경로 최적화 및 정보 포함 여부 테스트 중...")
        
        # 1. Mock 데이터 설정 (Distance Matrix)
        mock_matrix_result = {
            'rows': [
                {'elements': [{'status': 'OK', 'duration': {'value': 0}}, {'status': 'OK', 'duration': {'value': 600}}]}, 
                {'elements': [{'status': 'OK', 'duration': {'value': 600}}, {'status': 'OK', 'duration': {'value': 0}}]} 
            ]
        }
        mock_gmaps.distance_matrix.return_value = mock_matrix_result
        
        # 2. Mock 데이터 설정 (Directions: 상세 이동 경로)
        # ⚠️ [수정됨] vehicle 정보를 line 내부로 이동시켰습니다!
        mock_directions_result = [{
            'legs': [{
                'duration': {'text': '15분'},
                'distance': {'text': '3km'},
                'steps': [
                    {
                        'travel_mode': 'TRANSIT',
                        'transit_details': {
                            'line': {
                                'short_name': '143',
                                # vehicle은 line 안에 있어야 코드가 정상 작동합니다.
                                'vehicle': {'type': 'BUS'} 
                            }
                        }
                    }
                ]
            }]
        }]
        mock_gmaps.directions.return_value = mock_directions_result
        
        # 3. 함수 실행
        places = ["장소A", "장소B"]
        result = optimize_and_get_routes.invoke({"places": places})
        
        # 4. 검증
        print(f"   👉 결과 요약 (일부분):\n{result[:150]}...")
        
        self.assertIn("최적 순서", result)
        self.assertIn("상세 이동 경로", result)
        self.assertIn("[BUS] 143", result)
        print("✅ [Pass] 최적화 결과에 상세 대중교통 정보 포함 확인!")

# -------------------------------------------------------------------------
# [실행]
# -------------------------------------------------------------------------
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..


🧪 [Test 2] 상세 경로 최적화 및 정보 포함 여부 테스트 중...

--- [DEBUG] optimize_and_get_routes (Task 3: 상세 경로 최적화) 호출됨 ---
DEBUG: Input places: ['장소A', '장소B']
DEBUG: Distance Matrix API 호출 시도...
DEBUG: Distance Matrix API 호출 성공.
DEBUG: 최적화된 순서: ['장소A', '장소B']
DEBUG: optimize_and_get_routes (Task 3) 상세 경로 포함 완료.
   👉 결과 요약 (일부분):
--- 🗺️ 최적 경로 제안 (총 2곳) ---
**[최적 순서]**: 장소A → 장소B
**[전체 이동 시간 합]**: 약 10 분 (장소 간 이동 시간의 합)

### 🚌 상세 이동 경로
**장소A** ➡️ **장소B**
   - 예상 소요 시간: 15분 (3km)...
✅ [Pass] 최적화 결과에 상세 대중교통 정보 포함 확인!

🧪 [Test 1] 날씨 API 기간 한계 로직 테스트 중...

--- [DEBUG WEATHER] 날씨 검색 시작 ---
DEBUG_W_1: Agent 전달 목적지: 서울
DEBUG_W_2: Agent 전달 기간: 2026년 3월 13일부터 3일간
   👉 결과 요약: 
[날씨 API 한계 보고]
요청 기간: 2026년 3월 13일부터 3일간 (시작일: 2026-03-13)
...
✅ [Pass] 날짜 범위 초과 시 안내 메시지 정상 출력 확인!



----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


In [47]:
import pandas as pd
import sys
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# [설정] 프로젝트 루트 경로 추가
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

try:
    from src.config import review_faiss # DB 경로
    # 방금 수정한 updater에서 함수들을 가져옵니다.
    from src.rag_updater import create_documents_from_df, find_detailed_address
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")
    print("src/rag_updater.py 파일이 최신 버전으로 저장되었는지 확인해주세요!")

# --- 1. 가짜 리뷰 데이터 생성 (사용자 입력 시뮬레이션) ---
# 사용자는 '상세 주소'를 입력하지 않았습니다.
mock_data = {
    "지역": ["부산 해운대구"], 
    "장소명": ["한우리가"], 
    "카테고리": ["식당"],
    "리뷰": ["밀면이 정말 시원하고 만두가 맛있어요. 부산역 바로 앞이라 가기 편해요."],
    "평점": [5]
}
df_mock = pd.DataFrame(mock_data)

print("📝 [Step 1] 사용자 입력 데이터 (상세 주소 없음):")
display(df_mock) # 또는 print(df_mock)

# --- 2. 실제 FAISS DB 로드 (참조용) ---
print("\n🔄 [Step 2] 기존 지식 베이스(FAISS) 로드 중...")
embeddings = HuggingFaceEmbeddings(
    model_name="upskyy/bge-m3-korean",
    model_kwargs={"device": "cpu"}
)
try:
    db = FAISS.load_local(review_faiss, embeddings, allow_dangerous_deserialization=True)
    print("✅ DB 로드 성공!")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    # DB가 없으면 테스트 불가하므로 중단
    raise e

# --- 3. 변환 로직 실행 (핵심 검증) ---
print("\n⚙️ [Step 3] 문서 변환 및 주소 자동 보정 실행...")

# create_documents_from_df 함수가 내부적으로 find_detailed_address를 호출합니다.
docs = create_documents_from_df(df_mock, db)

# --- 4. 결과 확인 ---
if docs:
    doc = docs[0]
    print("\n🎉 [Step 4] 변환 결과 확인")
    print("=" * 60)
    
    # 1. 상세 주소가 채워졌는지 확인
    generated_address = doc.metadata.get("상세 주소")
    print(f"📍 [검증 1] 상세 주소 자동 채움 여부:")
    if generated_address:
        print(f"   ✅ 성공! 찾아낸 주소: '{generated_address}'")
    else:
        print(f"   ⚠️ 실패/미발견. (DB에 '한우리가' 데이터가 없거나 매칭 실패)")

    # 2. 텍스트 포맷이 맞는지 확인
    print(f"\n📄 [검증 2] 임베딩용 텍스트 포맷 (page_content):")
    print(f"   {doc.page_content[:100]}...") # 앞부분만 출력
    
    # 포맷 검사
    expected_format = "지역: 부산 해운대구 | 장소명: 한우리가 |"
    if expected_format in doc.page_content:
        print(f"   ✅ 성공! 기존 DB 포맷과 일치합니다.")
    else:
        print(f"   ❌ 실패. 포맷이 다릅니다.")
        
    print("=" * 60)
    print("💡 이 'doc' 객체가 그대로 벡터 DB에 add_documents() 됩니다.")
else:
    print("❌ 문서 생성 실패.")

📝 [Step 1] 사용자 입력 데이터 (상세 주소 없음):


,지역,장소명,카테고리,리뷰,평점
0,부산 해운대구,한우리가,식당,밀면이 정말 시원하고 만두가 맛있어요. 부산역 바로 앞이라 가기 편해요.,5



🔄 [Step 2] 기존 지식 베이스(FAISS) 로드 중...
✅ DB 로드 성공!

⚙️ [Step 3] 문서 변환 및 주소 자동 보정 실행...

🎉 [Step 4] 변환 결과 확인
📍 [검증 1] 상세 주소 자동 채움 여부:
   ⚠️ 실패/미발견. (DB에 '한우리가' 데이터가 없거나 매칭 실패)

📄 [검증 2] 임베딩용 텍스트 포맷 (page_content):
   지역: 부산 해운대구 | 장소명: 한우리가 | 카테고리: 식당 | 리뷰: 밀면이 정말 시원하고 만두가 맛있어요 부산역 바로 앞이라 가기 편해요...
   ✅ 성공! 기존 DB 포맷과 일치합니다.
💡 이 'doc' 객체가 그대로 벡터 DB에 add_documents() 됩니다.


In [50]:
import pandas as pd
import sys
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# [설정] 프로젝트 루트 경로 추가
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

try:
    from src.config import review_faiss # DB 경로
except ImportError as e:
    print(f"❌ 설정 파일 로드 실패: {e}")

# --- 1. FAISS DB 로드 ---
print("🔄 기존 지식 베이스(FAISS) 로드 중...")
embeddings = HuggingFaceEmbeddings(
    model_name="upskyy/bge-m3-korean",
    model_kwargs={"device": "cpu"}
)

try:
    db = FAISS.load_local(review_faiss, embeddings, allow_dangerous_deserialization=True)
    print(f"✅ DB 로드 성공! (총 문서 수: {db.index.ntotal}개)")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    sys.exit()

# --- 2. 검색 및 매칭 테스트 함수 ---
def test_search_and_match(place_name, region):
    print(f"\n🕵️‍♂️ [검색 테스트] 목표: {place_name} ({region})")
    print("-" * 60)
    
    # 2-1. 유사도 검색 수행
    # k를 넉넉하게(10개) 잡아서 비슷한 건 다 가져와 봅니다.
    results = db.similarity_search_with_score(place_name, k=10)
    
    if not results:
        print("   ❌ 검색 결과 없음 (유사한 장소명조차 DB에 없습니다.)")
        return

    found_exact_match = False
    
    print(f"   🔎 유사도 상위 {len(results)}개 결과 분석:")
    for i, (doc, score) in enumerate(results):
        meta = doc.metadata
        db_name = str(meta.get("장소명", "Unknown"))
        db_region = str(meta.get("지역", "Unknown"))
        db_address = str(meta.get("상세 주소", "없음"))
        
        # 공백 제거 후 비교 (매칭 로직 시뮬레이션)
        norm_input_name = place_name.replace(" ", "")
        norm_db_name = db_name.replace(" ", "")
        norm_input_region = region.replace(" ", "")
        norm_db_region = db_region.replace(" ", "")
        
        # 매칭 여부 판단
        name_match = (norm_input_name in norm_db_name) or (norm_db_name in norm_input_name)
        region_match = (norm_input_region in norm_db_region) or (norm_db_region in norm_input_region)
        
        match_status = "❌ 불일치"
        if name_match:
            if region_match:
                match_status = "✅ 일치 (정답 후보)"
                found_exact_match = True
            else:
                match_status = "⚠️ 이름만 같음 (지역 다름)"
        
        print(f"   [{i+1}] {db_name} | {db_region}")
        print(f"       - 상세주소: {db_address}")
        print(f"       - 유사도 거리: {score:.4f} (낮을수록 좋음)")
        print(f"       - 판정: {match_status}")
        
    print("-" * 60)
    if found_exact_match:
        print("🎉 결론: DB에 데이터가 있고, 로직상 찾아낼 수 있습니다.")
    else:
        print("💧 결론: 검색은 되지만, '이름'과 '지역'이 모두 맞는 데이터가 없습니다.")
        print("   -> 상세 주소를 못 채우는 것이 정상입니다. (사용자가 직접 입력해야 함)")

# --- 3. 테스트 실행 ---

# Case A: 아까 실패했던 케이스 (한우리가 / 부산 해운대구)
# -> DB에 없을 가능성이 큼
test_search_and_match("양산왕돼지국밥", "부산광역시 해운대구")

# Case B: DB에 확실히 있을 법한 유명 맛집 (초량밀면 / 부산 동구)
# -> 로직 검증용 (이게 성공하면 로직은 문제 없는 것)
test_search_and_match("초량밀면", "부산 동구")

🔄 기존 지식 베이스(FAISS) 로드 중...
✅ DB 로드 성공! (총 문서 수: 168591개)

🕵️‍♂️ [검색 테스트] 목표: 양산왕돼지국밥 (부산광역시 해운대구)
------------------------------------------------------------
   🔎 유사도 상위 10개 결과 분석:
   [1] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 521.5074 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [2] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 534.6143 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [3] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 546.5441 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [4] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 558.8123 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [5] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 559.4098 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [6] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 570.8265 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [7] 남도돼지국밥 | 울산광역시 울주군
       - 상세주소: 울산 울주군 범서읍 장검길 9-9 1층
       - 유사도 거리: 

In [1]:
# graph_debug_notebook.ipynb (Jupyter Notebook 셀 1)

import sys
import os

# 프로젝트 루트 경로 설정 (이 노트북 파일이 프로젝트 루트에 있다고 가정)
# 만약 노트북이 src/ 폴더 외부에 있다면, sys.path에 src 폴더를 추가합니다.
if 'src' not in os.listdir('.') and 'src' not in sys.path:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

from langchain_core.messages import HumanMessage, AIMessage
from src.graph_flow import build_graph, AgentState # build_graph 함수 임포트
from datetime import datetime

# Streamlit 의존성 우회 (config.py 및 1_trip_planner.py에서 사용됨)
# Jupyter 환경에서는 st.cache_resource를 사용할 수 없으므로,
# config.py가 FAISS를 로드할 때 오류가 나지 않도록 임시로 함수를 정의합니다.
# 실제 Streamlit 앱에서는 이 코드가 필요 없습니다.
class MockStCache:
    def clear(self):
        pass
    def __call__(self, func):
        return func

try:
    import streamlit as st
except ImportError:
    # Streamlit이 설치되지 않았거나 Jupyter 환경일 경우 Mock 객체 사용
    class MockStreamlit:
        cache_resource = MockStCache()
        def __init__(self):
            # st.error, st.toast 등이 오류를 일으키지 않도록 모킹
            self.error = lambda x: print(f"[MOCK ST.ERROR]: {x}")
            self.toast = lambda x, icon=None: print(f"[MOCK ST.TOAST]: {x}")
            self.stop = lambda: print("[MOCK ST.STOP] Execution Halted.")
            
    st = MockStreamlit()
    sys.modules['streamlit'] = st
    # 1_trip_planner.py에 있는 st.session_state 초기화를 위해 임시 상태 객체를 만듭니다.
    class MockSessionState(dict):
        def get(self, key, default=None):
            return self.get(key, default)
    st.session_state = MockSessionState()

# LangGraph 앱 로드
app = build_graph()
print("✅ LangGraph 애플리케이션 로드 및 컴파일 완료.")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ LangGraph 애플리케이션 로드 및 컴파일 완료.


In [9]:
!uv pip install grandalf


Using Python 3.10.0 environment at: /opt/homebrew/Caskroom/miniforge/base/envs/env10
Audited 1 package in 7ms


In [2]:
# Jupyter Notebook 셀 2
print("\n## 🔍 그래프 상태 (AgentState) 분석")
for key, type_hint in AgentState.__annotations__.items():
    print(f"* **{key}**: {type_hint}")

print("\n## 🧭 그래프 노드 및 엣지 분석")
display(app.get_graph().print_ascii())


## 🔍 그래프 상태 (AgentState) 분석
* **messages**: typing.Annotated[list, <function _add_messages_wrapper.<locals>._add_messages at 0x10d880280>]
* **current_weather**: <class 'str'>
* **itinerary**: typing.List[typing.Dict]
* **destination**: <class 'str'>
* **dates**: <class 'str'>
* **preference**: <class 'str'>
* **total_days**: <class 'int'>
* **activity_level**: <class 'int'>
* **current_planning_day**: <class 'int'>
* **show_pdf_button**: <class 'bool'>
* **next_node**: typing.Literal['InfoCollectorAgent', 'WeatherAgent', 'AttractionAgent', 'RestaurantAgent', 'DayTransitionAgent', 'ConfirmationAgent', 'PDFCreationAgent', 'end_node']

## 🧭 그래프 노드 및 엣지 분석


ValueError: no intersection found (point inside ?!). view: <langchain_core.runnables.graph_ascii.VertexViewer object at 0x115188340> topt: (92.5, 13.5)

In [2]:
# [Cell 1] 라이브러리 및 클라이언트 설정
import googlemaps
import os
from dotenv import load_dotenv

# .env 파일 로드 (없으면 직접 키 입력)
load_dotenv()
api_key = os.getenv("GMAPS_API_KEY") 
# api_key = "여기에_직접_API_키_입력" 

if not api_key:
    print("❌ API 키가 없습니다. 키를 설정해주세요.")
else:
    gmaps = googlemaps.Client(key=api_key)
    print("✅ Google Maps Client 연결 성공!")

✅ Google Maps Client 연결 성공!


In [3]:
# [Cell 2] 디버깅용 함수 정의
def inspect_reverse_geocoding(lat, lng):
    """
    주어진 좌표에 대해 Google Maps API가 반환하는 모든 주소 정보를 뜯어봅니다.
    """
    print(f"\n🔎 [상세 분석] 좌표: {lat}, {lng}")
    
    try:
        # 1. API 호출
        results = gmaps.reverse_geocode((lat, lng), language='ko')
        
        if not results:
            print("❌ 결과 없음 (Empty Result)")
            return

        print(f"✅ 총 {len(results)}개의 주소 결과가 반환되었습니다.\n")

        # 2. 모든 결과(Results) 순회 (보통 0번이 가장 정확하지만, 1번 등에 원본 데이터가 있을 수 있음)
        for idx, result in enumerate(results):
            print(f"--- [Result #{idx}] ---")
            print(f"formatted_address: {result.get('formatted_address')}")
            print(f"place_id: {result.get('place_id')}")
            print(f"types: {result.get('types')}")
            
            print("   📂 Address Components (상세 요소):")
            for comp in result.get('address_components', []):
                name = comp['long_name']
                types = comp['types']
                
                # 시각적으로 알아보기 쉽게 출력
                match_status = ""
                if 'administrative_area_level_1' in types:
                    match_status = "👈 [광역/도] 후보"
                elif 'sublocality_level_1' in types:
                    match_status = "👈 [기초/구] 후보 (Type A)"
                elif 'locality' in types:
                    match_status = "👈 [기초/시] 후보 (Type B)"
                
                print(f"      - [{name}] : {types} {match_status}")
            print("\n")
            
    except Exception as e:
        print(f"❌ 에러 발생: {e}")

def get_coords_and_inspect(place_name):
    """지명으로 좌표를 따고, 바로 상세 분석으로 넘기는 헬퍼"""
    print(f"📍 '{place_name}'의 좌표를 찾습니다...")
    geocode_result = gmaps.geocode(place_name, language='ko')
    
    if geocode_result:
        loc = geocode_result[0]['geometry']['location']
        lat, lng = loc['lat'], loc['lng']
        print(f"   -> 좌표 획득: {lat}, {lng}")
        inspect_reverse_geocoding(lat, lng)
    else:
        print("❌ 좌표 변환 실패")

In [7]:
# [Cell 3] 테스트 실행
print("부산")
# 1. 문제가 되었던 '부산' (광역만 나오고 구가 안 나오는 케이스)
get_coords_and_inspect("부산")
print("\n광안리")
# 2. 정상 작동이 예상되는 '광안리'
get_coords_and_inspect("광안리")
print("\n해운대")
# 3. 다른 테스트 케이스
get_coords_and_inspect("해운대")
print("\n부산역")
get_coords_and_inspect("부산역")

부산
📍 '부산'의 좌표를 찾습니다...
   -> 좌표 획득: 35.1731121, 129.0714122

🔎 [상세 분석] 좌표: 35.1731121, 129.0714122
✅ 총 9개의 주소 결과가 반환되었습니다.

--- [Result #0] ---
formatted_address: 대한민국 부산광역시 양정역
place_id: ChIJEcux_KnsaDUREAy9LFQRUpM
types: ['establishment', 'point_of_interest', 'transit_station']
   📂 Address Components (상세 요소):
      - [양정역] : ['establishment', 'point_of_interest', 'transit_station'] 
      - [부산광역시] : ['administrative_area_level_1', 'political'] 👈 [광역/도] 후보
      - [대한민국] : ['country', 'political'] 


--- [Result #1] ---
formatted_address: 대한민국 부산광역시 부산진구 양정동 336-8
place_id: ChIJSxO55ansaDUR5nxXg4wSYss
types: ['street_address']
   📂 Address Components (상세 요소):
      - [336-8] : ['premise'] 
      - [양정동] : ['political', 'sublocality', 'sublocality_level_2'] 
      - [부산진구] : ['political', 'sublocality', 'sublocality_level_1'] 👈 [기초/구] 후보 (Type A)
      - [부산광역시] : ['administrative_area_level_1', 'political'] 👈 [광역/도] 후보
      - [대한민국] : ['country', 'political'] 
      - [47221] : ['p

In [1]:
import googlemaps
import os
from dotenv import load_dotenv

# 1. 환경변수 로드
load_dotenv() 

# 2. 클라이언트 초기화
api_key = os.getenv("GMAPS_API_KEY") # 또는 직접 입력: "YOUR_KEY"
if not api_key:
    print("❌ API 키가 설정되지 않았습니다.")
else:
    GMAPS_CLIENT = googlemaps.Client(key=api_key)
    print("✅ Google Maps Client 연결 성공!")

✅ Google Maps Client 연결 성공!


In [2]:
def get_coordinates(location_name):
    """지명 -> 좌표 변환"""
    try:
        res = GMAPS_CLIENT.geocode(location_name, language='ko')
        if res:
            loc = res[0]['geometry']['location']
            return loc['lat'], loc['lng']
    except Exception as e:
        print(f"❌ 좌표 변환 에러: {e}")
    return None, None

def get_admin_district_smart_merge(lat, lng):
    """
    [핵심 테스트 대상]
    좌표 -> 행정구역 변환 (Smart Merge 적용)
    Result #0에 정보가 없으면 #1, #2...를 뒤져서라도 '광역+기초'를 찾아냄.
    """
    try:
        # 1. API 호출
        results = GMAPS_CLIENT.reverse_geocode((lat, lng), language='ko')
        if not results: return "결과 없음"

        print(f"   🔎 구글 반환 결과 개수: {len(results)}개")
        
        # 2. 가장 적절한 행정구역 찾기
        best_do = ""
        best_si_gu = ""

        # 결과 리스트 순회 (상위 5개)
        for i, result in enumerate(results[:5]):
            comps = result.get('address_components', [])
            current_do = ""
            current_si_gu = ""

            # 컴포넌트 분석
            for comp in comps:
                types = comp.get('types', [])
                if 'administrative_area_level_1' in types:
                    current_do = comp['long_name']
                elif 'sublocality_level_1' in types:
                    current_si_gu = comp['long_name']
                elif 'locality' in types:
                    if not current_si_gu:
                        current_si_gu = comp['long_name']
            
            print(f"      [Result #{i}] 분석: 광역='{current_do}', 기초='{current_si_gu}'")

            # [전략 A] 완벽한 세트 발견 시 즉시 반환
            if current_do and current_si_gu:
                print(f"      🎉 Result #{i}에서 완벽한 매칭 성공! (Smart Merge 작동)")
                return f"{current_do} {current_si_gu}".strip()
            
            # [전략 B] 백업용 정보 수집
            if not best_do and current_do: best_do = current_do
            if not best_si_gu and current_si_gu: best_si_gu = current_si_gu

        # 완벽한 세트가 없으면 조합해서 반환
        final_result = f"{best_do} {best_si_gu}".strip()
        print(f"      ⚠️ 완벽 매칭 실패. 조합 결과 반환: {final_result}")
        return final_result

    except Exception as e:
        return f"에러: {e}"

In [3]:
# 테스트할 장소 리스트
test_cases = [
    "부산",       # [난이도 상] Result #0에 '구' 정보가 없는 대표 케이스
    "부산역",     # [난이도 중] POI가 섞여서 주소가 복잡한 케이스
    "광안리",     # [난이도 중] 지명(POI) -> 행정구역 변환
    "해운대",     # [난이도 하] 비교적 명확한 지명
    "서울 강남",  # [난이도 하] 일반적인 검색
    "가평"        # [난이도 중] 군 단위 테스트
]

print(f"🚀 [검증 시작] 총 {len(test_cases)}개 케이스 테스트\n" + "="*60)

for place in test_cases:
    print(f"\n📍 입력 검색어: [{place}]")
    
    # 1. 좌표 획득
    lat, lng = get_coordinates(place)
    
    if lat and lng:
        print(f"   ✅ 좌표 획득: {lat}, {lng}")
        
        # 2. 행정구역 변환 (Smart Merge)
        final_admin = get_admin_district_smart_merge(lat, lng)
        
        print(f"   🎯 최종 변환 결과: [{final_admin}]")
        
        # 3. 간단 검증 (기초지자체가 포함되었는지 확인)
        if " " in final_admin:
            print("   ✅ PASS: 광역/기초 분리 성공")
        else:
            print("   ⚠️ WARNING: 기초지자체(시/군/구)가 누락되었을 수 있음")
    else:
        print("   ❌ 좌표 변환 실패")
        
print("\n" + "="*60 + "\n[검증 종료]")

🚀 [검증 시작] 총 6개 케이스 테스트

📍 입력 검색어: [부산]
   ✅ 좌표 획득: 35.1731121, 129.0714122
   🔎 구글 반환 결과 개수: 9개
      [Result #0] 분석: 광역='부산광역시', 기초=''
      [Result #1] 분석: 광역='부산광역시', 기초='부산진구'
      🎉 Result #1에서 완벽한 매칭 성공! (Smart Merge 작동)
   🎯 최종 변환 결과: [부산광역시 부산진구]
   ✅ PASS: 광역/기초 분리 성공

📍 입력 검색어: [부산역]
   ✅ 좌표 획득: 35.114495, 129.03933
   🔎 구글 반환 결과 개수: 11개
      [Result #0] 분석: 광역='부산광역시', 기초=''
      [Result #1] 분석: 광역='부산광역시', 기초='동구'
      🎉 Result #1에서 완벽한 매칭 성공! (Smart Merge 작동)
   🎯 최종 변환 결과: [부산광역시 동구]
   ✅ PASS: 광역/기초 분리 성공

📍 입력 검색어: [광안리]
   ✅ 좌표 획득: 35.1531696, 129.118666
   🔎 구글 반환 결과 개수: 8개
      [Result #0] 분석: 광역='부산광역시', 기초='수영구'
      🎉 Result #0에서 완벽한 매칭 성공! (Smart Merge 작동)
   🎯 최종 변환 결과: [부산광역시 수영구]
   ✅ PASS: 광역/기초 분리 성공

📍 입력 검색어: [해운대]
   ✅ 좌표 획득: 35.1719465, 129.1741038
   🔎 구글 반환 결과 개수: 8개
      [Result #0] 분석: 광역='부산광역시', 기초='해운대구'
      🎉 Result #0에서 완벽한 매칭 성공! (Smart Merge 작동)
   🎯 최종 변환 결과: [부산광역시 해운대구]
   ✅ PASS: 광역/기초 분리 성공

📍 입력 검색어: [서울 강남]
   ✅ 좌표 획득: 37.49666

In [4]:
# 날씨 테스트

import googlemaps
import requests
import os
import datetime
import json
from dotenv import load_dotenv

# 1. 환경변수 로드
load_dotenv()

# 2. API 키 가져오기 (없으면 직접 입력)
GMAPS_KEY = os.getenv("GMAPS_API_KEY") # or "YOUR_GMAPS_KEY"
OWM_KEY = os.getenv("OWM_API_KEY")     # or "YOUR_OWM_KEY"

# 3. 클라이언트 초기화
if not GMAPS_KEY or not OWM_KEY:
    print("❌ API 키가 부족합니다. .env 파일을 확인하거나 직접 키를 입력하세요.")
else:
    gmaps = googlemaps.Client(key=GMAPS_KEY)
    print("✅ API 설정 완료")

✅ API 설정 완료


In [5]:
def get_coordinates_debug(place_name):
    """구글맵으로 좌표를 따오는 헬퍼"""
    try:
        print(f"   [Google Maps] '{place_name}' 검색 중...")
        res = gmaps.geocode(place_name, language='ko')
        if res:
            loc = res[0]['geometry']['location']
            addr = res[0].get('formatted_address', '주소 미상')
            print(f"   ✅ 좌표 획득 성공: {loc['lat']}, {loc['lng']} ({addr})")
            return loc['lat'], loc['lng']
        else:
            print("   ❌ Google Maps 결과 없음")
    except Exception as e:
        print(f"   ❌ Google Maps 에러: {e}")
    return None, None

def debug_weather_forecast(destination, date_str="오늘"):
    """
    [핵심 테스트] 지명 -> 좌표 -> 날씨 조회 프로세스 검증
    """
    print(f"\n🚀 [테스트 시작] 목적지: {destination}, 날짜: {date_str}")
    
    # 1. 좌표 변환 (Google First)
    lat, lon = get_coordinates_debug(destination)
    
    # 2. 좌표 실패 시 OWM Fallback (테스트용)
    if not lat or not lon:
        print("   ⚠️ Google 좌표 실패. OWM 텍스트 검색 시도...")
        try:
            geo_url = "https://api.openweathermap.org/geo/1.0/direct"
            res = requests.get(geo_url, params={'q': f"{destination},KR", 'limit': 1, 'appid': OWM_KEY})
            data = res.json()
            if data:
                lat, lon = data[0]['lat'], data[0]['lon']
                print(f"   ✅ OWM Geocoding 성공: {lat}, {lon}")
            else:
                print("   ❌ OWM Geocoding도 실패. 종료.")
                return
        except Exception as e:
            print(f"   ❌ OWM API 에러: {e}")
            return

    # 3. 날씨 예보 요청 (좌표 사용)
    print(f"   ☁️ [OWM Forecast] 좌표({lat}, {lon})로 날씨 조회 중...")
    try:
        url = "https://api.openweathermap.org/data/2.5/forecast"
        params = {
            'lat': lat, 'lon': lon, 
            'appid': OWM_KEY, 
            'units': 'metric', 
            'lang': 'kr'
        }
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"   ❌ API 응답 실패: {response.status_code} / {response.text}")
            return

        data = response.json()
        city_name = data.get('city', {}).get('name', 'Unknown')
        forecasts = data.get('list', [])
        print(f"   ✅ 데이터 수신 완료 (감지된 도시명: {city_name})")
        
        # 4. 결과 파싱 (오늘 날짜 기준 5개만 출력해봄)
        print("\n   📊 [수신 데이터 샘플 (UTC 기준)]")
        count = 0
        for item in forecasts:
            dt_txt = item['dt_txt']
            temp = item['main']['temp']
            desc = item['weather'][0]['description']
            
            # 오늘/내일 데이터만 살짝 보여주기
            print(f"      - {dt_txt}: {temp}℃, {desc}")
            count += 1
            if count >= 5: break # 5개만 찍고 종료
            
    except Exception as e:
        print(f"   ❌ 예보 조회 중 치명적 에러: {e}")

In [6]:
# 테스트 케이스
test_locations = [
    "광안리",    # 아까 실패했던 케이스 (POI)
    "서면",      # 부산 번화가 (POI)
    "부산역",    # 역 (POI)
    "해운대",    # 지명
    "제주도"     # 광역
]

for loc in test_locations:
    debug_weather_forecast(loc)
    print("-" * 50)
    


🚀 [테스트 시작] 목적지: 광안리, 날짜: 오늘
   [Google Maps] '광안리' 검색 중...
   ✅ 좌표 획득 성공: 35.1531696, 129.118666 (대한민국 부산광역시 광안리해수욕장)
   ☁️ [OWM Forecast] 좌표(35.1531696, 129.118666)로 날씨 조회 중...
   ✅ 데이터 수신 완료 (감지된 도시명: Busan)

   📊 [수신 데이터 샘플 (UTC 기준)]
      - 2025-12-04 18:00:00: 1.1℃, 맑음
      - 2025-12-04 21:00:00: -0.19℃, 맑음
      - 2025-12-05 00:00:00: -0.05℃, 맑음
      - 2025-12-05 03:00:00: 2.73℃, 맑음
      - 2025-12-05 06:00:00: 4.67℃, 맑음
--------------------------------------------------

🚀 [테스트 시작] 목적지: 서면, 날짜: 오늘
   [Google Maps] '서면' 검색 중...
   ✅ 좌표 획득 성공: 38.0109074, 128.5250862 (대한민국 강원특별자치도 양양군 서면)
   ☁️ [OWM Forecast] 좌표(38.0109074, 128.5250862)로 날씨 조회 중...
   ✅ 데이터 수신 완료 (감지된 도시명: Yangyang)

   📊 [수신 데이터 샘플 (UTC 기준)]
      - 2025-12-04 18:00:00: -5.99℃, 구름조금
      - 2025-12-04 21:00:00: -9.29℃, 구름조금
      - 2025-12-05 00:00:00: -9.76℃, 맑음
      - 2025-12-05 03:00:00: -4.3℃, 맑음
      - 2025-12-05 06:00:00: -3.09℃, 맑음
--------------------------------------------------

🚀 [테스트 시작] 목적지: 부산

In [7]:
# [Cell 1] 라이브러리 및 설정
import googlemaps
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GMAPS_API_KEY")

if not api_key:
    print("❌ API 키 없음")
else:
    GMAPS_CLIENT = googlemaps.Client(key=api_key)
    print("✅ Google Maps Client 연결 성공!")

# -----------------------------------------------------------
# 1. 헬퍼 함수들 (좌표 변환 & 행정구역 추출)
# -----------------------------------------------------------

def get_geocode_result(query):
    """검색어로 좌표와 포맷된 주소를 가져옵니다."""
    try:
        res = GMAPS_CLIENT.geocode(query, language='ko')
        if res:
            loc = res[0]['geometry']['location']
            addr = res[0]['formatted_address']
            return loc['lat'], loc['lng'], addr
    except Exception as e:
        return None, None, str(e)
    return None, None, "결과 없음"

def get_admin_area_level_1(lat, lng):
    """좌표에서 '광역지자체명'(예: 부산광역시)만 쏙 뽑아냅니다."""
    try:
        res = GMAPS_CLIENT.reverse_geocode((lat, lng), language='ko')
        if not res: return None
        
        for comp in res[0]['address_components']:
            if 'administrative_area_level_1' in comp['types']:
                return comp['long_name'] # 예: 부산광역시, 강원특별자치도
    except:
        pass
    return None

# -----------------------------------------------------------
# 2. [핵심] 문맥 보정 로직 테스트 함수
# -----------------------------------------------------------

def test_context_aware_search(destination, start_location):
    print(f"\n🧪 [테스트 케이스] 목적지: '{destination}' / 출발지: '{start_location}'")
    print("="*60)

    # --- Case A: 그냥 검색했을 때 (Bad Case) ---
    lat_a, lng_a, addr_a = get_geocode_result(destination)
    print(f"❌ [보정 전] 검색어: '{destination}'")
    print(f"   -> 결과 주소: {addr_a}")
    print(f"   -> 좌표: {lat_a}, {lng_a}")
    
    if not start_location:
        print("   (출발지 정보가 없어 보정 테스트 생략)")
        return

    print("-" * 60)

    # --- Case B: 문맥 보정 로직 적용 (Good Case) ---
    print(f"🔄 [보정 로직 작동 중...]")
    
    # 1. 출발지 좌표 확인
    s_lat, s_lng, s_addr = get_geocode_result(start_location)
    if not s_lat:
        print("   ⚠️ 출발지 좌표를 못 찾아 보정 불가.")
        return

    # 2. 출발지의 광역 지자체 확인
    start_province = get_admin_area_level_1(s_lat, s_lng)
    print(f"   1. 출발지('{start_location}')의 광역 지역 파악 -> '{start_province}'")

    if not start_province:
        print("   ⚠️ 광역 지역 명칭 추출 실패.")
        return

    # 3. 검색어 재조합 (Destination이 짧을 경우)
    refined_query = destination
    if len(destination) <= 3: # "서면", "중구" 등 짧은 지명
        refined_query = f"{start_province} {destination}"
        print(f"   2. 지명이 짧아 문맥 추가함: '{destination}' -> '{refined_query}'")
    else:
        print(f"   2. 지명이 충분히 길어 보정 안함.")

    # 4. 보정된 검색어로 재검색
    lat_b, lng_b, addr_b = get_geocode_result(refined_query)
    
    print(f"\n✅ [보정 후] 검색어: '{refined_query}'")
    print(f"   -> 결과 주소: {addr_b}")
    print(f"   -> 좌표: {lat_b}, {lng_b}")

    # 검증
    if addr_b and start_province in addr_b:
        print("\n✨ 결론: 성공! 출발지 지역 근처로 검색됨.")
    else:
        print("\n❓ 결론: 여전히 다른 지역이거나 판단 불가.")

# 테스트 시나리오
scenarios = [
    ("서면", "부산역"),      # [핵심] 강원도 양양 vs 부산 부산진구
    ("중구", "대구역"),      # [중복지명] 서울 중구 vs 대구 중구
    ("중구", "부산역"),      # [중복지명] 서울 중구 vs 부산 중구
    ("광안리", "부산역"),    # [일반] 보정이 필요 없거나 잘 되는지 확인
    ("양양", "서울역")       # [거리멂] 서울 사람이 양양 갈 때 (양양은 짧지만 고유명사라 강원도로 잘 잡혀야 함)
]

for dest, start in scenarios:
    test_context_aware_search(dest, start)

✅ Google Maps Client 연결 성공!

🧪 [테스트 케이스] 목적지: '서면' / 출발지: '부산역'
❌ [보정 전] 검색어: '서면'
   -> 결과 주소: 대한민국 강원특별자치도 양양군 서면
   -> 좌표: 38.0109074, 128.5250862
------------------------------------------------------------
🔄 [보정 로직 작동 중...]
   1. 출발지('부산역')의 광역 지역 파악 -> '부산광역시'
   2. 지명이 짧아 문맥 추가함: '서면' -> '부산광역시 서면'

✅ [보정 후] 검색어: '부산광역시 서면'
   -> 결과 주소: 대한민국 부산광역시 부산진구 중앙대로 지하 730 서면역
   -> 좌표: 35.15781570000001, 129.0600331

✨ 결론: 성공! 출발지 지역 근처로 검색됨.

🧪 [테스트 케이스] 목적지: '중구' / 출발지: '대구역'
❌ [보정 전] 검색어: '중구'
   -> 결과 주소: 대한민국 서울특별시 중구
   -> 좌표: 37.5601443, 126.9959649
------------------------------------------------------------
🔄 [보정 로직 작동 중...]
   1. 출발지('대구역')의 광역 지역 파악 -> '대구광역시'
   2. 지명이 짧아 문맥 추가함: '중구' -> '대구광역시 중구'

✅ [보정 후] 검색어: '대구광역시 중구'
   -> 결과 주소: 대한민국 대구광역시 중구
   -> 좌표: 35.8654887, 128.5933653

✨ 결론: 성공! 출발지 지역 근처로 검색됨.

🧪 [테스트 케이스] 목적지: '중구' / 출발지: '부산역'
❌ [보정 전] 검색어: '중구'
   -> 결과 주소: 대한민국 서울특별시 중구
   -> 좌표: 37.5601443, 126.9959649
---------------------------------------------------

In [1]:
import pandas as pd
import os
import shutil
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from src.config import review_faiss # config.py 경로 확인 필요
from src.rag_updater import update_vector_db_if_needed, create_documents_from_df

# 테스트용 파일 이름 설정
TEST_REVIEWS_FILE = "test_new_reviews.csv"
ORIGINAL_FAISS_PATH = review_faiss # 원본 DB 경로
BACKUP_FAISS_PATH = "faiss_db_backup" # 백업용 경로 (안전장치)

# 원본 DB 백업 (테스트 후 복구용)
if os.path.exists(ORIGINAL_FAISS_PATH):
    if os.path.exists(BACKUP_FAISS_PATH):
        shutil.rmtree(BACKUP_FAISS_PATH)
    shutil.copytree(ORIGINAL_FAISS_PATH, BACKUP_FAISS_PATH)
    print(f"✅ 원본 DB 백업 완료: {BACKUP_FAISS_PATH}")
else:
    print("❌ 원본 FAISS DB가 없습니다. 경로를 확인하세요.")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ 원본 DB 백업 완료: faiss_db_backup


In [8]:
# 1. 테스트용 가짜 리뷰 데이터 생성 (9개 - 업데이트 기준 미달)
reviews_under_10 = [
    {"지역": "서울 종로구", "장소명": "경복궁", "카테고리_통합": "관광지", "리뷰": "야간 개장이 너무 예뻤어요. 한복 입고 가면 무료입장이라 좋았습니다.", "평점": 5},
    {"지역": "부산 해운대구", "장소명": "해운대암소갈비", "카테고리_통합": "식당", "리뷰": "웨이팅이 길었지만 감자면 사리는 정말 맛있네요.", "평점": 4},
    {"지역": "제주 서귀포시", "장소명": "올레시장", "카테고리_통합": "관광지", "리뷰": "먹거리가 다양해서 좋았습니다. 오메기떡 추천!", "평점": 5},
    {"지역": "강원 강릉시", "장소명": "안목해변 카페거리", "카테고리_통합": "카페", "리뷰": "바다 뷰가 환상적입니다. 커피 맛은 평범해요.", "평점": 3},
    {"지역": "경주", "장소명": "첨성대", "카테고리_통합": "관광지", "리뷰": "밤에 조명이 켜지니 사진 찍기 좋습니다.", "평점": 4},
    {"지역": "전주 완산구", "장소명": "전주한옥마을", "카테고리_통합": "관광지", "리뷰": "길거리 음식이 맛있고 한복 대여해서 입기 좋아요.", "평점": 5},
    {"지역": "서울 마포구", "장소명": "망원시장", "카테고리_통합": "시장", "리뷰": "고로케랑 닭강정이 맛있습니다. 사람 많아요.", "평점": 4},
    {"지역": "부산 수영구", "장소명": "광안리 해수욕장", "카테고리_통합": "관광지", "리뷰": "드론쇼 볼만합니다. 야경이 끝내줘요.", "평점": 5},
    {"지역": "여수", "장소명": "여수밤바다", "카테고리_통합": "관광지", "리뷰": "낭만포차 거리 분위기 좋습니다.", "평점": 4}
]

# CSV로 저장
df_9 = pd.DataFrame(reviews_under_10)
df_9.to_csv(TEST_REVIEWS_FILE, index=False, encoding="utf-8-sig")

print(f"✅ 리뷰 9개 저장 완료 ({TEST_REVIEWS_FILE})")
print(df_9.head(2)) # 데이터 확인

✅ 리뷰 9개 저장 완료 (test_new_reviews.csv)
        지역      장소명 카테고리_통합                                      리뷰  평점
0   서울 종로구      경복궁     관광지  야간 개장이 너무 예뻤어요. 한복 입고 가면 무료입장이라 좋았습니다.   5
1  부산 해운대구  해운대암소갈비      식당              웨이팅이 길었지만 감자면 사리는 정말 맛있네요.   4


In [9]:
# 업데이트 함수 호출 (10개 미만이라 업데이트 안 되어야 함)
result = update_vector_db_if_needed(TEST_REVIEWS_FILE)
print(f"👉 결과: {result}")

# 파일이 그대로 있는지 확인
if os.path.exists(TEST_REVIEWS_FILE):
    print("✅ 파일이 삭제되지 않고 유지됨 (정상)")
else:
    print("❌ 파일이 삭제됨 (오류)")

👉 결과: 리뷰 9개 누적됨. (10개 이상이어야 업데이트)
✅ 파일이 삭제되지 않고 유지됨 (정상)


In [10]:
# 리뷰 1개 추가 (총 10개 만들기)
new_review = {"지역": "대전 유성구", "장소명": "성심당", "카테고리_통합": "빵집", "리뷰": "튀김소보로 줄이 길지만 기다릴 가치가 있습니다. 명란바게트도 최고!", "평점": 5}
df_new = pd.DataFrame([new_review])
df_new.to_csv(TEST_REVIEWS_FILE, mode='a', header=False, index=False, encoding="utf-8-sig")

print("✅ 리뷰 1개 추가 완료 (총 10개)")

# 업데이트 함수 호출 (이제 업데이트 되어야 함)
# 주의: 실제 DB가 수정되므로 백업 필수 (Cell 1에서 했음)
print("🔄 DB 업데이트 시도 중...")
result = update_vector_db_if_needed(TEST_REVIEWS_FILE)
print(f"👉 결과: {result}")

# 파일 삭제 확인
if not os.path.exists(TEST_REVIEWS_FILE):
    print("✅ CSV 파일이 정상적으로 삭제됨 (업데이트 완료)")
else:
    print("❌ CSV 파일이 남아있음 (삭제 실패)")

2025-12-05 00:53:44.744 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 00:53:44.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 00:53:44.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ 리뷰 1개 추가 완료 (총 10개)
🔄 DB 업데이트 시도 중...
--- [RAG Updater] 리뷰 10개 DB 업데이트 시작 ---
[RAG Updater] 10개의 새 문서를 생성했습니다.
[RAG Updater] 기존 FAISS 인덱스를 로드했습니다.
[RAG Updater] FAISS 인덱스에 새 문서를 추가했습니다.


2025-12-05 00:53:53.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 00:53:53.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 00:53:53.378 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


[RAG Updater] FAISS 인덱스를 로컬에 저장했습니다.
[RAG Updater] Streamlit 캐시를 삭제했습니다.
[RAG Updater] test_new_reviews.csv 파일을 삭제했습니다.
👉 결과: 벡터 DB 업데이트 완료!
✅ CSV 파일이 정상적으로 삭제됨 (업데이트 완료)


In [11]:
# 1. 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(
    model_name="upskyy/bge-m3-korean",
    model_kwargs={"device": "cpu"}
)

# 2. 업데이트된 DB 로드
new_db = FAISS.load_local(review_faiss, embeddings, allow_dangerous_deserialization=True)
print(f"✅ DB 로드 완료. 총 문서 수: {new_db.index.ntotal}")

# 3. 방금 추가한 '성심당' 검색으로 검증
query = "성심당 튀김소보로"
docs = new_db.similarity_search(query, k=3)

print("\n🔍 [검색 결과 검증]")
for i, doc in enumerate(docs):
    print(f"\n--- 문서 #{i+1} ---")
    print(f"내용: {doc.page_content[:100]}...")
    print(f"메타데이터: {doc.metadata}")
    
    # 메타데이터 양식 체크
    required_keys = ['place_name', 'region', 'category', 'rating']
    if all(key in doc.metadata for key in required_keys):
        print("✅ 메타데이터 양식 정상")
    else:
        print(f"❌ 메타데이터 양식 오류 (필수 키 누락): {doc.metadata.keys()}")

✅ DB 로드 완료. 총 문서 수: 168601

🔍 [검색 결과 검증]

--- 문서 #1 ---
내용: 성심당은(는) 대전 유성구에 위치한 빵집입니다. 리뷰 내용은 다음과 같습니다: 튀김소보로 줄이 길지만 기다릴 가치가 있습니다 명란바게트도 최고...
메타데이터: {'장소명': '성심당', '지역': '대전 유성구', '카테고리': '빵집', '별점': '5', '리뷰': '튀김소보로 줄이 길지만 기다릴 가치가 있습니다. 명란바게트도 최고!'}
❌ 메타데이터 양식 오류 (필수 키 누락): dict_keys(['장소명', '지역', '카테고리', '별점', '리뷰'])

--- 문서 #2 ---
내용: 해운대암소갈비은(는) 부산 해운대구에 위치한 식당입니다. 리뷰 내용은 다음과 같습니다: 웨이팅이 길었지만 감자면 사리는 정말 맛있네요...
메타데이터: {'장소명': '해운대암소갈비', '지역': '부산 해운대구', '카테고리': '식당', '별점': '4', '리뷰': '웨이팅이 길었지만 감자면 사리는 정말 맛있네요.'}
❌ 메타데이터 양식 오류 (필수 키 누락): dict_keys(['장소명', '지역', '카테고리', '별점', '리뷰'])

--- 문서 #3 ---
내용: 지역: 대전광역시 동구 | 장소명: 성심당 대전역점 | 카테고리: 맛집 | 리뷰: 현장에서 직접 튀소 오픈키친 과정을 볼 수 있게 한것이 직접 매력적이었고, 튀소랑 부추빵 보문산메...
메타데이터: {'지역': '대전광역시 동구', '카테고리': '맛집', '장소명': '성심당 대전역점', '별점': 4.1, '상세 주소': '대전 동구 중앙로 215 2층', '리뷰': '현장에서 직접 튀소 오픈키친 과정을 볼 수 있게 한것이 직접 매력적이었고, 튀소랑 부추빵 보문산메아리 진짜 맛있게 잘 먹었습니다. 소방안전관리자 1급 재시험 보러 대전에 왔다가 버스시간이 1시간 40분 남아서 성심당 대전역점에 직접 방문한것은 10년만에 다시 와서 진짜 감회가 새로웠습니다. 저는 20분만에 줄을 서서

In [13]:
# 원본 복구
if os.path.exists(BACKUP_FAISS_PATH):
    if os.path.exists(review_faiss):
        shutil.rmtree(review_faiss)
    shutil.copytree(BACKUP_FAISS_PATH, review_faiss)
    print("✅ 원본 DB로 복구 완료")
else:
    print("⚠️ 백업 파일이 없어 복구할 수 없습니다.")

✅ 원본 DB로 복구 완료


In [14]:
# [Cell 7] 주소 자동 채우기 기능 테스트

# 1. 테스트용 리뷰 생성 (상세 주소 누락시킴)
# 가정: '성심당'은 이미 DB에 주소('대전 중구...')가 있는 상태라고 가정
test_no_address = [
    {"지역": "대전", "장소명": "성심당", "카테고리_통합": "빵집", "리뷰": "튀소 먹으러 왔는데 역시나 맛있네요. 주소 쓰는 건 귀찮아서 패스.", "평점": 5}
]
# 10개를 채워야 하니 나머지 9개는 더미로 채움
for i in range(9):
    test_no_address.append({"지역": "테스트", "장소명": f"테스트장소{i}", "리뷰": "테스트", "평점": 5})

# CSV 저장
df_test = pd.DataFrame(test_no_address)
df_test.to_csv("test_address_fill.csv", index=False, encoding="utf-8-sig")

print("✅ 테스트 파일 생성 (주소 없는 성심당 리뷰 포함)")

# 2. 업데이트 실행 (rag_updater가 DB를 조회해서 주소를 채울 것임)
from src.rag_updater import update_vector_db_if_needed
update_vector_db_if_needed("test_address_fill.csv")

# 3. 결과 확인
# DB 다시 로드해서 가장 최근에 추가된(혹은 검색된) 성심당 문서 확인
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from src.config import review_faiss

embeddings = HuggingFaceEmbeddings(model_name="upskyy/bge-m3-korean", model_kwargs={"device": "cpu"})
db = FAISS.load_local(review_faiss, embeddings, allow_dangerous_deserialization=True)

# 방금 넣은 리뷰 내용으로 검색
docs = db.similarity_search("주소 쓰는 건 귀찮아서 패스", k=1)

print("\n🔍 [주소 자동 채우기 검증]")
if docs:
    doc = docs[0]
    print(f"장소명: {doc.metadata.get('장소명')}")
    print(f"리뷰내용: {doc.metadata.get('리뷰')}")
    print(f"채워진 주소: {doc.metadata.get('상세 주소')}")
    
    if doc.metadata.get('상세 주소'):
        print("🎉 성공! 기존 DB에서 주소를 가져와서 채웠습니다.")
    else:
        print("⚠️ 실패. 주소가 여전히 비어있습니다. (기존 DB에 성심당 정보가 없거나 매칭 실패)")
else:
    print("❌ 문서를 찾을 수 없습니다.")

2025-12-05 01:00:27.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 01:00:27.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 01:00:27.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ 테스트 파일 생성 (주소 없는 성심당 리뷰 포함)
--- [RAG Updater] 리뷰 10개 DB 업데이트 시작 ---
[RAG Updater] 1개의 새 문서를 생성했습니다.
[RAG Updater] 기존 FAISS 인덱스를 로드했습니다.
[RAG Updater] FAISS 인덱스에 새 문서를 추가했습니다.


2025-12-05 01:00:38.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 01:00:38.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 01:00:38.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


[RAG Updater] FAISS 인덱스를 로컬에 저장했습니다.
[RAG Updater] Streamlit 캐시를 삭제했습니다.
[RAG Updater] test_address_fill.csv 파일을 삭제했습니다.

🔍 [주소 자동 채우기 검증]
장소명: 오브제토
리뷰내용: 응대도 귀찮은듯. 계산 손님이 새치기해도 여자 사장 지인 인지  그냥 넘어감
채워진 주소: 경북 청도군 화양읍 이슬미로 61-7
🎉 성공! 기존 DB에서 주소를 가져와서 채웠습니다.


In [15]:
# pip install fpdf2  # 혹시 안 깔려있다면 설치
from fpdf import FPDF
from fpdf.enums import XPos, YPos
import os

# 폰트 파일이 있는지 확인 (없으면 테스트 불가하므로 경로 수정 필요)
if not os.path.exists("NanumGothic.ttf"):
    print("❌ 경고: 현재 폴더에 'NanumGothic.ttf' 파일이 없습니다.")
    print("   테스트를 위해 폰트 파일을 노트북과 같은 폴더에 복사해주세요.")
else:
    print("✅ 폰트 파일 확인 완료.")

✅ 폰트 파일 확인 완료.


In [16]:
# pages/1_trip_planner.py

from fpdf import FPDF
from fpdf.enums import XPos, YPos
import os

def create_itinerary_pdf(itinerary, destination, dates, weather, final_routes, total_days):
    """
    여행 계획 PDF 생성 함수 (SmartScheduler 이동 경로 반영 & 최신 FPDF2 문법 적용)
    """
    pdf = FPDF()
    pdf.add_page()
    
    # 1. 폰트 로드 (NanumGothic)
    font_path = 'NanumGothic.ttf'
    bold_font_path = 'NanumGothicBold.ttf'
    
    # 폰트 파일 존재 여부 확인 및 등록
    has_korean_font = False
    try:
        if os.path.exists(font_path):
            pdf.add_font('NanumGothic', '', font_path)
            if os.path.exists(bold_font_path):
                pdf.add_font('NanumGothic', 'B', bold_font_path)
            else:
                pdf.add_font('NanumGothic', 'B', font_path) # 볼드 없으면 일반으로 대체
            
            pdf.set_font('NanumGothic', '', 12)
            has_korean_font = True
        else:
            print("⚠️ [PDF 생성] 폰트 파일이 없습니다. 기본 폰트(Arial)를 사용합니다.")
            pdf.set_font('Arial', '', 12)
    except Exception as e:
        print(f"⚠️ [PDF 생성] 폰트 로드 에러: {e}")
        return None

    # 2. 헤더 (여행지 및 기간)
    pdf.set_font_size(24)
    # ln=True -> new_x=XPos.LMARGIN, new_y=YPos.NEXT 로 변경
    pdf.cell(0, 20, text=f"{destination} 여행 계획", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
    
    pdf.set_font_size(12)
    pdf.cell(0, 10, text=f"기간: {dates} | 날씨: {weather if weather else '정보 없음'}", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
    pdf.ln(10)

    # 안전한 정렬 (day 키 기준)
    try:
        sorted_itinerary = sorted(itinerary, key=lambda x: int(x.get('day', 1)))
    except:
        sorted_itinerary = itinerary

    # 3. 일차별 상세 일정 작성
    for day_num in range(1, total_days + 1):
        # 날짜 헤더
        pdf.set_font_size(18)
        if has_korean_font: pdf.set_font('NanumGothic', 'B', 18)
        
        pdf.cell(0, 15, text=f"Day {day_num}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        
        # 폰트 원복
        pdf.set_font_size(11)
        if has_korean_font: pdf.set_font('NanumGothic', '', 11)

        # 해당 날짜 아이템 필터링
        items_today = [item for item in sorted_itinerary if int(item.get('day', 1)) == day_num]
        
        if not items_today:
            pdf.cell(0, 10, text="  - 계획된 일정이 없습니다.", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
            pdf.ln(5)
            continue

        for item in items_today:
            item_type = item.get('type', 'activity')

            # --- [Case A] 이동 정보 (SmartScheduler가 생성한 'move') ---
            if item_type == 'move':
                # 이동은 회색으로 작게 표시하여 시각적 구분
                pdf.set_text_color(100, 100, 100) # Gray
                pdf.set_font_size(10)
                
                start_t = item.get('start', '')
                end_t = item.get('end', '')
                duration = item.get('duration_text', '')
                transport = item.get('transport', '이동')
                
                # 예: "⬇️ 10:30~11:00 (30분) : 1003번 버스"
                move_text = f"      ⬇️  {start_t} ~ {end_t} ({duration}) : {transport}"
                pdf.cell(0, 8, text=move_text, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
                
                # 색상 및 크기 원복
                pdf.set_text_color(0, 0, 0) # Black
                pdf.set_font_size(11)

            # --- [Case B] 장소 방문 (activity/식당/관광지 등) ---
            else:
                # 시간 정보
                start_t = item.get('start', '')
                end_t = item.get('end', '')
                time_info = f"[{start_t}-{end_t}]" if start_t else "[시간 미정]"
                
                place_name = item.get('name', '이름 없음')
                category = item.get('category', item.get('type', '장소'))
                
                # 제목 라인 (볼드)
                if has_korean_font: pdf.set_font('NanumGothic', 'B', 12)
                
                main_text = f"  ● {time_info} {place_name} ({category})"
                pdf.cell(0, 8, text=main_text, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
                
                # 설명 라인 (일반)
                if item.get('description'):
                    if has_korean_font: pdf.set_font('NanumGothic', '', 10)
                    
                    # 들여쓰기 후 설명 출력
                    pdf.set_x(20) 
                    pdf.multi_cell(0, 5, text=f"{item['description']}")
                    pdf.ln(2)

        pdf.ln(10) # 날짜 간 간격

    return bytes(pdf.output())

In [17]:
# SmartScheduler가 만들었을 법한 데이터 (activity와 move가 섞여 있음)
mock_itinerary = [
    # --- Day 1 ---
    {
        "day": 1, "type": "activity", "name": "부산역", "category": "출발지",
        "start": "10:00", "end": "10:30", "description": "여행 시작! 짐 보관소에 짐 맡기기"
    },
    {
        "day": 1, "type": "move", 
        "start": "10:30", "end": "11:00", "duration_text": "30분", 
        "transport": "[버스] 1003번 (부산역 → 서면역)"
    },
    {
        "day": 1, "type": "activity", "name": "착한돼지 서면점", "category": "식당",
        "start": "11:00", "end": "12:30", "description": "가성비 좋은 고기 뷔페에서 점심 식사"
    },
    {
        "day": 1, "type": "move", 
        "start": "12:30", "end": "13:00", "duration_text": "30분", 
        "transport": "[지하철] 2호선 (서면역 → 광안리역)"
    },
    {
        "day": 1, "type": "activity", "name": "광안리 해수욕장", "category": "관광지",
        "start": "13:00", "end": "15:00", "description": "해변 산책 및 드론쇼 관람 예정"
    },
    
    # --- Day 2 ---
    {
        "day": 2, "type": "activity", "name": "호텔 체크아웃", "category": "숙소",
        "start": "11:00", "end": "11:30", "description": "체크아웃 후 짐 정리"
    }
]

# PDF 생성 실행
pdf_bytes = create_itinerary_pdf(
    itinerary=mock_itinerary,
    destination="부산",
    dates="2025-12-05 ~ 12-06",
    weather="맑음 (8도)",
    final_routes="", # 이건 이제 안 씀 (상세 일정에 포함됨)
    total_days=2
)

if pdf_bytes:
    # 파일로 저장해서 눈으로 확인
    output_filename = "test_smart_schedule.pdf"
    with open(output_filename, "wb") as f:
        f.write(pdf_bytes)
    print(f"🎉 PDF 생성 완료! '{output_filename}' 파일을 열어보세요.")
else:
    print("❌ PDF 생성 실패.")

Font MPDFAA+NanumGothic is missing the following glyphs: '⬇' (\u2b07), '️' (\ufe0f)
TSI0 NOT subset; don't know how to subset; dropped
TSI1 NOT subset; don't know how to subset; dropped
TSI2 NOT subset; don't know how to subset; dropped
TSI3 NOT subset; don't know how to subset; dropped
TSI5 NOT subset; don't know how to subset; dropped
TSI0 NOT subset; don't know how to subset; dropped
TSI1 NOT subset; don't know how to subset; dropped
TSI2 NOT subset; don't know how to subset; dropped
TSI3 NOT subset; don't know how to subset; dropped
TSI5 NOT subset; don't know how to subset; dropped


🎉 PDF 생성 완료! 'test_smart_schedule.pdf' 파일을 열어보세요.


In [1]:
import googlemaps
import os
import datetime
from dotenv import load_dotenv
from itertools import permutations
from src.scheduler.smart_scheduler import SmartScheduler # 사용자님의 스케줄러 클래스
from src.tools import get_detailed_route     # 사용자님의 경로 조회 함수

# 1. 환경변수 및 클라이언트 설정
load_dotenv()
api_key = os.getenv("GMAPS_API_KEY")

if not api_key:
    print("❌ API 키가 없습니다.")
else:
    GMAPS_CLIENT = googlemaps.Client(key=api_key)
    print("✅ Google Maps Client 연결 성공!")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ Google Maps Client 연결 성공!


In [4]:
# [Cell 2] 🛠️ 장소 명칭 보정 함수 (재시도 로직 추가)

def sanitize_places(places):
    print("--- 🛠️ 장소 명칭 검증 및 보정 시작 ---")
    sanitized_list = []
    
    for place in places:
        try:
            # 1차 시도: 원본 검색
            res = GMAPS_CLIENT.geocode(place, language='ko')
            
            # 2차 시도: 실패 시 '부산' 붙여서 검색 (문맥 추가)
            if not res and "부산" not in place:
                print(f"   ⚠️ '{place}' 1차 실패 -> '부산 {place}'로 재시도")
                res = GMAPS_CLIENT.geocode(f"부산 {place}", language='ko')
            
            if res:
                official_name = res[0].get('name') or res[0]['formatted_address']
                loc = res[0]['geometry']['location']
                lat, lng = loc['lat'], loc['lng']
                
                print(f"✅ '{place}' -> '{official_name}' (좌표: {lat}, {lng})")
                sanitized_list.append(official_name)
            else:
                print(f"❌ '{place}' : 끝내 찾을 수 없음. (원본 사용)")
                sanitized_list.append(place)
                
        except Exception as e:
            print(f"⚠️ 에러 발생 ({place}): {e}")
            sanitized_list.append(place)
            
    print("-" * 40)
    return sanitized_list

def debug_optimize_route(start_location, places, mode="transit"):
    # 1. 최적화 대상 리스트 (출발지 + 방문지)
    all_places = [start_location] + places
    print(f"\n--- 1. 경로 최적화 시작 ({mode}) ---")
    
    # 2. Distance Matrix API 호출
    now = datetime.datetime.now()
    try:
        matrix = GMAPS_CLIENT.distance_matrix(
            origins=all_places, destinations=all_places, 
            mode=mode, departure_time=now if mode == "transit" else None
        )
    except Exception as e:
        print(f"❌ 거리 계산 API 오류: {e}")
        return []

    # 3. 매트릭스 파싱 & 오류 체크
    duration_matrix = []
    for r_idx, row in enumerate(matrix['rows']):
        row_vals = []
        for c_idx, el in enumerate(row['elements']):
            status = el.get('status', 'ERROR')
            if status == 'OK':
                val = el.get('duration', {}).get('value', 0)
            else:
                # 🚨 어떤 구간이 에러인지 정확히 출력
                origin = all_places[r_idx]
                dest = all_places[c_idx]
                # print(f"   ⚠️ 경로 탐색 불가: {origin} -> {dest} (Status: {status})")
                val = float('inf')
            row_vals.append(val)
        duration_matrix.append(row_vals)

    # 4. TSP 알고리즘
    min_duration = float('inf')
    best_indices = []
    n = len(all_places)
    other_indices = list(range(1, n))
    
    for p in permutations(other_indices):
        current_indices = [0] + list(p)
        current_dur = sum(duration_matrix[current_indices[i]][current_indices[i+1]] for i in range(len(current_indices)-1))
        
        if current_dur < min_duration:
            min_duration = current_dur
            best_indices = current_indices

    # 5. 결과 반환
    if min_duration == float('inf'):
        print("⚠️ 경로 최적화 실패 (이동 불가 구간 존재 -> 원본 순서 사용)")
        return places # 최적화 실패 시 입력된 순서 그대로 반환 (출발지 제외)
    else:
        # 출발지 제외하고 방문지 순서만 반환
        optimized_places = [all_places[i] for i in best_indices if i != 0]
        full_path = [all_places[i] for i in best_indices]
        print(f"✅ 최적 순서: {' -> '.join(full_path)}")
        print(f"⏱️ 예상 총 이동 시간: 약 {min_duration // 60}분")
        return optimized_places

In [5]:
def run_full_simulation(start_node, raw_places, transport_mode="transit"):
    print("="*60)
    print(f"🚀 [통합 테스트] 이동 모드: {transport_mode}")
    print("="*60)
    
    # [Step 0] 명칭 보정 (Sanitize)
    # 출발지도 보정
    sanitized_start = sanitize_places([start_node])[0]
    sanitized_places = sanitize_places(raw_places)

    # [Step 1] 경로 최적화
    # 보정된 이름들을 넣음
    optimized_places = debug_optimize_route(sanitized_start, sanitized_places, mode=transport_mode)
    
    # [Step 2] 스케줄러 입력 데이터 생성
    scheduler_input = []
    
    # 출발지 (0분 체류)
    scheduler_input.append({
        "name": sanitized_start, "type": "숙소", "description": "출발"
    })
    
    # 방문지들
    for name in optimized_places:
        place_type = "관광지"
        if "식당" in name or "맛집" in name: place_type = "식당"
        elif "백화점" in name or "시장" in name: place_type = "쇼핑"
        
        scheduler_input.append({
            "name": name, "type": place_type, "description": "방문"
        })

    print(f"\n--- 2. 스마트 스케줄러 실행 ---")
    
    # [Step 3] 스케줄러 가동
    # 여기서 get_detailed_route가 보정된 이름("롯데백화점 부산본점")을 쓰므로 성공 확률 매우 높음
    scheduler = SmartScheduler(start_time_str="10:00")
    timeline = scheduler.plan_day(scheduler_input)

    # [Step 4] 결과 출력
    print("\n📊 [최종 타임라인 결과]")
    for item in timeline:
        if item['type'] == 'move':
            print(f"  ⬇️ [이동] {item['start']} ~ {item['end']} ({item['duration_text']})")
            transport_info = item.get('transport', '이동 정보 없음')
            print(f"     └ 수단: {transport_info}")
        else:
            print(f"  📍 [활동] {item['start']} ~ {item['end']} | {item['name']} ({item['category']})")
            print(f"     └ 체류: {item['duration_minutes']}분")

# --- 실행 ---
start = "부산역"
places_to_visit = [
    "해동용궁사",
    "태종대",
    "서면 롯데백화점" # 👈 이 친구가 '롯데백화점 부산본점'으로 바뀔 것입니다.
]

# 1. 대중교통 테스트
run_full_simulation(start, places_to_visit, transport_mode="transit")

# 2. 자차 테스트 (필요시 주석 해제)
run_full_simulation(start, places_to_visit, transport_mode="driving")

🚀 [통합 테스트] 이동 모드: transit
--- 🛠️ 장소 명칭 검증 및 보정 시작 ---
✅ '부산역' -> '대한민국 부산광역시 초량동 부산역' (좌표: 35.114495, 129.03933)
----------------------------------------
--- 🛠️ 장소 명칭 검증 및 보정 시작 ---
✅ '해동용궁사' -> '대한민국 부산광역시 기장군 용궁길 86' (좌표: 35.1883491, 129.2233197)
✅ '태종대' -> '대한민국 부산광역시 영도구 전망로 24 태종대유원지' (좌표: 35.05307, 129.0872)
   ⚠️ '서면 롯데백화점' 1차 실패 -> '부산 서면 롯데백화점'로 재시도
✅ '서면 롯데백화점' -> '대한민국 부산광역시 부산진구 가야대로 772' (좌표: 35.156827, 129.0564967)
----------------------------------------

--- 1. 경로 최적화 시작 (transit) ---
⚠️ 경로 최적화 실패 (이동 불가 구간 존재 -> 원본 순서 사용)

--- 2. 스마트 스케줄러 실행 ---
DEBUG: ⚠️ 경로 없음 (transit). 좌표 기반 추정 시도.
DEBUG: ⚠️ 경로 없음 (transit). 좌표 기반 추정 시도.

📊 [최종 타임라인 결과]
  📍 [활동] 10:00 ~ 10:00 | 대한민국 부산광역시 초량동 부산역 (숙소)
     └ 체류: 0분
  ⬇️ [이동] 10:00 ~ 11:21 (1시간 21분)
     └ 수단: 🚶 도보 (2분)
  📍 [활동] 11:21 ~ 13:21 | 대한민국 부산광역시 기장군 용궁길 86 (관광지)
     └ 체류: 120분
  ⬇️ [이동] 13:21 ~ 14:07 (46분)
     └ 수단: 🚌 대중교통/택시 이동 (약 46분 예상 / 직선거리 기반 추정)
  📍 [활동] 14:07 ~ 16:07 | 대한민국 부산광역시 영도구 전망로 24 태종대유원지 (관광지)
     └ 체류:

In [6]:
# 테스트 데이터
start = "부산역"
places_to_visit = [
    "해동용궁사",   # 부산 동쪽 끝 (기장)
    "태종대",       # 부산 남쪽 끝 (영도)
    "서면 롯데백화점" # 부산 중심 (진구)
]

# 1. 대중교통(transit) 모드 테스트
run_full_simulation(start, places_to_visit, transport_mode="transit")

print("\n\n" + "#"*60 + "\n\n")

# 2. 자차(driving) 모드 테스트
# (자차는 경로와 소요 시간이 대중교통과 다르게 나옵니다)
run_full_simulation(start, places_to_visit, transport_mode="driving")

🚀 [통합 테스트] 이동 모드: transit

--- 1. 경로 최적화 시작 (transit) ---
📍 입력 순서: 부산역 -> 해동용궁사 -> 태종대 -> 서면 롯데백화점
⚠️ 경로 최적화 실패 (이동 불가 구간 존재)

--- 2. 스마트 스케줄러 실행 (상세 경로 조회) ---
ERROR: 상세 경로 조회 오류: NOT_FOUND

📊 [최종 타임라인 결과]
  📍 [활동] 10:00 ~ 10:00 | 부산역 (숙소)
     └ 체류: 0분
  ⬇️ [이동] 10:00 ~ 11:21 (1시간 21분)
     └ 수단: 🚶 도보 이동 (2분)
  📍 [활동] 11:21 ~ 13:21 | 해동용궁사 (관광지)
     └ 체류: 120분
  📍 [활동] 13:31 ~ 15:31 | 태종대 (관광지)
     └ 체류: 120분
  📍 [활동] 15:41 ~ 17:41 | 서면 롯데백화점 (관광지)
     └ 체류: 120분


############################################################


🚀 [통합 테스트] 이동 모드: driving

--- 1. 경로 최적화 시작 (driving) ---
📍 입력 순서: 부산역 -> 해동용궁사 -> 태종대 -> 서면 롯데백화점
⚠️ 경로 최적화 실패 (이동 불가 구간 존재)

--- 2. 스마트 스케줄러 실행 (상세 경로 조회) ---
ERROR: 상세 경로 조회 오류: NOT_FOUND

📊 [최종 타임라인 결과]
  📍 [활동] 10:00 ~ 10:00 | 부산역 (숙소)
     └ 체류: 0분
  ⬇️ [이동] 10:00 ~ 11:21 (1시간 21분)
     └ 수단: 🚶 도보 이동 (2분)
  📍 [활동] 11:21 ~ 13:21 | 해동용궁사 (관광지)
     └ 체류: 120분
  📍 [활동] 13:31 ~ 15:31 | 태종대 (관광지)
     └ 체류: 120분
  📍 [활동] 15:41 ~ 17:41 | 서면 롯데백화점 (관광지)
     

In [4]:
import googlemaps
import os
import datetime
import re
from dotenv import load_dotenv

# 1. API 키 설정
load_dotenv()
api_key = os.getenv("GMAPS_API_KEY")
GMAPS_CLIENT = googlemaps.Client(key=api_key) if api_key else None

# 2. 상세 경로 조회 함수 (아까 수정한 최신 버전)
def get_detailed_route_debug(start_place, end_place, mode="transit"):
    print(f"\n📡 Google Maps API 요청: [{mode}] {start_place} -> {end_place}")
    
    if not GMAPS_CLIENT:
        return "API 키 없음"

    # 모드별 출발 시간 설정
    dep_time = datetime.datetime.now() if mode == "transit" else None

    try:
        # API 호출
        resp = GMAPS_CLIENT.directions(
            origin=start_place,
            destination=end_place,
            mode=mode,
            departure_time=dep_time,
            language="ko",
            region="KR"
        )
        
        if not resp:
            print("   ❌ 경로를 찾을 수 없음")
            return None

        # 결과 파싱
        leg = resp[0]['legs'][0]
        duration = leg['duration']['text']
        distance = leg['distance']['text']
        
        print(f"   ⏱️ 소요 시간: {duration}")
        print(f"   📏 이동 거리: {distance}")
        print(f"   📝 상세 경로 (Steps):")

        # 단계(Step)별 상세 정보 출력
        for i, step in enumerate(leg['steps']):
            t_mode = step['travel_mode']
            instr = step.get('html_instructions', '')
            # HTML 태그 제거
            instr_clean = re.sub(r'<[^>]+>', '', instr)
            
            if t_mode == 'TRANSIT':
                transit = step['transit_details']
                line = transit['line']['short_name'] if 'short_name' in transit['line'] else transit['line']['name']
                vehicle = transit['line']['vehicle']['name']
                dep = transit['departure_stop']['name']
                arr = transit['arrival_stop']['name']
                print(f"      {i+1}. [🚌 {vehicle}] {line}: {dep} 승차 -> {arr} 하차 ({step['duration']['text']})")
            
            elif t_mode == 'WALKING':
                print(f"      {i+1}. [🚶 도보] {step['duration']['text']} 이동")
                
            elif t_mode == 'DRIVING':
                print(f"      {i+1}. [🚗 운전] {instr_clean} ({step['duration']['text']})")
                
            else:
                print(f"      {i+1}. [{t_mode}] {instr_clean}")

    except Exception as e:
        print(f"   ❌ 에러 발생: {e}")

In [5]:
start = "부산역"
end = "광안리 해수욕장"

# 1. 대중교통 (Transit)
# 특징: 버스 번호, 지하철 호선, 정류장 이름이 나옴
get_detailed_route_debug(start, end, mode="transit")

print("-" * 60)

# 2. 자차 운전 (Driving)
# 특징: "우회전", "고가도로 진입" 같은 내비게이션 지시가 나옴
get_detailed_route_debug(start, end, mode="driving")

print("-" * 60)

# 3. 도보 (Walking)
# 특징: 거리가 멀면 엄청난 시간(2시간 등)이 나옴. 횡단보도, 지하도 정보가 나옴
get_detailed_route_debug(start, end, mode="walking")


📡 Google Maps API 요청: [transit] 부산역 -> 광안리 해수욕장
   ⏱️ 소요 시간: 43분
   📏 이동 거리: 9.6 km
   📝 상세 경로 (Steps):
      1. [🚶 도보] 2분 이동
      2. [🚌 버스] 41: 부산역 승차 -> 서호병원 하차 (29분)
      3. [🚶 도보] 4분 이동
------------------------------------------------------------

📡 Google Maps API 요청: [driving] 부산역 -> 광안리 해수욕장
   ❌ 경로를 찾을 수 없음
------------------------------------------------------------

📡 Google Maps API 요청: [walking] 부산역 -> 광안리 해수욕장
   ❌ 경로를 찾을 수 없음


In [6]:
import time
import asyncio
from functools import lru_cache

# --- A. 환경 설정 및 클라이언트 ---
# (실제 환경에서는 src/config.py에서 GMAPS_CLIENT를 사용합니다.)
# (여기서는 시뮬레이션을 위해 간단한 sleep 함수와 Mocking을 사용합니다.)

# [Mock] DB 로드 함수 (실제 FAISS 로드 시 3초 걸린다고 가정)
def mock_load_faiss_index():
    """실제 DB 로드 지연을 시뮬레이션합니다."""
    print("   [⏳ DB 로딩 시작...]")
    time.sleep(3) 
    print("   [✅ DB 로딩 완료]")
    return "FAISS_DB_INSTANCE_SUCCESS"

# [Mock] API 호출 함수 (실제 Geocoding API 호출 시 0.5초 걸린다고 가정)
async def mock_api_call(place_name):
    """비동기 API I/O 대기 시간을 시뮬레이션합니다."""
    await asyncio.sleep(0.5) 
    return f"좌표: {place_name}"

# [Timing Helper]
def measure_time(func):
    start = time.time()
    result = func()
    end = time.time()
    print(f"⏱️ 실행 시간: {end - start:.2f} 초")
    return result

print("✅ 테스트 환경 준비 완료.")

✅ 테스트 환경 준비 완료.


In [7]:
# 1. 캐싱이 없다고 가정 (기존 시스템)
print("\n--- 1. [기존 시스템] 비효율적인 DB 로드 (매번 3초 소요) ---")
measure_time(mock_load_faiss_index)
measure_time(mock_load_faiss_index) # 두 번째 호출에서도 다시 3초 소요

# ----------------------------------------------------

# 2. 캐싱 로직 적용 (st.cache_resource 시뮬레이션)
# (get_cached_faiss_index를 대신하는 테스트용 함수 정의)
@lru_cache(maxsize=1)
def get_cached_db_mock():
    """캐싱 로직이 적용된 DB 로드."""
    print("   [💾 캐시 미스: DB 로딩 시작...]")
    time.sleep(3)
    print("   [✅ 캐시 미스: DB 로딩 완료]")
    return "CACHED_FAISS_DB"

print("\n--- 2. [최적화 시스템] 캐싱된 DB 로드 (두 번째는 즉시 반환) ---")

# 첫 번째 호출 (캐시 미스)
measure_time(get_cached_db_mock) 

# 두 번째 호출 (캐시 히트)
# 💥 여기서 시간이 거의 0초가 나와야 성공입니다.
measure_time(get_cached_db_mock)


--- 1. [기존 시스템] 비효율적인 DB 로드 (매번 3초 소요) ---
   [⏳ DB 로딩 시작...]
   [✅ DB 로딩 완료]
⏱️ 실행 시간: 3.01 초
   [⏳ DB 로딩 시작...]
   [✅ DB 로딩 완료]
⏱️ 실행 시간: 3.01 초

--- 2. [최적화 시스템] 캐싱된 DB 로드 (두 번째는 즉시 반환) ---
   [💾 캐시 미스: DB 로딩 시작...]
   [✅ 캐시 미스: DB 로딩 완료]
⏱️ 실행 시간: 3.00 초
⏱️ 실행 시간: 0.00 초


'CACHED_FAISS_DB'

In [9]:
# [Cell 1] 설정 및 함수 정의 (이 코드를 실행해야 실제 비교가 가능합니다.)

import time
import asyncio
from typing import List
import os
from dotenv import load_dotenv
import googlemaps

# --- 환경 설정 (실제 환경과 동일하게 클라이언트 초기화) ---
load_dotenv()
api_key = os.getenv("GMAPS_API_KEY") 
GMAPS_CLIENT = googlemaps.Client(key=api_key) if api_key else None
KOREAN_CITIES_AND_PROVINCES = ["서울", "부산", "대구", "인천"] # 테스트를 위해 일부만 사용

# --- 비동기 get_coordinates 함수 정의 (위에서 수정한 코드) ---
# NOTE: 이 코드는 GMAPS_CLIENT와 KOREAN_CITIES_AND_PROVINCES 변수에 접근합니다.
# (위의 async def get_coordinates 함수 코드를 복사해서 여기에 붙여넣으세요)
async def get_coordinates(location_name: str):
    # ... (복사된 get_coordinates 함수 본문) ...
    if not GMAPS_CLIENT: return None, None
    
    def _geocode_sync(query):
        return GMAPS_CLIENT.geocode(query, language='ko')

    final_res = None
    
    try:
        # 1차 시도: 원본 검색
        res = await asyncio.to_thread(_geocode_sync, location_name)
        
        # 2차 시도: 실패 시 전국 주요 광역시/도 컨텍스트로 재시도
        if not res:
            for province in KOREAN_CITIES_AND_PROVINCES:
                if province in location_name:
                    continue

                retry_query = f"{province} {location_name}"
                res = await asyncio.to_thread(_geocode_sync, retry_query)
                
                if res:
                    final_res = res
                    break
        
        # 결과 반환
        if final_res:
            loc = final_res[0]['geometry']['location']
            return loc['lat'], loc['lng']
            
    except Exception as e:
        print(f"DEBUG: 좌표 변환 실패 ({location_name}): {e}")
    return None, None


# --- 테스트 함수 ---
async def run_performance_test():
    # '서면'은 컨텍스트가 필요하므로 실패 시 재시도가 일어나 시간이 걸립니다.
    # '부산역'은 1차에 성공하므로 시간이 짧게 걸립니다.
    places = ["서면", "광안리", "부산역"] 

    # 1. 비동기 없이 순차적으로 실행 (Task 1 -> Task 2 -> Task 3)
    print("\n--- 1. [순차 실행 시뮬레이션] ---")
    start_sync = time.time()
    
    # 동기적 순서대로 await (내부적으로는 to_thread가 순차적으로 실행됨)
    for place in places:
        # 각 Task가 끝날 때까지 기다립니다.
        await get_coordinates(place) 
    
    end_sync = time.time()
    print(f"❌ 순차 실행 시간: {end_sync - start_sync:.2f} 초 (각 API 호출 시간을 합산)")

    # 2. asyncio.gather로 병렬 실행 (Task 1, 2, 3 동시 실행)
    print("\n--- 2. [병렬 실행] asyncio.gather ---")
    start_async = time.time()
    
    # 모든 Task를 리스트에 담아 동시에 실행
    await asyncio.gather(
        get_coordinates(places[0]),
        get_coordinates(places[1]),
        get_coordinates(places[2])
    )
    
    end_async = time.time()
    print(f"✅ 병렬 실행 시간: {end_async - start_async:.2f} 초 (가장 긴 Task 시간만큼 소요)")

# 실행
await run_performance_test()


--- 1. [순차 실행 시뮬레이션] ---
❌ 순차 실행 시간: 0.55 초 (각 API 호출 시간을 합산)

--- 2. [병렬 실행] asyncio.gather ---
✅ 병렬 실행 시간: 0.22 초 (가장 긴 Task 시간만큼 소요)


In [1]:
# [Cell 1] 설정 및 라이브러리 (tools.py와 동일한 환경을 가정)
import time
import asyncio
from src.tools import get_detailed_route # 👈 이제 async 함수
from src.tools import get_weather_forecast # 👈 이제 async 함수

# --- 테스트 함수 ---
async def run_performance_test():
    # 여러 I/O 작업 (경로 계산)을 동시에 테스트할 장소들
    places_a = ("부산역", "광안리 해수욕장")
    places_b = ("해운대 해수욕장", "송정 해수욕장")
    places_c = ("서면", "남포동")

    # 1. 순차 실행 (Async/Await 사용, 하지만 순서대로 호출)
    print("\n--- 1. [순차 실행] 3가지 경로/날씨 조회 ---")
    start_sync = time.time()
    
    # Task 1: 경로 계산
    await get_detailed_route(places_a[0], places_a[1], mode="transit")
    
    # Task 2: 날씨 조회
    await get_weather_forecast(places_b[0], "오늘")
    
    # Task 3: 다른 경로 계산
    await get_detailed_route(places_c[0], places_c[1], mode="transit")
    
    end_sync = time.time()
    print(f"❌ 순차 실행 시간: {end_sync - start_sync:.2f} 초 (각 API 호출 시간이 합산됨)")

    # 2. 병렬 실행 (Asyncio.gather로 동시 호출)
    print("\n--- 2. [병렬 실행] 3가지 경로/날씨 조회 ---")
    start_async = time.time()
    
    # Task를 리스트에 담아 동시에 실행
    await asyncio.gather(
        get_detailed_route(places_a[0], places_a[1], mode="transit"),
        get_weather_forecast(places_b[0], "오늘"),
        get_detailed_route(places_c[0], places_c[1], mode="transit")
    )
    
    end_async = time.time()
    # 💥 이 시간이 순차 실행 시간보다 현저히 짧아야 성공입니다.
    print(f"✅ 병렬 실행 시간: {end_async - start_async:.2f} 초 (가장 오래 걸린 Task 시간에 수렴)")

# 실행
await run_performance_test()

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


FAISS 인덱스를 로드합니다...
FAISS 인덱스 로드가 완료되었습니다.


ImportError: cannot import name 'get_detailed_route' from 'src.tools' (/Users/seoungmun/Documents/work/3-2/project/travle_agent4/src/tools.py)

In [ ]:
# [Cell 1] 기본 설정
import sys
import os
import random
import json
import asyncio
import pandas as pd
from tqdm.asyncio import tqdm
import nest_asyncio

# 주피터 노트북의 비동기 충돌 방지
nest_asyncio.apply()

# .env 로드 (API KEY)
from dotenv import load_dotenv
load_dotenv()

# 프로젝트 경로 설정
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# 모듈 임포트
from src.tools import find_and_select_best_place, resolve_admin_region
from src.region_cut_fuzz import normalize_region_name

print("✅ 환경 설정 완료")

In [ ]:
# [Cell 2] 테스트 데이터 생성

# 1. 테스트용 지역 데이터 (입력값 -> 예상되는 핵심 행정구역 키워드)
# 비공식 지명(POI)과 공식 행정구역을 섞어서 정의
LOCATION_DATA = {
    # [비공식/POI -> 예상 매칭 키워드]
    "부산 해운대": {"dest": "부산", "expect": "해운대"},
    "광안리": {"dest": "부산", "expect": "수영"},
    "서면": {"dest": "부산", "expect": "부산진"},
    "전포 카페거리": {"dest": "부산", "expect": "부산진"},
    "흰여울문화마을": {"dest": "부산", "expect": "영도"},
    "홍대": {"dest": "서울", "expect": "마포"},
    "강남역": {"dest": "서울", "expect": "강남"},
    "성수동": {"dest": "서울", "expect": "성동"},
    "익선동": {"dest": "서울", "expect": "종로"},
    "한옥마을": {"dest": "전주", "expect": "완산"},
    "황리단길": {"dest": "경주", "expect": "경주"},
    
    # [공식 행정구역]
    "제주시": {"dest": "제주", "expect": "제주"},
    "강릉": {"dest": "강원", "expect": "강릉"},
    "여수": {"dest": "전남", "expect": "여수"},
    "서울 강남구": {"dest": "서울", "expect": "강남"},
    "부산 수영구": {"dest": "부산", "expect": "수영"}
}

# 2. 랜덤 변수들
GROUP_TYPES = ["친구", "연인", "가족", "혼자", "직장동료"]
PREFERENCES = ["맛집 탐방", "인스타 감성", "힐링/휴식", "액티비티", "가성비", "럭셔리"]
QUERIES = ["점심 맛집", "분위기 좋은 카페", "저녁 술집", "관광지 추천", "오션뷰 숙소"]

def generate_test_cases(n=1000):
    test_cases = []
    keys = list(LOCATION_DATA.keys())
    
    for _ in range(n):
        # 랜덤 선택
        loc_input = random.choice(keys)
        meta = LOCATION_DATA[loc_input]
        
        case = {
            "input_location": loc_input,       # 사용자가 입력한 지명 (예: 광안리)
            "destination": meta["dest"],       # 여행 전체 목적지 (예: 부산)
            "expected_keyword": meta["expect"],# 결과에 꼭 있어야 할 단어 (예: 수영)
            "group": random.choice(GROUP_TYPES),
            "preference": random.choice(PREFERENCES),
            "query_core": random.choice(QUERIES)
        }
        
        # 실제 검색어 조합 (예: "친구와 함께가는 인스타 감성 점심 맛집")
        case["full_query"] = f"{case['group']}와 함께가는 {case['preference']} {case['query_core']}"
        test_cases.append(case)
        
    return test_cases

# 데이터 생성 확인 (5개만 미리보기)
TOTAL_TEST_COUNT = 1000 # ⚠️ API 비용 주의: 테스트할 때는 10~50개로 먼저 해보세요!
mock_data = generate_test_cases(TOTAL_TEST_COUNT)
print(f"🎲 {len(mock_data)}개의 테스트 케이스 생성 완료")
print(mock_data[:3])

In [ ]:
# [Cell 3] 테스트 실행 엔진

async def run_single_test(case):
    results = {
        "input": case["input_location"],
        "query": case["full_query"],
        "status": "FAIL",
        "resolved_region": "",
        "search_result_name": "",
        "search_result_address": "",
        "match_success": False,
        "error": ""
    }
    
    try:
        # 1. [검증 포인트 1] 지역명 변환 테스트
        # 비공식 지명(광안리) -> 공식 행정구역(부산 수영구) 변환 확인
        resolved = await resolve_admin_region(case["input_location"], case["destination"])
        results["resolved_region"] = resolved
        
        # 2. [검증 포인트 2] 실제 검색 테스트
        user_info = f"모임:{case['group']}, 선호:{case['preference']}"
        
        search_json = await find_and_select_best_place.ainvoke({
            "query": case["full_query"],
            "destination": case["destination"],
            "anchor": "", # 초기 검색이라 앵커 없음
            "exclude_places": [],
            "user_info": user_info
        })
        
        # 3. 결과 파싱
        try:
            data = json.loads(search_json)
            results["search_result_name"] = data.get("name", "N/A")
            results["search_result_address"] = data.get("address", "")
            results["status"] = "SUCCESS"
            
            # 4. [검증 포인트 3] 정합성 체크
            # 변환된 행정구역이나 예상 키워드가 실제 결과 주소에 포함되어 있는지 확인
            expected = case["expected_keyword"]
            # 주소에 '수영' 또는 '광안' 등이 포함되어 있는지
            if expected in results["search_result_address"] or expected in resolved:
                results["match_success"] = True
            
        except json.JSONDecodeError:
            results["error"] = "JSON Parsing Error"
            results["search_result_name"] = str(search_json)[:50]

    except Exception as e:
        results["error"] = str(e)
        
    return results

# 실행 함수
async def run_bulk_tests(test_cases):
    results_list = []
    # tqdm으로 진행률 표시
    for case in tqdm(test_cases, desc="🔍 검색 기능 점검 중"):
        res = await run_single_test(case)
        results_list.append(res)
    return results_list

# ⚠️ 주의: 1000개 실행 시 Google Maps API 및 OpenAI 비용이 발생합니다.
# 일단 10개만 먼저 돌려보시는 것을 추천합니다.
LIMIT = 20  # 전체 다 하려면 len(mock_data)로 변경
print(f"🚀 테스트 시작 (총 {LIMIT}건 실행)...")

final_results = await run_bulk_tests(mock_data[:LIMIT])
print("✅ 테스트 종료")

In [2]:
import sys
import os
from dotenv import load_dotenv

# 1. 프로젝트 루트 경로 잡기
sys.path.append(os.path.abspath("..")) 

# 2. .env 파일 로드
load_dotenv()

# 3. 도구 가져오기
from src.tools import AVAILABLE_TOOLS

# 디버깅: 사용 가능한 도구 목록 확인
print("사용 가능한 도구 목록:", list(AVAILABLE_TOOLS.keys()))

# 4. 날씨 도구 직접 실행 테스트
target_tool_name = "get_weather_forecast"

if target_tool_name in AVAILABLE_TOOLS:
    print(f"Running {target_tool_name}...")
    
    # [수정] 도구가 'destination'을 요구하므로 키 이름을 'location'에서 'destination'으로 변경
    test_args = {"destination": "부산", "dates": "2025-12-12"} 
    
    # 실행
    try:
        weather_result = await AVAILABLE_TOOLS[target_tool_name].ainvoke(test_args)
        print("\n[날씨 도구 실행 결과]")
        print(weather_result)
    except Exception as e:
        print(f"\n[실행 중 에러 발생]: {e}")
else:
    print(f"Error: '{target_tool_name}' 도구를 찾을 수 없습니다. tools.py를 확인하세요.")

사용 가능한 도구 목록: ['find_and_select_best_place', 'plan_itinerary_timeline', 'optimize_and_get_routes', 'get_weather_forecast', 'delete_place', 'replace_place', 'confirm_and_download_pdf']
Running get_weather_forecast...

[날씨 도구 실행 결과]
[부산 (2025-12-13) 날씨 예보 (OWM)]
- 00:00 (UTC): 7.5℃, 튼구름
- 03:00 (UTC): 10.7℃, 구름조금
- 06:00 (UTC): 11.4℃, 튼구름
- 09:00 (UTC): 10.9℃, 온흐림
- 12:00 (UTC): 9.1℃, 실 비
- 15:00 (UTC): 6.6℃, 실 비
- 18:00 (UTC): 5.1℃, 튼구름
- 21:00 (UTC): 3.8℃, 구름조금



In [5]:
import asyncio
from langchain_core.messages import AIMessage
# from src.graph_flow import call_tools_node  # 실제 파일 경로에 맞게

# 1. 가짜 State 생성
mock_tool_call_id = "call_12345"
target_tool_name = "get_weather_forecast" 

# [수정 포인트] args의 키를 'location' -> 'destination'으로 변경
mock_message = AIMessage(
    content="",
    tool_calls=[{
        "name": target_tool_name,
        "args": {"destination": "제주도", "dates": "내일"}, 
        "id": mock_tool_call_id
    }]
)

# 초기 상태 정의
mock_state = {
    "messages": [mock_message],
    "itinerary": [],
    "current_anchor": None,
    "total_days": 2,
    "current_weather": "Old Weather Data", 
    "group_type": "친구",
    "style": "활동적",
    "preference": "맛집"
}

print(f"변경 전 날씨 상태: {mock_state['current_weather']}")

# 2. call_tools_node 실행
print("\n--- 노드 실행 중... ---")
# call_tools_node 함수가 정의되어 있거나 import 되어 있어야 합니다.
new_state = await call_tools_node(mock_state)

# 3. 결과 확인
print("\n[실행 결과 확인]")
updated_weather = new_state.get("current_weather")
print(f"변경 후 날씨 상태: {updated_weather}")

# 검증
if updated_weather != "Old Weather Data" and updated_weather:
    print("✅ 성공! 날씨 정보가 State에 정상적으로 업데이트되었습니다.")
else:
    print("❌ 실패. 날씨 정보가 업데이트되지 않았습니다.")
    
# 로그 확인
print("\n[생성된 도구 메시지]")
for msg in new_state['messages']:
    print(msg)

변경 전 날씨 상태: Old Weather Data

--- 노드 실행 중... ---

[실행 결과 확인]
변경 후 날씨 상태: Old Weather Data
❌ 실패. 날씨 정보가 업데이트되지 않았습니다.

[생성된 도구 메시지]
content='[제주도 (2025-12-13) 날씨 예보 (OWM)]\n- 00:00 (UTC): 4.8℃, 튼구름\n- 03:00 (UTC): 7.2℃, 구름조금\n- 06:00 (UTC): 5.4℃, 실 비\n- 09:00 (UTC): 3.1℃, 보통 비\n- 12:00 (UTC): 1.4℃, 실 비\n- 15:00 (UTC): 0.9℃, 실 비\n- 18:00 (UTC): 0.5℃, 실 비\n- 21:00 (UTC): -0.1℃, 가벼운 눈\n' tool_call_id='call_12345'


In [7]:
import sys
import os
import asyncio
from dotenv import load_dotenv

# 1. 환경 설정
# (경로 설정은 기존과 동일하게 유지)
current_dir = os.getcwd()
sys.path.append(current_dir)

load_dotenv()

# 2. 테스트할 함수 임포트
try:
    from src.tools import find_and_select_best_place
    print("✅ 함수 임포트 성공!")
except ImportError as e:
    print(f"❌ 임포트 실패: {e}")

# 3. 테스트 실행 함수
async def run_test():
    # --- [테스트 케이스 설정] ---
    test_destination = "서울 강남"
    test_query = "분위기 좋은 곳"
    test_category = "점심"  # 🚨 '점심' -> '식당' 변환 테스트
    test_user_info = "스타일: 맛집 탐방, 동행: 연인"
    
    print(f"\n🚀 테스트 시작: {test_destination} / 카테고리: '{test_category}'")
    
    # [수정 포인트] 직접 호출 대신 .ainvoke() 사용 + 딕셔너리 전달
    input_args = {
        "query": test_query,
        "destination": test_destination,
        "anchor": "",
        "exclude_places": [],
        "user_info": test_user_info,
        "category_filter": test_category
    }
    
    # 도구 실행 (비동기)
    result_json = await find_and_select_best_place.ainvoke(input_args)
    
    print("\n📊 --- [최종 결과 JSON] ---")
    print(result_json)

# 4. 실행
await run_test()

✅ 함수 임포트 성공!

🚀 테스트 시작: 서울 강남 / 카테고리: '점심'

--- [DEBUG] find_and_select_best_place 호출 ---
DEBUG: 🗺️ 행정구역 식별 시도: '서울 강남 분위기 좋은 곳'
DEBUG: ✅ 변환 성공: '서울 강남 분위기 좋은 곳' -> '서울특별시 강남구'
DEBUG: 📍 기준점 좌표 조회: '서울특별시 강남구'
DEBUG: 🔍 1차 검색 시도 (선호 포함): '서울특별시 강남구 분위기 좋은 곳 스타일: 맛집 탐방, 동행: 연인 점심'
DEBUG: 🔍 벡터 DB 검색 시도: '서울특별시 강남구 분위기 좋은 곳 스타일: 맛집 탐방, 동행: 연인 점심'
DEBUG: 🐢 무거운 라이브러리(Langchain, FAISS) 로딩 시작 (함수 내부)...
DEBUG: 🚀 임베딩 모델 및 FAISS 인덱스 로딩 중...
DEBUG: ✅ Vector DB(Faiss) 로딩 완료!
DEBUG: ⚙️ 필터 적용 - 지역키워드:['서울', '강남'] / 카테고리:점심
DEBUG: 🎯 1차 후보군 수: 8
✅ 최종 추천: 빌즈 강남

📊 --- [최종 결과 JSON] ---
{"name": "빌즈 강남", "type": "식당", "description": "맛집 탐방을 즐기는 연인과 함께라면, 맛보다는 분위기가 압도적인 빌즈 강남에서 로맨틱한 데이트를 즐기기 좋아요.", "address": "서울특별시 강남구", "coordinates": null}


In [1]:
import sys
import os
from dotenv import load_dotenv

# .env 파일 로드 (OpenAI, Google Maps API 키 등)
load_dotenv()

# 현재 주피터 파일이 프로젝트 루트에 있다고 가정하거나, src 상위 폴더를 경로에 추가
if os.path.abspath('.') not in sys.path:
    sys.path.append(os.path.abspath('.'))

print("환경 설정 완료")

환경 설정 완료


In [2]:
from src.tools import query_gen_chain

# 테스트용 입력 데이터
test_input = {
    "target_region": "부산",
    "query": "카페",
    "user_info": "사람이 별로 없고 조용하며, 바다가 보이는 뷰가 좋은 곳을 선호해",
    "category_filter": "카페"
}

print(f"--- [입력] 사용자 취향: {test_input['user_info']} ---")

# 체인 실행 (동기 방식 invoke 사용)
try:
    generated_queries = query_gen_chain.invoke(test_input)
    print("\n--- [결과] 생성된 멀티 쿼리 ---")
    print(generated_queries)
    
    # 리스트로 변환 확인
    query_list = [q.strip() for q in generated_queries.split(',') if q.strip()]
    print(f"\n파싱된 리스트: {query_list}")
    
except Exception as e:
    print(f"오류 발생: {e}")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


--- [입력] 사용자 취향: 사람이 별로 없고 조용하며, 바다가 보이는 뷰가 좋은 곳을 선호해 ---

--- [결과] 생성된 멀티 쿼리 ---
부산 카페, 부산 조용한 오션뷰 카페, 부산 오션뷰 카페

파싱된 리스트: ['부산 카페', '부산 조용한 오션뷰 카페', '부산 오션뷰 카페']


In [3]:
import json
from src.tools import find_and_select_best_place

# 검색 도구 실행 (비동기 호출)
print("--- 검색 도구 실행 시작 ---")

# find_and_select_best_place는 LangChain 도구이므로 ainvoke로 호출
# (내부에서 print 문으로 디버그 로그가 찍힐 것입니다)
result_json = await find_and_select_best_place.ainvoke({
    "query": "카페",
    "destination": "부산",
    "user_info": "조용하고 바다가 보이는", 
    "category_filter": "카페",
    "anchor": "해운대" # 기준점 (선택 사항)
})

print("\n--- [최종 결과] ---")
print(result_json)

# 결과 파싱 확인
if result_json:
    data = json.loads(result_json)
    print(f"\n장소명: {data.get('name')}")
    print(f"추천 이유: {data.get('description')}")

--- 검색 도구 실행 시작 ---

--- [DEBUG] find_and_select_best_place 호출 ---
DEBUG: 🗺️ 행정구역 식별 시도: '부산 해운대'
DEBUG: ✅ 변환 성공: '해운대' -> '부산광역시 해운대구'
DEBUG: 📍 기준점 좌표 조회: '해운대'
DEBUG: 🧠 생성된 멀티 쿼리: ['부산광역시 해운대구 카페', '부산광역시 해운대구 조용한 오션뷰 카페', '부산광역시 해운대구 오션뷰 카페']
DEBUG: 🔍 벡터 DB 검색 시도: '부산광역시 해운대구 카페'
DEBUG: 🐢 무거운 라이브러리(Langchain, FAISS) 로딩 시작 (함수 내부)...
DEBUG: 🚀 임베딩 모델 및 FAISS 인덱스 로딩 중...
DEBUG: ✅ Vector DB(Faiss) 로딩 완료!
DEBUG: 🔍 벡터 DB 검색 시도: '부산광역시 해운대구 조용한 오션뷰 카페'
DEBUG: 🔍 벡터 DB 검색 시도: '부산광역시 해운대구 오션뷰 카페'


I0000 00:00:1765591304.135504 10828513 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


DEBUG: ⚙️ 필터 적용 - 지역키워드:['부산', '해운대'] / 카테고리:카페
DEBUG: 🎯 필터링 후 후보군 수: 5
✅ 최종 추천: 스타벅스 해운대점

--- [최종 결과] ---
{"name": "스타벅스 해운대점", "type": "카페", "description": "조용히 바다를 감상하고 싶은 당신에게, 스타벅스 해운대점은 탁 트인 해변 뷰를 선사해 편안한 시간을 보내기 좋아요.", "address": "부산광역시 해운대구", "coordinates": null}

장소명: 스타벅스 해운대점
추천 이유: 조용히 바다를 감상하고 싶은 당신에게, 스타벅스 해운대점은 탁 트인 해변 뷰를 선사해 편안한 시간을 보내기 좋아요.


In [1]:
import sys
import os
import asyncio
from dotenv import load_dotenv

# 1. 환경 변수 로드 (API 키 등)
# .env 파일이 있는 경로에서 실행해야 합니다.
load_dotenv()

# 2. 프로젝트 루트 경로 추가 (모듈 import를 위해)
if os.path.abspath('.') not in sys.path:
    sys.path.append(os.path.abspath('.'))

# 3. 모듈 임포트
# GMAPS_CLIENT가 제대로 생성되었는지 확인하기 위해 config도 가져옵니다.
try:
    from src.config import GMAPS_CLIENT
    from src.tools import get_detailed_route
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")
    print("현재 경로가 프로젝트 루트인지 확인해주세요.")

# 4. Google Maps 클라이언트 상태 점검
print("--- [설정 점검] ---")
if GMAPS_CLIENT:
    print("✅ Google Maps Client가 정상적으로 로드되었습니다.")
else:
    print("❌ Google Maps Client가 None입니다. .env 파일의 GOOGLE_MAPS_API_KEY를 확인하세요.")
    print("   (이 상태면 무조건 30분으로 나옵니다)")

# 5. 실제 거리 계산 테스트 함수
async def test_route():
    if not GMAPS_CLIENT:
        return

    start_place = "강남역"
    end_place = "코엑스"
    
    print(f"\n--- [경로 검색 테스트] {start_place} ➡️ {end_place} ---")
    
    # 도구 함수 호출
    try:
        result = await get_detailed_route(start_place, end_place, mode="transit")
        
        if result:
            print("\n✅ API 호출 성공!")
            print(f"⏱️ 소요 시간: {result.get('duration')} (값: {result.get('duration_value')}초)")
            print(f"📏 이동 거리: {result.get('distance')}")
            print(f"🚌 이동 수단: {result.get('steps')}")
            
            # 30분(1800초) 고정 문제 확인
            if result.get('duration_value') == 1800 and result.get('steps') == ["직선거리"]:
                print("\n⚠️ 경고: API 호출은 되었으나 Fallback(직선거리) 로직이 실행되었습니다.")
        else:
            print("\n❌ 결과가 None입니다. (API 호출 실패)")
            
    except Exception as e:
        print(f"\n❌ 에러 발생: {e}")

# 6. 실행 (주피터 노트북 환경)
await test_route()

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- [설정 점검] ---
✅ Google Maps Client가 정상적으로 로드되었습니다.

--- [경로 검색 테스트] 강남역 ➡️ 코엑스 ---
DEBUG: 🗺️ 경로 검색 요청: 강남역 -> 코엑스

✅ API 호출 성공!
⏱️ 소요 시간: 14분 (값: 833초)
📏 이동 거리: 3.8 km
🚌 이동 수단: ['[지하철] 2호선', '🚶 도보']


In [1]:
import asyncio
import json
from langchain_core.messages import AIMessage
from src.graph_flow import call_tools_node  # 수정된 코드가 있는 파일 import

# 1. 테스트용 가짜 상태(Mock State) 만들기
# 상황: 이미 일정에 '별양집'이 있는데, 에이전트가 마음에 안 들어서 '대려도'를 새로 찾은 상황
mock_state = {
    "messages": [
        AIMessage(
            content="",
            tool_calls=[{
                "name": "find_and_select_best_place",
                "args": {"query": "강남 맛집"},
                "id": "call_test_123"
            }]
        )
    ],
    "itinerary": [
        # 기존에 잘못 들어간 장소 (같은 날, 같은 식당 카테고리)
        {"day": 1, "name": "별양집", "type": "식당", "description": "고기집"}
    ],
    "current_anchor": "강남역",
    "destination": "서울",
    "group_type": "연인",
    "style": "느긋한",
    "preference": "양식 선호",
    "dialog_stage": "planning"
}

# 2. 실제 도구(AVAILABLE_TOOLS)를 '가짜 결과'를 뱉도록 잠시 바꿔치기 (Mocking)
# (실제 API 호출 없이 로직만 테스트하기 위함입니다. 실제 호출하려면 이 부분 건너뛰세요)
from langchain_core.tools import tool
from src.tools import AVAILABLE_TOOLS

@tool
def mock_find_place(query: str, **kwargs):
    """테스트용 가짜 도구: 대려도를 리턴함"""
    return json.dumps({
        "name": "대려도",
        "type": "식당", # 별양집과 같은 '식당' 타입 -> 교체되어야 함!
        "description": "중식당",
        "address": "서울 강남구"
    }, ensure_ascii=False)

# 도구 덮어쓰기 (테스트 후엔 커널 재시작 필요)
AVAILABLE_TOOLS["find_and_select_best_place"] = mock_find_place

# 3. 함수 실행 및 결과 확인
print("--- [테스트 시작] ---")
result = await call_tools_node(mock_state)

print("\n--- [테스트 결과] ---")
final_itinerary = result['itinerary']
print(f"최종 일정 개수: {len(final_itinerary)}")
for item in final_itinerary:
    print(f"- {item['name']} ({item['type']})")

# 4. 검증 포인트
# '별양집'이 사라지고 '대려도'만 남아있어야 성공입니다.
if len(final_itinerary) == 1 and final_itinerary[0]['name'] == "대려도":
    print("\n✅ 성공: 기존 장소가 삭제되고 새 장소로 교체되었습니다.")
else:
    print("\n❌ 실패: 교체 로직이 작동하지 않았습니다.")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


--- [테스트 시작] ---
DEBUG: 🔄 재검색 결과 반영 - 기존 '별양집' 삭제 후 '대려도' 교체

--- [테스트 결과] ---
최종 일정 개수: 1
- 대려도 (식당)

✅ 성공: 기존 장소가 삭제되고 새 장소로 교체되었습니다.


In [2]:
import sys
import os
import asyncio
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage

# 1. 환경 설정
load_dotenv()
if os.path.abspath('.') not in sys.path:
    sys.path.append(os.path.abspath('.'))

# 2. 그래프 빌더 가져오기
# (수정하신 로직이 적용된 파일이어야 합니다)
try:
    from src.graph_flow import build_graph
except ImportError as e:
    print(f"❌ 임포트 실패: {e}")

# 3. 그래프 컴파일
app = build_graph()

# 4. [핵심] 첫 페이지 입력 정보 시뮬레이션 (Mock State)
# Streamlit에서 사용자가 입력했다고 가정하는 데이터입니다.
initial_inputs = {
    # 사용자 입력 정보
    "destination": "서울 강남",
    "dates": "2025-12-25",
    "total_days": 1,  # 1일(당일치기) 테스트
    "group_type": "연인",
    "style": "럭셔리하고 여유로운",
    "preference": "분위기 좋은 양식 레스토랑과 조용한 카페, 실내 데이트",
    "current_weather": "맑음",
    
    # 초기 상태 설정
    "dialog_stage": "planning",
    "messages": [HumanMessage(content="여행 계획 짜줘")], # 시작 트리거 메시지
    "itinerary": [],
    "show_pdf_button": False
}

print(f"--- 🏁 테스트 시작: {initial_inputs['destination']} ({initial_inputs['total_days']}일) ---")
print("⏳ 에이전트가 일정을 생성 중입니다... (약 1~2분 소요될 수 있음)")

try:
    # 5. 그래프 실행 (비동기 호출)
    # ainvoke를 사용하면 그래프가 'END'에 도달할 때까지 알아서 루프를 돕니다.
    final_state = await app.ainvoke(initial_inputs)

    print("\n--- 🎉 실행 완료! 최종 결과 확인 ---")
    
    # 6. 결과 검증
    itinerary = final_state.get('itinerary', [])
    
    print(f"✅ 총 일정 개수: {len(itinerary)}개")
    print("-" * 50)
    for idx, item in enumerate(itinerary):
        # 장소 정보 출력
        if item.get('type') == 'move':
            print(f"   ⬇️  [이동] {item.get('transport_detail', '이동')} ({item.get('duration_text', '')})")
        else:
            print(f"[{idx+1}] Day {item.get('day')}: {item.get('name')} ({item.get('type')})")
            print(f"     설명: {item.get('description', '')[:50]}...")
    print("-" * 50)

    # 7. 종료 조건 확인 (PDF 버튼)
    if final_state.get('show_pdf_button'):
        print("\n✅ PDF 다운로드 버튼: 활성화됨 (정상 종료)")
    else:
        print("\n⚠️ PDF 다운로드 버튼: 활성화 안됨 (중간에 멈춤)")
        print("마지막 메시지:", final_state['messages'][-1].content)

except Exception as e:
    print(f"\n❌ 실행 중 에러 발생: {e}")

--- 🏁 테스트 시작: 서울 강남 (1일) ---
⏳ 에이전트가 일정을 생성 중입니다... (약 1~2분 소요될 수 있음)

--- 🎉 실행 완료! 최종 결과 확인 ---
✅ 총 일정 개수: 0개
--------------------------------------------------
--------------------------------------------------

⚠️ PDF 다운로드 버튼: 활성화 안됨 (중간에 멈춤)
마지막 메시지: 어디로 여행 가시나요? 며칠 동안 여행하실 계획이신가요?


In [3]:
import sys
import os
import asyncio
import json
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage

# 1. 환경 설정
load_dotenv()
if os.path.abspath('.') not in sys.path:
    sys.path.append(os.path.abspath('.'))

# 2. 그래프 빌더 가져오기
try:
    from src.graph_flow import build_graph
except ImportError as e:
    print(f"❌ 임포트 실패: {e}")

# 3. 그래프 컴파일
app = build_graph()

# ==============================================================================
# [시나리오 설정]
# 상황: 1일차 계획이 이미 잡혀있음.
# 사용자 요청: "별양집(고기집)이 마음에 안 들어서 '중식당'으로 바꾸고 싶어함"
# ==============================================================================

# 4. 가상의 '현재 일정' 데이터 (이미 PlannerAgent가 만든 상태라고 가정)
existing_itinerary = [
    {
        "day": 1,
        "type": "식당",
        "name": "별양집", # <--- 사용자가 바꾸고 싶어하는 곳
        "description": "유명한 양대창 전문점입니다.",
        "address": "서울 강남구"
    },
    {
        "day": 1,
        "type": "카페",
        "name": "디올카페",
        "description": "청담동의 고급스러운 카페입니다.",
        "address": "서울 강남구"
    },
    {
        "day": 1,
        "type": "관광지",
        "name": "가로수길",
        "description": "쇼핑과 산책하기 좋은 거리입니다.",
        "address": "서울 강남구"
    }
]

# 5. 입력 상태 (Mock State) 생성
# 'dialog_stage'를 'editing'으로 설정하거나, 라우터가 '대신' 키워드를 감지하도록 유도
edit_inputs = {
    # 기존 정보 유지
    "destination": "서울 강남",
    "dates": "2025-12-25",
    "total_days": 1,
    "group_type": "연인",
    "style": "맛있는거 먹기",
    "preference": "중식을 좋아함",
    
    # [핵심] 이미 존재하는 일정 넣어주기
    "itinerary": existing_itinerary,
    
    # [핵심] 수정 요청 메시지
    "messages": [
        # 이전 대화 기록 (생략 가능하지만 문맥을 위해 추가)
        AIMessage(content="1일차 일정을 짰습니다. 별양집, 디올카페, 가로수길 어떠세요?"),
        
        # 사용자 수정 요청
        HumanMessage(content="별양집 대신에 맛있는 중식당으로 바꿔줘") 
    ],
    
    # 명시적으로 수정 단계임을 알리거나(선택), 라우터에 맡김
    # "dialog_stage": "editing" 
}

print(f"--- 🔄 수정(Editing) 테스트 시작 ---")
print(f"📋 변경 전 일정: {[item['name'] for item in existing_itinerary]}")
print(f"🗣️ 사용자 요청: '{edit_inputs['messages'][-1].content}'")
print("⏳ 에이전트가 수정을 진행 중입니다... (약 1분 소요)")

try:
    # 6. 그래프 실행
    final_state_edit = await app.ainvoke(edit_inputs)

    print("\n--- 🎉 수정 완료! 결과 확인 ---")
    
    # 7. 결과 비교
    old_names = [item['name'] for item in existing_itinerary]
    new_itinerary = final_state_edit.get('itinerary', [])
    new_names = [item['name'] for item in new_itinerary if item.get('type') != 'move'] # 이동 제외하고 장소만 비교
    
    print(f"🔴 [Before] {old_names}")
    print(f"🟢 [After]  {new_names}")
    
    # 검증 로직
    if "별양집" not in new_names and len(new_names) == 3:
        # 중식당(대려도 등)이 들어갔는지 확인
        new_place = set(new_names) - set(old_names)
        if new_place:
            print(f"\n✅ 성공: '별양집'이 삭제되고 '{list(new_place)[0]}'(으)로 교체되었습니다.")
        else:
            print("\n⚠️ 경고: 별양집은 삭제됐는데 새 장소가 안 보입니다.")
    else:
        print("\n❌ 실패: 장소가 제대로 교체되지 않았습니다.")

    # 8. 최종 메시지 확인 (EditorAgent가 뭐라 했는지)
    print("\n🤖 에이전트 답변:")
    print(final_state_edit['messages'][-1].content)

except Exception as e:
    print(f"\n❌ 실행 중 에러 발생: {e}")

--- 🔄 수정(Editing) 테스트 시작 ---
📋 변경 전 일정: ['별양집', '디올카페', '가로수길']
🗣️ 사용자 요청: '별양집 대신에 맛있는 중식당으로 바꿔줘'
⏳ 에이전트가 수정을 진행 중입니다... (약 1분 소요)
DEBUG: 🔄 수정 요청 감지 -> EditorAgent로 전환
DEBUG: 🔄 재검색 결과 반영 - 기존 '대려도' 삭제 후 '대려도' 교체
DEBUG: 🔄 재검색 결과 반영 - 기존 '대려도' 삭제 후 '대려도' 교체
DEBUG: 🔄 재검색 결과 반영 - 기존 '대려도' 삭제 후 '대려도' 교체
DEBUG: 🔄 재검색 결과 반영 - 기존 '대려도' 삭제 후 '대려도' 교체

--- 🎉 수정 완료! 결과 확인 ---
🔴 [Before] ['별양집', '디올카페', '가로수길']
🟢 [After]  ['디올카페', '가로수길', '대려도']

✅ 성공: '별양집'이 삭제되고 '대려도'(으)로 교체되었습니다.

🤖 에이전트 답변:
[{'type': 'text', 'text': "죄송합니다. '별양집' 대신 '대려도'를 점심 식당으로 변경했습니다. 하지만 저녁 식당을 찾으려고 여러 번 시도했지만, 계속해서 '대려도'만 추천되고 있습니다. 현재 시스템으로는 '대려도' 외에 다른 저녁 식당을 찾을 수 없는 상황입니다.\n\n이대로 일정을 확정할까요? 아니면 다른 대안을 찾아볼까요?", 'extras': {'signature': 'CvMOAXLI2nwRBBb7LKhGZeuP/tX24PlWYLWpBbX1pjsxRDmlZ4X8eHPYAMRlESMfkUxp7JxnDLLbiMfVskE4DXSGOrVk91XhqfFEHNoWU2MFzamBsMFERvUR6DKsH5Wimf4O1IF9av4wrh4PHVEtRflD9was0cZvbaIVXcCzDM/43JQsR9a38Zmmg9m95ohgYRIwR9JIFskkAC42QNE3ZrdtlqdKnYuYVM8yvXS4DXz07ICzH7ybrRnRFIhZMrtNXW2rciLNyhaDEHcq/urEdDFN9VZ/9oR

In [4]:
import asyncio
import json
from typing import TypedDict, Annotated, List, Dict, Any
from langchain_core.messages import AIMessage, ToolMessage, BaseMessage

# ==============================================================================
# 1. [환경 설정] 테스트를 위한 가짜(Mock) 객체 및 타입 정의
# ==============================================================================

# 가짜 AgentState 정의 (실제와 동일 구조)
class AgentState(TypedDict):
    messages: List[BaseMessage]
    itinerary: List[Dict]
    current_anchor: str
    destination: str
    group_type: str
    style: str
    preference: str
    dialog_stage: str
    show_pdf_button: bool

# 테스트용 가짜 도구 클래스
class MockTool:
    def __init__(self, name, return_value_generator):
        self.name = name
        self.generator = return_value_generator

    async def ainvoke(self, args):
        # 입력된 args에 따라 동적으로 결과 반환 (테스트 시나리오별 대응)
        return self.generator(self.name, args)

# ==============================================================================
# 2. [핵심] 수정된 call_tools_node 함수 (검증 대상)
# ==============================================================================
# (사용자님이 파일에 적용하실 그 코드입니다. 여기서 직접 정의하여 테스트합니다.)

async def call_tools_node_target(state: AgentState, mock_tools: Dict):
    last_message = state['messages'][-1]
    new_itinerary = state.get('itinerary', []).copy()
    new_anchor = state.get('current_anchor')
    
    user_info_str = "테스트 유저 정보"
    current_stage = state.get("dialog_stage", "planning")
    show_pdf = state.get("show_pdf_button", False)
    
    tool_calls = last_message.tool_calls
    tool_outputs = []

    # [헬퍼 1] 카테고리 정규화 및 비교
    def get_category_group(type_str):
        t = str(type_str).replace("맛집", "식당").replace("음식점", "식당")
        if any(x in t for x in ["식당", "요리", "레스토랑", "반점", "회관", "고기", "뷔페"]): return "식당"
        if any(x in t for x in ["카페", "커피", "베이커리", "디저트", "찻집"]): return "카페"
        if any(x in t for x in ["관광", "명소", "여행", "공원", "박물관", "미술관", "산책", "전시"]): return "관광지"
        return "기타"

    def is_same_category(type1, type2):
        return get_category_group(type1) == get_category_group(type2)

    # [헬퍼 2] 지능형 일정 정렬 (핵심 로직)
    def reorganize_itinerary(items):
        if not items: return []
        days = sorted(list(set(item.get('day', 1) for item in items)))
        final_list = []
        for day in days:
            day_items = [x for x in items if x.get('day', 1) == day]
            restaurants = [x for x in day_items if get_category_group(x.get('type')) == "식당"]
            cafes = [x for x in day_items if get_category_group(x.get('type')) == "카페"]
            tourists = [x for x in day_items if get_category_group(x.get('type')) == "관광지"]
            others = [x for x in day_items if get_category_group(x.get('type')) == "기타"]
            
            # 순서: 식당(점심) -> 카페 -> 관광지 -> 기타 -> 식당(저녁)
            sorted_day = []
            if restaurants: sorted_day.append(restaurants.pop(0)) # 점심
            sorted_day.extend(cafes)
            sorted_day.extend(tourists)
            sorted_day.extend(others)
            sorted_day.extend(restaurants) # 저녁 (나머지 식당)
            final_list.extend(sorted_day)
        return final_list

    async def call_tool_executor(tool_call):
        tool_name = tool_call.get("name")
        args = tool_call.get("args", {})
        
        # 실제 환경에서는 args 주입 로직이 있지만 테스트에선 생략 가능
        
        if tool_name in mock_tools:
            try:
                res = await mock_tools[tool_name].ainvoke(args)
                raw_output = str(res)
                # 시스템 인스트럭션 주입 확인용 로직은 유지
                llm_content = raw_output
                if tool_name == "plan_itinerary_timeline":
                    llm_content += "\n\n[SYSTEM INSTRUCTION: 확인용]"
                return ToolMessage(tool_call_id=tool_call['id'], content=llm_content), tool_name, raw_output
            except Exception as e:
                return ToolMessage(tool_call_id=tool_call['id'], content=f"Error: {e}"), tool_name, None
        return None, None, None

    results = await asyncio.gather(*(call_tool_executor(t) for t in tool_calls))

    for tool_message, tool_name, raw_json_output in results:
        if tool_message:
            tool_outputs.append(tool_message)
            
            if raw_json_output:
                if tool_name == "find_and_select_best_place":
                    try:
                        item_json = json.loads(raw_json_output)
                        if item_json.get('name') == "추천 장소 없음":
                            print("  DEBUG LOG: ⚠️ 검색 실패 - 일정 추가 안 함")
                        else:
                            # 덮어쓰기 로직
                            if new_itinerary:
                                last_item = new_itinerary[-1]
                                if (item_json.get('day', 1) == last_item.get('day', 1) and 
                                    is_same_category(item_json.get('type'), last_item.get('type'))):
                                    if item_json.get('name') == last_item.get('name'):
                                        print(f"  DEBUG LOG: ⏭️ 중복 장소 무시 ('{item_json['name']}')")
                                    else:
                                        print(f"  DEBUG LOG: 🔄 교체 ('{last_item['name']}' -> '{item_json['name']}')")
                                        new_itinerary.pop()
                                        new_itinerary.append(item_json)
                                        new_anchor = item_json.get('name')
                                        continue 

                            if not any(x.get('name') == item_json.get('name') for x in new_itinerary):
                                current_places = [i for i in new_itinerary if i.get('type') != 'move']
                                day_to_add = 1
                                if current_places:
                                    day_to_add = current_places[-1].get('day', 1)
                                item_json['day'] = day_to_add
                                new_itinerary.append(item_json)
                                new_anchor = item_json.get('name')
                                print(f"  DEBUG LOG: ✅ 추가 ('{item_json['name']}')")
                                
                    except Exception as e: print(f"Error: {e}")

                elif tool_name in ["delete_place", "replace_place"]:
                    try:
                        action_data = json.loads(raw_json_output)
                        target = action_data.get('place_name') or action_data.get('old')
                        if target:
                            new_itinerary = [i for i in new_itinerary if target not in i.get('name', '')]
                            print(f"  DEBUG LOG: 🗑️ 삭제 ('{target}')")
                    except: pass
                
                elif tool_name == "confirm_and_download_pdf":
                    show_pdf = True

    # 마지막 정렬 수행
    new_itinerary = reorganize_itinerary(new_itinerary)

    return {
        "messages": tool_outputs, 
        "itinerary": new_itinerary,
        "show_pdf_button": show_pdf,
        "dialog_stage": current_stage
    }

# ==============================================================================
# 3. [검증 시나리오] 4가지 케이스 테스트
# ==============================================================================

async def run_tests():
    print("🚦 [테스트 시작] 수정된 call_tools_node 기능 검증\n")

    # --- 공통 Mock Tool Generator ---
    def tool_behavior(name, args):
        # 테스트 시나리오별로 args['mock_return']에 들어있는 JSON을 반환하도록 설정
        return args.get('mock_return', "{}")

    mock_tools = {
        "find_and_select_best_place": MockTool("find_and_select_best_place", tool_behavior),
        "delete_place": MockTool("delete_place", tool_behavior),
    }

    # --------------------------------------------------------------------------
    # Case 1: [기능 검증] 뒤죽박죽 추가 시 자동 정렬 (Reorganize)
    # 목표: 관광지 -> 카페 -> 식당 순으로 추가해도, [식당 -> 카페 -> 관광지]로 정렬되는가?
    # --------------------------------------------------------------------------
    print("▶️ Case 1: 일정 자동 정렬 테스트 (관광지 -> 카페 -> 식당 순 입력)")
    
    # 입력: 관광지 -> 카페 -> 식당 순으로 도구 호출 결과가 들어왔다고 가정 (혹은 기존 일정에 쌓임)
    state_1 = {
        "messages": [AIMessage(content="", tool_calls=[])], # 실제 호출은 아래서 수동 처리
        "itinerary": [], # 빈 상태에서 시작
        "dialog_stage": "planning"
    }
    
    # 1. 관광지 추가
    msg1 = AIMessage(content="", tool_calls=[{
        "name": "find_and_select_best_place", 
        "args": {"mock_return": json.dumps({"name": "가로수길", "type": "관광지", "day": 1})}, 
        "id": "1"
    }])
    state_1['messages'] = [msg1]
    res1 = await call_tools_node_target(state_1, mock_tools)
    state_1['itinerary'] = res1['itinerary']

    # 2. 카페 추가
    msg2 = AIMessage(content="", tool_calls=[{
        "name": "find_and_select_best_place", 
        "args": {"mock_return": json.dumps({"name": "디올카페", "type": "카페", "day": 1})}, 
        "id": "2"
    }])
    state_1['messages'] = [msg2]
    res2 = await call_tools_node_target(state_1, mock_tools)
    state_1['itinerary'] = res2['itinerary']

    # 3. 식당 추가 (점심)
    msg3 = AIMessage(content="", tool_calls=[{
        "name": "find_and_select_best_place", 
        "args": {"mock_return": json.dumps({"name": "창고43", "type": "식당", "day": 1})}, 
        "id": "3"
    }])
    state_1['messages'] = [msg3]
    final_res_1 = await call_tools_node_target(state_1, mock_tools)
    
    print(f"  결과 일정: {[item['name'] for item in final_res_1['itinerary']]}")
    if [item['name'] for item in final_res_1['itinerary']] == ['창고43', '디올카페', '가로수길']:
        print("  ✅ Pass: 순서가 [식당, 카페, 관광지]로 자동 정렬됨")
    else:
        print("  ❌ Fail: 정렬 실패")
    print("-" * 50)


    # --------------------------------------------------------------------------
    # Case 2: [기능 검증] 덮어쓰기 (Overwrite)
    # 목표: '창고43'이 있는 상태에서 '대려도(식당)'를 찾으면 교체되는가?
    # --------------------------------------------------------------------------
    print("▶️ Case 2: 재검색 시 덮어쓰기 테스트")
    state_2 = {
        "messages": [],
        "itinerary": [{"name": "창고43", "type": "식당", "day": 1}], # 기존 일정
        "dialog_stage": "planning"
    }
    
    # 대려도(식당) 검색 결과 도착
    msg_overwrite = AIMessage(content="", tool_calls=[{
        "name": "find_and_select_best_place", 
        "args": {"mock_return": json.dumps({"name": "대려도", "type": "식당", "day": 1})}, 
        "id": "4"
    }])
    state_2['messages'] = [msg_overwrite]
    
    final_res_2 = await call_tools_node_target(state_2, mock_tools)
    
    items_2 = [item['name'] for item in final_res_2['itinerary']]
    print(f"  결과 일정: {items_2}")
    if "창고43" not in items_2 and "대려도" in items_2 and len(items_2) == 1:
        print("  ✅ Pass: 기존 식당이 삭제되고 새 식당으로 교체됨")
    else:
        print("  ❌ Fail: 덮어쓰기 실패")
    print("-" * 50)


    # --------------------------------------------------------------------------
    # Case 3: [기능 검증] 중복 장소 무시 (Ignore Duplicate)
    # 목표: '대려도'가 있는데 또 '대려도'를 찾으면 무시하는가?
    # --------------------------------------------------------------------------
    print("▶️ Case 3: 동일 장소 중복 추가 방지 테스트")
    state_3 = {
        "messages": [],
        "itinerary": [{"name": "대려도", "type": "식당", "day": 1}],
        "dialog_stage": "planning"
    }
    
    # 대려도 또 검색됨
    msg_dup = AIMessage(content="", tool_calls=[{
        "name": "find_and_select_best_place", 
        "args": {"mock_return": json.dumps({"name": "대려도", "type": "식당", "day": 1})}, 
        "id": "5"
    }])
    state_3['messages'] = [msg_dup]
    
    final_res_3 = await call_tools_node_target(state_3, mock_tools)
    
    items_3 = [item['name'] for item in final_res_3['itinerary']]
    print(f"  결과 일정: {items_3}")
    if len(items_3) == 1 and items_3[0] == "대려도":
        print("  ✅ Pass: 리스트가 늘어나지 않고 기존 상태 유지됨")
    else:
        print("  ❌ Fail: 중복 처리가 안됨")
    print("-" * 50)


    # --------------------------------------------------------------------------
    # Case 4: [예외 처리] "추천 장소 없음" 방어
    # 목표: 검색 실패 메시지가 일정에 추가되지 않는가?
    # --------------------------------------------------------------------------
    print("▶️ Case 4: '추천 장소 없음' 예외 처리 테스트")
    state_4 = {
        "messages": [],
        "itinerary": [{"name": "가로수길", "type": "관광지", "day": 1}],
        "dialog_stage": "planning"
    }
    
    msg_fail = AIMessage(content="", tool_calls=[{
        "name": "find_and_select_best_place", 
        "args": {"mock_return": json.dumps({"name": "추천 장소 없음", "type": "정보없음"})}, 
        "id": "6"
    }])
    state_4['messages'] = [msg_fail]
    
    final_res_4 = await call_tools_node_target(state_4, mock_tools)
    
    items_4 = [item['name'] for item in final_res_4['itinerary']]
    print(f"  결과 일정: {items_4}")
    if "추천 장소 없음" not in items_4 and len(items_4) == 1:
        print("  ✅ Pass: 실패 메시지가 일정에 추가되지 않음")
    else:
        print("  ❌ Fail: 이상한 데이터가 추가됨")
    print("-" * 50)

# 실행
await run_tests()

🚦 [테스트 시작] 수정된 call_tools_node 기능 검증

▶️ Case 1: 일정 자동 정렬 테스트 (관광지 -> 카페 -> 식당 순 입력)
  DEBUG LOG: ✅ 추가 ('가로수길')
  DEBUG LOG: ✅ 추가 ('디올카페')
  DEBUG LOG: ✅ 추가 ('창고43')
  결과 일정: ['창고43', '디올카페', '가로수길']
  ✅ Pass: 순서가 [식당, 카페, 관광지]로 자동 정렬됨
--------------------------------------------------
▶️ Case 2: 재검색 시 덮어쓰기 테스트
  DEBUG LOG: 🔄 교체 ('창고43' -> '대려도')
  결과 일정: ['대려도']
  ✅ Pass: 기존 식당이 삭제되고 새 식당으로 교체됨
--------------------------------------------------
▶️ Case 3: 동일 장소 중복 추가 방지 테스트
  DEBUG LOG: ⏭️ 중복 장소 무시 ('대려도')
  결과 일정: ['대려도']
  ✅ Pass: 리스트가 늘어나지 않고 기존 상태 유지됨
--------------------------------------------------
▶️ Case 4: '추천 장소 없음' 예외 처리 테스트
  DEBUG LOG: ⚠️ 검색 실패 - 일정 추가 안 함
  결과 일정: ['가로수길']
  ✅ Pass: 실패 메시지가 일정에 추가되지 않음
--------------------------------------------------


In [5]:
import asyncio
import json
from typing import TypedDict, List, Dict, Any
from langchain_core.messages import AIMessage, ToolMessage, BaseMessage

# ==============================================================================
# 1. [환경 설정] Mock 객체 및 수정된 함수 정의
# ==============================================================================

class AgentState(TypedDict):
    messages: List[BaseMessage]
    itinerary: List[Dict]
    current_anchor: str
    destination: str
    group_type: str
    style: str
    preference: str
    dialog_stage: str
    show_pdf_button: bool

class MockTool:
    def __init__(self, name, return_value_generator):
        self.name = name
        self.generator = return_value_generator
    async def ainvoke(self, args):
        return self.generator(self.name, args)

# --- [검증 대상] 수정된 call_tools_node (새로운 기능 포함 버전) ---
async def call_tools_node_target(state: AgentState, mock_tools: Dict):
    last_message = state['messages'][-1]
    new_itinerary = state.get('itinerary', []).copy()
    new_anchor = state.get('current_anchor')
    
    current_stage = state.get("dialog_stage", "planning")
    show_pdf = state.get("show_pdf_button", False)
    
    tool_calls = last_message.tool_calls
    tool_outputs = []

    # (헬퍼 함수들 생략 없이 포함)
    def get_category_group(type_str):
        t = str(type_str).replace("맛집", "식당").replace("음식점", "식당")
        if any(x in t for x in ["식당", "요리", "레스토랑", "반점", "회관", "고기", "뷔페"]): return "식당"
        if any(x in t for x in ["카페", "커피", "베이커리", "디저트", "찻집"]): return "카페"
        if any(x in t for x in ["관광", "명소", "여행", "공원", "박물관", "미술관", "산책", "전시"]): return "관광지"
        return "기타"

    def is_same_category(type1, type2):
        return get_category_group(type1) == get_category_group(type2)

    def reorganize_itinerary(items):
        if not items: return []
        days = sorted(list(set(item.get('day', 1) for item in items)))
        final_list = []
        for day in days:
            day_items = [x for x in items if x.get('day', 1) == day]
            restaurants = [x for x in day_items if get_category_group(x.get('type')) == "식당"]
            cafes = [x for x in day_items if get_category_group(x.get('type')) == "카페"]
            tourists = [x for x in day_items if get_category_group(x.get('type')) == "관광지"]
            others = [x for x in day_items if get_category_group(x.get('type')) == "기타"]
            
            sorted_day = []
            if restaurants: sorted_day.append(restaurants.pop(0)) # 점심
            sorted_day.extend(cafes)
            sorted_day.extend(tourists)
            sorted_day.extend(others)
            sorted_day.extend(restaurants) # 저녁
            final_list.extend(sorted_day)
        return final_list

    async def call_tool_executor(tool_call):
        tool_name = tool_call.get("name")
        args = tool_call.get("args", {})
        
        if tool_name in mock_tools:
            try:
                res = await mock_tools[tool_name].ainvoke(args)
                raw_output = str(res)
                # 시스템 메시지 주입 로직 유지
                llm_content = raw_output
                if tool_name == "plan_itinerary_timeline":
                    llm_content += "\n\n[SYSTEM INSTRUCTION: 완료]"
                return ToolMessage(tool_call_id=tool_call['id'], content=llm_content), tool_name, raw_output
            except Exception as e:
                return ToolMessage(tool_call_id=tool_call['id'], content=f"Error: {e}"), tool_name, None
        return None, None, None

    results = await asyncio.gather(*(call_tool_executor(t) for t in tool_calls))

    for tool_message, tool_name, raw_json_output in results:
        if tool_message:
            tool_outputs.append(tool_message)
            if raw_json_output:
                if tool_name == "find_and_select_best_place":
                    try:
                        item_json = json.loads(raw_json_output)
                        if item_json.get('name') == "추천 장소 없음": pass
                        else:
                            # 덮어쓰기 로직
                            if new_itinerary:
                                last_item = new_itinerary[-1]
                                if (item_json.get('day', 1) == last_item.get('day', 1) and 
                                    is_same_category(item_json.get('type'), last_item.get('type'))):
                                    if item_json.get('name') != last_item.get('name'):
                                        new_itinerary.pop()
                                        new_itinerary.append(item_json)
                                        continue
                            # 일반 추가
                            if not any(x.get('name') == item_json.get('name') for x in new_itinerary):
                                current_places = [i for i in new_itinerary if i.get('type') != 'move']
                                day_to_add = 1
                                if current_places: day_to_add = current_places[-1].get('day', 1)
                                item_json['day'] = day_to_add
                                new_itinerary.append(item_json)
                                new_anchor = item_json.get('name')
                    except: pass

                elif tool_name in ["delete_place", "replace_place"]:
                    try:
                        action_data = json.loads(raw_json_output)
                        target = action_data.get('place_name') or action_data.get('old')
                        if target:
                            new_itinerary = [i for i in new_itinerary if target not in i.get('name', '')]
                    except: pass

                elif tool_name == "plan_itinerary_timeline":
                    try:
                        new_itinerary = json.loads(raw_json_output)
                    except: pass
                
                elif tool_name == "confirm_and_download_pdf":
                    show_pdf = True

    new_itinerary = reorganize_itinerary(new_itinerary)

    return {
        "messages": tool_outputs, 
        "itinerary": new_itinerary,
        "show_pdf_button": show_pdf,
        "dialog_stage": current_stage
    }

# ==============================================================================
# 2. [기존 기능 검증 시나리오]
# ==============================================================================

async def run_regression_tests():
    print("🚦 [기존 기능 무결성 테스트] 시작\n")

    # Mock 도구 동작 정의
    def tool_behavior(name, args):
        if name == "find_and_select_best_place":
            # 무조건 args로 요청한 이름의 장소를 리턴
            q = args.get('query', '장소')
            return json.dumps({"name": q, "type": "관광지", "day": 1})
        elif name == "delete_place":
            return json.dumps({"place_name": args.get('place_name')})
        elif name == "plan_itinerary_timeline":
            # 입력된 리스트에 시간을 추가해서 리턴하는 척 함
            itinerary = args.get('itinerary', [])
            for item in itinerary:
                item['time'] = "10:00" # 가짜 시간 추가
            return json.dumps(itinerary)
        elif name == "confirm_and_download_pdf":
            return "PDF OK"
        return "{}"

    mock_tools = {
        k: MockTool(k, tool_behavior) 
        for k in ["find_and_select_best_place", "delete_place", "plan_itinerary_timeline", "confirm_and_download_pdf"]
    }

    # --------------------------------------------------------------------------
    # Test 1: [기본] 장소 추가 (find_and_select_best_place)
    # --------------------------------------------------------------------------
    print("▶️ Test 1: 장소 일반 추가 기능")
    state_1 = {"messages": [], "itinerary": [], "dialog_stage": "planning"}
    msg1 = AIMessage(content="", tool_calls=[{
        "name": "find_and_select_best_place", 
        "args": {"query": "남산타워"}, 
        "id": "1"
    }])
    state_1['messages'] = [msg1]
    
    res1 = await call_tools_node_target(state_1, mock_tools)
    items1 = [i['name'] for i in res1['itinerary']]
    
    if "남산타워" in items1:
        print(f"  ✅ Pass: 장소가 정상적으로 추가됨 ({items1})")
    else:
        print("  ❌ Fail: 장소 추가 실패")
    print("-" * 50)

    # --------------------------------------------------------------------------
    # Test 2: [기본] 장소 삭제 (delete_place)
    # --------------------------------------------------------------------------
    print("▶️ Test 2: 장소 삭제 기능")
    state_2 = {"messages": [], "itinerary": [{"name": "남산타워", "type": "관광지", "day": 1}], "dialog_stage": "planning"}
    msg2 = AIMessage(content="", tool_calls=[{
        "name": "delete_place", 
        "args": {"place_name": "남산타워"}, 
        "id": "2"
    }])
    state_2['messages'] = [msg2]
    
    res2 = await call_tools_node_target(state_2, mock_tools)
    items2 = [i['name'] for i in res2['itinerary']]
    
    if "남산타워" not in items2 and len(items2) == 0:
        print("  ✅ Pass: 장소가 정상적으로 삭제됨")
    else:
        print(f"  ❌ Fail: 삭제 실패 ({items2})")
    print("-" * 50)

    # --------------------------------------------------------------------------
    # Test 3: [기본] 타임라인 계획 (plan_itinerary_timeline)
    # --------------------------------------------------------------------------
    print("▶️ Test 3: 타임라인 계획 및 반영")
    # 상황: 장소는 있는데 시간이 없는 상태
    raw_itinerary = [{"name": "남산타워", "type": "관광지", "day": 1}]
    state_3 = {"messages": [], "itinerary": raw_itinerary, "dialog_stage": "planning"}
    msg3 = AIMessage(content="", tool_calls=[{
        "name": "plan_itinerary_timeline", 
        "args": {"itinerary": raw_itinerary}, 
        "id": "3"
    }])
    state_3['messages'] = [msg3]
    
    res3 = await call_tools_node_target(state_3, mock_tools)
    updated_itinerary = res3['itinerary']
    
    # Mock 도구가 시간을 추가해줬는지 확인
    if updated_itinerary and updated_itinerary[0].get('time') == "10:00":
        print("  ✅ Pass: 타임라인 결과가 일정에 반영됨")
    else:
        print("  ❌ Fail: 타임라인 업데이트 실패")
    print("-" * 50)

    # --------------------------------------------------------------------------
    # Test 4: [기본] PDF 다운로드 확정 (confirm_and_download_pdf)
    # --------------------------------------------------------------------------
    print("▶️ Test 4: PDF 다운로드 활성화")
    state_4 = {"messages": [], "itinerary": [], "show_pdf_button": False}
    msg4 = AIMessage(content="", tool_calls=[{
        "name": "confirm_and_download_pdf", 
        "args": {}, 
        "id": "4"
    }])
    state_4['messages'] = [msg4]
    
    res4 = await call_tools_node_target(state_4, mock_tools)
    
    if res4['show_pdf_button'] is True:
        print("  ✅ Pass: PDF 버튼 상태가 True로 변경됨")
    else:
        print("  ❌ Fail: PDF 상태 변경 실패")
    print("-" * 50)

# 실행
await run_regression_tests()

🚦 [기존 기능 무결성 테스트] 시작

▶️ Test 1: 장소 일반 추가 기능
  ✅ Pass: 장소가 정상적으로 추가됨 (['남산타워'])
--------------------------------------------------
▶️ Test 2: 장소 삭제 기능
  ✅ Pass: 장소가 정상적으로 삭제됨
--------------------------------------------------
▶️ Test 3: 타임라인 계획 및 반영
  ✅ Pass: 타임라인 결과가 일정에 반영됨
--------------------------------------------------
▶️ Test 4: PDF 다운로드 활성화
  ✅ Pass: PDF 버튼 상태가 True로 변경됨
--------------------------------------------------


In [6]:
import re
import os
from fpdf import FPDF
from fpdf.enums import XPos, YPos

# ==============================================================================
# 1. [검증 대상] 데이터 정규화 함수 (핵심 로직)
# ==============================================================================
def _normalize_itinerary_for_pdf(itinerary, total_days=None):
    print("🔄 데이터 정규화 시작...")
    norm = []
    for item in itinerary:
        if not isinstance(item, dict): continue
        it = item.copy()
        
        # 1. Day 값 강제 추출 ("Day 1" -> 1, "2" -> 2 등)
        raw_day = it.get('day', 1)
        try:
            found = re.search(r'(\d+)', str(raw_day))
            day = int(found.group(1)) if found else 1
        except:
            day = 1
            
        # 2. total_days 범위 체크
        if total_days:
            try:
                td = int(total_days)
                if day > td: day = td 
                if day < 1: day = 1
            except: pass
            
        it['day'] = day
        it['name'] = it.get('name') or it.get('장소명') or "이름 없음"
        it['type'] = it.get('type') or it.get('category') or "activity"
        it['description'] = it.get('description', '')
        if 'reviews' not in it: it['reviews'] = []
        
        norm.append(it)
    
    # 3. 정렬 (Day -> 시작시간)
    norm.sort(key=lambda x: (x['day'], x.get('start', '23:59')))
    
    print("✅ 정규화 완료! (샘플 데이터 확인):")
    for item in norm:
        print(f"   - Day {item['day']} | {item['start']} | {item['name']}")
    return norm

# ==============================================================================
# 2. [검증 대상] PDF 생성 함수 (Streamlit 의존성 제거됨)
# ==============================================================================
def create_itinerary_pdf(itinerary, destination, dates, weather, total_days):
    pdf = FPDF()
    pdf.add_page()
    
    # 폰트 설정 (주피터 환경에 폰트가 없으면 Arial 사용)
    font_path = 'NanumGothic.ttf' # 같은 폴더에 폰트 파일이 있어야 한글이 나옵니다.
    bold_font_path = 'NanumGothicBold.ttf'
    has_korean_font = False

    try:
        if os.path.exists(font_path):
            pdf.add_font('NanumGothic', '', font_path)
            if os.path.exists(bold_font_path): pdf.add_font('NanumGothic', 'B', bold_font_path)
            else: pdf.add_font('NanumGothic', 'B', font_path)
            pdf.set_font('NanumGothic', '', 12)
            has_korean_font = True
            print("✅ 한글 폰트 로드 성공")
        else:
            pdf.set_font('Arial', '', 12)
            print("⚠️ 한글 폰트 없음 (Arial 사용 - 한글 깨질 수 있음)")
    except Exception as e:
        pdf.set_font('Arial', '', 12)
        print(f"⚠️ 폰트 에러: {e}")

    # 제목
    pdf.set_font_size(24)
    pdf.cell(0, 20, text=f"Travel Plan: {destination}", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
    pdf.set_font_size(12)
    pdf.cell(0, 10, text=f"Date: {dates}", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
    pdf.ln(10)

    # 정규화 실행
    clean_itinerary = _normalize_itinerary_for_pdf(itinerary, total_days)

    # 출력 루프
    for day_num in range(1, total_days + 1):
        items_today = [item for item in clean_itinerary if item['day'] == day_num]
        
        if day_num > 1: pdf.ln(10)
        
        # Day 헤더
        pdf.set_font_size(18)
        if has_korean_font: pdf.set_font('NanumGothic', 'B', 18)
        pdf.cell(0, 15, text=f"Day {day_num}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        
        pdf.set_font_size(11)
        if has_korean_font: pdf.set_font('NanumGothic', '', 11)

        if not items_today:
            pdf.cell(0, 10, text="  - No Schedule", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
            continue

        for item in items_today:
            # 시간, 이름 출력
            time_str = f"[{item.get('start', '??:??')}-{item.get('end', '??:??')}]"
            name_str = item.get('name', 'No Name')
            
            # (한글 폰트 없을 때를 대비해 영어로 출력 테스트)
            if not has_korean_font:
                name_str = "Place Name (Korean Font Missing)"
            
            main_text = f"  * {time_str} {name_str} ({item.get('type')})"
            pdf.cell(0, 8, text=main_text, new_x=XPos.LMARGIN, new_y=YPos.NEXT)

        pdf.ln(5)
        pdf.line(pdf.get_x(), pdf.get_y(), pdf.get_x() + 190, pdf.get_y()) # 구분선

    return bytes(pdf.output())

# ==============================================================================
# 3. [시나리오] 가짜 데이터로 테스트 실행
# ==============================================================================

# 상황: 챗봇이 데이터를 뒤죽박죽 줬다고 가정 (문자열 "Day 1", 숫자 2, 섞임)
mock_itinerary = [
    {"day": "Day 1", "start": "10:00", "end": "11:30", "name": "해운대", "type": "관광지"},
    {"day": 2,       "start": "10:00", "end": "12:00", "name": "광안리", "type": "관광지"},
    {"day": "1",     "start": "12:00", "end": "13:00", "name": "개미집", "type": "식당"}, # Day 1 점심 (순서 섞임)
    {"day": "Day 2", "start": "13:00", "end": "14:00", "name": "카페", "type": "카페"},
]

print("🚀 PDF 생성 테스트 시작...")

# 함수 실행
pdf_data = create_itinerary_pdf(
    itinerary=mock_itinerary,
    destination="Busan",
    dates="2025-01-01",
    weather="Sunny",
    total_days=2
)

# 파일 저장
output_filename = "test_output.pdf"
with open(output_filename, "wb") as f:
    f.write(pdf_data)

print(f"\n🎉 테스트 완료! '{output_filename}' 파일을 열어서 Day 1과 Day 2가 잘 나뉘었는지 확인하세요.")

🚀 PDF 생성 테스트 시작...


TSI0 NOT subset; don't know how to subset; dropped
TSI1 NOT subset; don't know how to subset; dropped
TSI2 NOT subset; don't know how to subset; dropped
TSI3 NOT subset; don't know how to subset; dropped
TSI5 NOT subset; don't know how to subset; dropped
TSI0 NOT subset; don't know how to subset; dropped
TSI1 NOT subset; don't know how to subset; dropped
TSI2 NOT subset; don't know how to subset; dropped
TSI3 NOT subset; don't know how to subset; dropped
TSI5 NOT subset; don't know how to subset; dropped


✅ 한글 폰트 로드 성공
🔄 데이터 정규화 시작...
✅ 정규화 완료! (샘플 데이터 확인):
   - Day 1 | 10:00 | 해운대
   - Day 1 | 12:00 | 개미집
   - Day 2 | 10:00 | 광안리
   - Day 2 | 13:00 | 카페

🎉 테스트 완료! 'test_output.pdf' 파일을 열어서 Day 1과 Day 2가 잘 나뉘었는지 확인하세요.


In [1]:
# 필요 라이브러리 임포트
import os
import re
import asyncio
import pandas as pd
from typing import List, Dict, Optional
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# ---------------------------------------------------------
# [설정] 테스트를 위한 임베딩 모델 로드
# ---------------------------------------------------------
print("⏳ 임베딩 모델 로딩 중... (최초 실행 시 시간이 걸릴 수 있습니다)")
embeddings = HuggingFaceEmbeddings(
    model_name="upskyy/bge-m3-korean",
    model_kwargs={"device": "cpu"}
)
print("✅ 모델 로드 완료")


/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⏳ 임베딩 모델 로딩 중... (최초 실행 시 시간이 걸릴 수 있습니다)
✅ 모델 로드 완료


In [2]:
# =========================================================
# 1. 지역명 정규화 함수 (region_cut_fuzz.py)
# =========================================================
REGION_STRUCTURE = {
    "서울특별시": ["강남구", "서초구"], # 테스트용 일부만 정의
    "광주광역시": ["동구", "서구", "남구", "북구", "광산구"],
    "경기도": ["광주시", "성남시", "수원시"],
}

SIMPLE_ALIASES = {
    "서울": "서울특별시", "광주": "광주광역시", "경기": "경기도"
}

def normalize_region_name(user_input: str) -> str:
    if not user_input: return ""
    clean = user_input.strip()
    
    # 1. 전체 일치 확인
    if clean in REGION_STRUCTURE: return clean
    if clean in SIMPLE_ALIASES: return SIMPLE_ALIASES[clean]
    
    # 2. 복합어 처리 (예: "광주 맛집" -> "광주광역시")
    parts = clean.split()
    if len(parts) >= 1:
        head = parts[0]
        if head in REGION_STRUCTURE: return head
        if head in SIMPLE_ALIASES: return SIMPLE_ALIASES[head]
        
    return clean

# =========================================================
# 2. 수정된 필터링 로직 (tools.py) - STRICT 모드
# =========================================================
async def _filter_candidates(docs, target_region: str, category_filter: str):
    candidates = []
    
    print(f"\n🔍 [필터링 실행] 타겟 지역: '{target_region}' | 카테고리: '{category_filter}'")
    
    for doc in docs:
        name = doc.metadata.get('장소명', '이름미상')
        address = doc.metadata.get('지역', '')  # DB에 저장된 주소
        doc_cat = doc.metadata.get('카테고리', '')
        
        # [핵심] 지역명 엄격 필터링 (Strict Filtering)
        # target_region(예: 광주광역시)이 address(주소)에 포함되어야 함
        if target_region:
            # 주소에 타겟 지역명이 없으면 탈락
            if target_region not in address:
                print(f"   ❌ [제외] {name} (주소: {address}) -> 지역 불일치")
                continue
        
        # 카테고리 필터
        if category_filter:
            if category_filter == "관광지" and "관광" not in doc_cat:
                print(f"   ❌ [제외] {name} -> 카테고리 불일치 ({doc_cat})")
                continue
                
        print(f"   ✅ [통과] {name} (주소: {address})")
        candidates.append(doc)
            
    return candidates

In [3]:
# =========================================================
# 3. 테스트용 가상 벡터 DB 생성
# =========================================================
def create_mock_db():
    # 🚨 [시나리오] DB에 광주광역시와 경기도 광주시 데이터가 섞여 있음
    mock_data = [
        # 1. 광주광역시 데이터
        {"name": "무등산 국립공원", "region": "광주광역시 동구", "cat": "관광지", "desc": "광주를 대표하는 산"},
        {"name": "1913송정역시장", "region": "광주광역시 광산구", "cat": "관광지", "desc": "레트로 감성 시장"},
        {"name": "여유커피", "region": "광주광역시 서구", "cat": "카페", "desc": "분위기 좋은 카페"},
        
        # 2. 경기도 광주시 데이터 (오탐지 유발 함정 카드)
        {"name": "남한산성", "region": "경기도 광주시 남한산성면", "cat": "관광지", "desc": "역사적인 산성"},
        {"name": "화담숲", "region": "경기도 광주시 도척면", "cat": "관광지", "desc": "아름다운 수목원"},
        
        # 3. 서울 데이터
        {"name": "남산타워", "region": "서울특별시 용산구", "cat": "관광지", "desc": "서울의 랜드마크"}
    ]
    
    docs = []
    for item in mock_data:
        # DB 저장 시 정규화 로직 시뮬레이션
        norm_region = normalize_region_name(item['region'].split()[0]) # '광주광역시' or '경기도'
        full_addr = item['region'] # 원본 주소
        
        doc = Document(
            page_content=f"{item['name']} {item['desc']}",
            metadata={
                "장소명": item['name'],
                "지역": full_addr, # 메타데이터엔 풀 주소가 들어감
                "카테고리": item['cat']
            }
        )
        docs.append(doc)
    
    print(f"📁 테스트용 문서 {len(docs)}개 생성 완료")
    return FAISS.from_documents(docs, embeddings)

# DB 생성 (메모리 상)
mock_db = create_mock_db()

# =========================================================
# 4. 통합 테스트 실행 함수
# =========================================================
async def run_search_test(user_query, destination):
    print("="*60)
    print(f"🧪 [테스트 시나리오] 입력: '{user_query}' / 목적지: '{destination}'")
    
    # 1. 지역명 정규화
    if destination:
        anchor = destination
    else:
        # 목적지가 없으면 쿼리에서 추출 시도
        anchor = user_query
        
    target_region = normalize_region_name(anchor)
    print(f"1️⃣  지역 정규화 결과: '{anchor}' -> '{target_region}'")
    
    # 2. 벡터 검색 (유사도 기반 - 필터 없이 일단 다 가져옴)
    # 실제로는 여기서 filter를 걸 수도 있지만, 필터링 로직 검증을 위해 다 가져와서 파이썬단에서 거름
    raw_docs = mock_db.similarity_search(user_query, k=10)
    print(f"2️⃣  1차 검색 결과 (유사도 순, 상위 5개):")
    for d in raw_docs[:5]:
        print(f"    - {d.metadata['장소명']} ({d.metadata['지역']})")
        
    # 3. 필터링 적용 (수정된 로직)
    filtered_results = await _filter_candidates(
        raw_docs, 
        target_region=target_region, 
        category_filter="관광지" # 테스트 편의상 고정
    )
    
    print(f"\n3️⃣  최종 필터링 결과 ({len(filtered_results)}건):")
    if not filtered_results:
        print("    ❌ 결과 없음 (필터링에 의해 모두 제거됨)")
    for d in filtered_results:
        print(f"    🌟 추천: {d.metadata['장소명']} ({d.metadata['지역']})")
    print("="*60 + "\n")

📁 테스트용 문서 6개 생성 완료


In [4]:
# [Cell 2] 테스트용 데이터셋 생성 (함정 데이터 포함)
# 실제 rag_updater가 이 데이터를 어떻게 처리하는지 확인합니다.

# 🚨 시나리오: 사용자 입장에서 헷갈리는 데이터들이 섞여 있음
test_data = {
    "장소명": ["무등산 커볶", "남한산성 백숙", "송정역시장", "화담숲", "펭귄마을"],
    "지역": [
        "광주 동구 지산동",         # -> 광주광역시 (정상)
        "경기 광주 남한산성면",      # -> 경기도 (함정: '광주' 포함)
        "광주광역시 광산구",        # -> 광주광역시 (이미 정규화됨)
        "경기도 광주시 도척면",      # -> 경기도 (함정: '광주' 포함)
        "광주 남구 양림동"          # -> 광주광역시 (정상)
    ],
    "카테고리": ["카페", "식당", "관광지", "관광지", "관광지"],
    "리뷰": ["뷰가 좋아요", "맛있어요", "야시장 최고", "단풍이 예뻐요", "사진찍기 좋아요"]
}

df_test = pd.DataFrame(test_data)
print("📊 테스트 데이터 프레임:")
display(df_test)

📊 테스트 데이터 프레임:


,장소명,지역,카테고리,리뷰
0,무등산 커볶,광주 동구 지산동,카페,뷰가 좋아요
1,남한산성 백숙,경기 광주 남한산성면,식당,맛있어요
2,송정역시장,광주광역시 광산구,관광지,야시장 최고
3,화담숲,경기도 광주시 도척면,관광지,단풍이 예뻐요
4,펭귄마을,광주 남구 양림동,관광지,사진찍기 좋아요


In [6]:
# [Cell 3] 실제 [rag_updater]를 통해 DB 생성 및 메타데이터 검증
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import src.rag_updater

# 1. 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(
    model_name="upskyy/bge-m3-korean",
    model_kwargs={"device": "cpu"}
)

# 2. [검증 포인트 1] create_documents_from_df가 지역명을 제대로 '광주광역시'로 정규화하는가?
# (existing_db=None으로 설정하여 주소 검색은 스킵)
docs = src.rag_updater.create_documents_from_df(df_test, existing_db=None)

print(f"\n📂 생성된 문서 개수: {len(docs)}")
print("\n🕵️‍♀️ [메타데이터 정규화 확인]")
for doc in docs:
    region_meta = doc.metadata['지역']
    place_name = doc.metadata['장소명']
    
    # 시각적 검증: '광주' 입력이 '광주광역시'로 바뀌었는지 확인
    status = "✅ 정규화됨" if "광역시" in region_meta or "경기도" in region_meta else "❌ 정규화 실패"
    print(f" - {place_name}: {doc.metadata['지역']} ({status})")

# 3. 테스트용 인메모리 FAISS DB 생성
test_db = FAISS.from_documents(docs, embeddings)
print("\n💾 테스트용 FAISS DB 생성 완료")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)



📂 생성된 문서 개수: 4

🕵️‍♀️ [메타데이터 정규화 확인]
 - 무등산 커볶: 광주 동구 지산동 (❌ 정규화 실패)
 - 송정역시장: 광주광역시 광산구 (✅ 정규화됨)
 - 화담숲: 경기도 광주시 도척면 (✅ 정규화됨)
 - 펭귄마을: 광주 남구 양림동 (❌ 정규화 실패)

💾 테스트용 FAISS DB 생성 완료


In [8]:
# [Cell 4] 실제 [tools] 함수에 테스트 DB 주입 (Dependency Injection)
import src.tools
# 🚨 중요: src.tools가 파일에 있는 DB를 읽지 않고, 방금 만든 test_db를 읽도록 함수를 가로챕니다.
# 이렇게 해야 실제 코드를 수정하지 않고도 테스트 DB로 로직 검증이 가능합니다.
src.tools.load_faiss_index = lambda: test_db
print("💉 테스트 DB 주입 완료 (load_faiss_index 오버라이딩)")

# 이제 실제 함수 호출
async def test_real_search_function(query, destination):
    print("="*60)
    print(f"🚀 [실제 함수 테스트] Query: '{query}' / Dest: '{destination}'")
    
    # tools.find_and_select_best_place 호출
    # (JSON 문자열로 리턴되므로 확인을 위해 파싱 필요할 수 있음)
    result_json = await src.tools.find_and_select_best_place(
        query=query,
        destination=destination,
        anchor="",          # anchor는 테스트 편의상 비움
        exclude_places=[],
        user_info="친구와 여행",
        category_filter=""
    )
    
    import json
    result = json.loads(result_json)
    
    print(f"\n🏁 결과 장소명: {result.get('name')}")
    print(f"📍 결과 주소  : {result.get('address')}")
    
    # 검증 로직
    if destination == "광주":
        if "광주광역시" in result.get('address', ''):
            print("✅ TEST PASS: 광주광역시 장소가 정확히 선택됨.")
        else:
            print("❌ TEST FAIL: 엉뚱한 지역이 선택됨.")
            
    elif destination == "경기 광주":
        if "경기도" in result.get('address', ''):
             print("✅ TEST PASS: 경기도 장소가 정확히 선택됨.")
        else:
             print("❌ TEST FAIL: 엉뚱한 지역이 선택됨.")

# --- 실제 테스트 실행 ---
# Case 1: "광주" -> 무등산, 송정역 등이 나와야 함 (남한산성 X)
await test_real_search_function("관광지 추천", "광주")

# Case 2: "경기 광주" -> 화담숲, 남한산성 등이 나와야 함 (무등산 X)
await test_real_search_function("관광지 추천", "경기 광주")

💉 테스트 DB 주입 완료 (load_faiss_index 오버라이딩)
🚀 [실제 함수 테스트] Query: '관광지 추천' / Dest: '광주'


TypeError: 'StructuredTool' object is not callable

In [ ]:
import json
import sys
import os
import re

# 1. src 폴더 경로 설정 (주피터가 src를 못 찾을 경우를 대비)
sys.path.append(os.path.abspath(".."))  # 상위 폴더가 프로젝트 루트라고 가정

# 2. 사용자님의 함수 임포트 (수정한 파일이 반영되도록)
# ⚠️ 주의: graph_flow.py를 수정했다면 주피터 커널을 재시작(Restart Kernel)해야 반영됩니다!
try:
    from src.graph_flow import normalize_name, process_deletions, process_additions
    print("✅ src.graph_flow 함수 임포트 성공!")
except ImportError as e:
    print(f"❌ 임포트 실패: {e}")
    print("현재 경로:", os.getcwd())

# --- [검증 데이터 준비] ---

# 3. 아까 문제가 됐던 '해운대별밤'이 포함된 일정 데이터 (Day 2 부분만 발췌)
mock_itinerary = [
    {
        "type": "관광지", "name": "해운대", "day": 2
    },
    {   # 🚨 지워져야 할 녀석 (Index 1)
        "type": "식당", "name": "해운대별밤", "description": "부산광역시 해운대구...", "day": 2
    },
    {
        "type":"관광지", "name":"해수용장","description": "부산광역시 해운대구...", "day": 2
    }
]

# 4. LLM이 보냈다고 가정하는 도구 결과값 (Mocking)
# 상황: "해운대별밤" 삭제하고 "해운대가야밀면" 추가
mock_tool_results = [
    # (ToolMessage, ToolName, JSON_Output)
    (None, "delete_place", json.dumps({"place_name": "해운대별밤"})),
    (None, "find_and_select_best_place", json.dumps({
        "name": "해운대가야밀면", "type": "식당", "description": "밀면 맛집", "day": 2
    }))
]

# 5. 가짜 상태 객체 (process_additions 함수가 state를 요구하므로)
class MockState(dict):
    def get(self, key, default=None):
        return super().get(key, default)

mock_state = MockState({
    "dialog_stage": "editing", 
    "total_days": 2, 
    "current_anchor": None
})

# --- [검증 시작] ---

print("\n" + "="*50)
print("🚀 [Step 1] 이름 정규화(Normalize) 테스트")
print("="*50)

target_name = "해운대별밤"
db_name = "해운대별밤 부산광역시 해운대구..."
norm_target = normalize_name(target_name)
norm_db = normalize_name(db_name)

print(f"🔹 검색어 정규화: '{target_name}' -> '{norm_target}'")
print(f"🔹 DB이름 정규화: '{db_name}' -> '{norm_db}'")

if norm_target in norm_db:
    print("✅ 포함 관계 확인됨 (삭제 가능)")
else:
    print("❌ 포함 관계 아님 (삭제 실패 원인!)")


print("\n" + "="*50)
print("🚀 [Step 2] 삭제 로직(process_deletions) 실행")
print("="*50)

# 함수 호출
updated_itinerary, empty_slot_info, mod_deleted, _, _ = process_deletions(mock_tool_results, mock_itinerary)

print(f"🔹 삭제 후 리스트 길이: {len(updated_itinerary)} (기존: {len(mock_itinerary)})")
print(f"🔹 확보된 빈자리 정보(empty_slot_info): {empty_slot_info}")

if empty_slot_info and empty_slot_info['index'] == 1:
    print("✅ 성공: 정확한 인덱스(1)를 잡았습니다.")
elif empty_slot_info is None:
    print("❌ 실패: 빈자리를 못 잡았습니다. (이름 매칭 실패)")
else:
    print(f"⚠️ 주의: 인덱스를 잡긴 했는데 이상합니다. ({empty_slot_info})")


print("\n" + "="*50)
print("🚀 [Step 3] 추가 로직(process_additions) 실행")
print("="*50)

# 삭제된 리스트와 빈자리 정보를 넘겨줌
final_itinerary, _, mod_added, _ = process_additions(
    mock_tool_results, updated_itinerary, empty_slot_info, mock_state
)

print("🔹 최종 결과 리스트:")
for idx, item in enumerate(final_itinerary):
    print(f" [{idx}] {item['name']} (Day {item.get('day')})")

# 검증
target_item = final_itinerary[1] if len(final_itinerary) > 1 else None
if target_item and target_item['name'] == "해운대가야밀면":
    print("\n🎉 [최종 성공] 별밤이 사라진 자리에 밀면이 정확히 들어갔습니다!")
else:
    print("\n💥 [최종 실패] 밀면이 엉뚱한 곳에 있거나 추가되지 않았습니다.")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ src.graph_flow 함수 임포트 성공!

🚀 [Step 1] 이름 정규화(Normalize) 테스트
🔹 검색어 정규화: '해운대별밤' -> '해운대별밤'
🔹 DB이름 정규화: '해운대별밤 부산광역시 해운대구...' -> '해운대별밤부산광역시해운대구'
✅ 포함 관계 확인됨 (삭제 가능)

🚀 [Step 2] 삭제 로직(process_deletions) 실행
🔹 삭제 후 리스트 길이: 1 (기존: 2)
🔹 확보된 빈자리 정보(empty_slot_info): {'index': 0, 'day': 2}
⚠️ 주의: 인덱스를 잡긴 했는데 이상합니다. ({'index': 0, 'day': 2})

🚀 [Step 3] 추가 로직(process_additions) 실행
DEBUG: ♻️ 빈자리(Index 0)에 '해운대가야밀면' 삽입
🔹 최종 결과 리스트:
 [0] 해운대가야밀면 (Day 2)
 [1] 해운대별밤 (Day 2)

💥 [최종 실패] 밀면이 엉뚱한 곳에 있거나 추가되지 않았습니다.
